# Traces for the previous choice
Here we use the components extracted in the previous step and the timestamped data from the behavior

# Prep: libraries, data, functions

### Libraries

In [ ]:
# Load the libraries
import caiman as cm
from caiman.source_extraction import cnmf

import matplotlib.pyplot as plt
from matplotlib.widgets import Button, TextBox
from matplotlib.patches import Polygon
from matplotlib.path import Path
import matplotlib.gridspec as gridspec
from matplotlib.widgets import CheckButtons

import numpy as np
import pandas as pd
import json
import pickle # yeah let's pickle
from scipy.sparse import csc_matrix
from scipy.stats import sem
from scipy.interpolate import interp1d
from scipy import stats
from scipy.stats import permutation_test as permutt
from scipy.stats import wilcoxon
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA

import sqlite3

from IPython import get_ipython
import IPython

import math

import os
import itertools
import time
from ipyfilechooser import FileChooser

from scipy.ndimage import binary_erosion

try:
    if __IPYTHON__:
        get_ipython().run_line_magic('load_ext', 'autoreload')
        get_ipython().run_line_magic('autoreload', '2')
except NameError:
    pass

## Load the data & create a mega data structure 

In [ ]:
# Load the data
## ===== DELETE THE COLUMNS WITH FRAME LASER AND TONE FRAME ===========================
path = 'D:\\OneDrive - Rutgers University\\CaIm\\b3\\' 
mouse_ids =['os85a', 'os85b', 'os86a', 'os86b', 'os87', 'os88', 'os89']
time_shifts = {'os85a': 0, 'os85b': 0, 'os86a': 0, 'os86b': 0, 
                     'os87': 250, 'os88': 0, 'os89': 0}
data = {}
tone_laser_t_diff = {'os85a': 954, 'os85b': 955, 'os86a': 956, 'os86b': 955, 
                     'os87': 956, 'os88': 955, 'os89': 955}
for id in mouse_ids:
    time_shift = time_shifts[id]
    path_mouse = path + id + '\\'
    experiment = {}
    entries_exp = os.listdir(path_mouse)
    for entry in entries_exp:
        if entry[-7:] == '_B3.csv':
            path_behav = path_mouse + '\\' + entry
    behav = pd.read_csv(path_behav)
    
    # do the time
    behav['laser_ts'] = [behav['laser_ts'][n] + time_shift for n in np.arange(behav.shape[0])]

    # ============== add extra columns to behav ==============
    # random tag
    behav['rand'] = [np.random.randint(2) for t in behav['laser_ts'] ]  # column with random values
    # previous choice
    prev_ch = [0]
    prev=[behav['choice'][n-1] for n in np.arange(1,behav.shape[0])]
    prev_ch.extend(prev)
    behav['prev_choice']=prev_ch
    # previous side
    prev_side = [0]
    prev_s=[behav['side'][n-1] for n in np.arange(1,behav.shape[0])]
    prev_side.extend(prev_s)
    behav['prev_side']=prev_side
    # previous reward
    prev_rew = [0]
    for n in np.arange(1,behav.shape[0]):
        if behav['side'][n-1] == behav['choice'][n-1]:
            prev_r = [behav['side'][n-1]]
        elif behav['free'][n-1]:
            prev_r = [behav['side'][n-1]]
        else:
            prev_r = [0]
        prev_rew.extend(prev_r)
    behav['prev_reward']=prev_rew
    behav['tone_ts'] = [t-955 for t in behav['laser_ts'] ]
    # add inter trial interval
    iti = [None]
    itis=[behav['laser_ts'][n] - behav['laser_ts'][n-1] for n in np.arange(1,behav.shape[0])]
    iti.extend(itis)
    behav['ITI']=iti
    
    # load the lick data
    licks_L = pd.read_csv(path_behav[0:-4] + '_t_licks_L.csv', header = None)
    licks_R = pd.read_csv(path_behav[0:-4] + '_t_licks_R.csv', header = None)
    licks_L = [licks_L[0][n] + time_shift for n in np.arange(licks_L.shape[0])]
    licks_R = [licks_R[0][n] + time_shift for n in np.arange(licks_R.shape[0])]

     # frame timings
    for entry in entries_exp:
        if entry[-10:] == 'timing.csv':
            path_frame_t = path_mouse + '\\' + entry
    frame_ts = pd.read_csv(path_frame_t)
         
    # Load aucs
    # Load stats and aucs files if they exist
    if os.path.isfile(path + id +'\\aucs.pkl'):
        with open(path + id +'\\aucs.pkl', 'rb') as f:
            aucs = pickle.load(f)
    experiment['aucs'] = aucs
    stats_mouse = pd.read_csv(path + id + '\\stats.csv')
    experiment['stats'] = stats_mouse
    
    # get the neuronal traces
    path_neu = path_mouse + 'traces.json'
    with open(path_neu, 'r') as json_file:
        neu_traces = json.load(json_file)
    neu_traces = np.array(neu_traces)
    
    path_deconv = path_mouse + 'deconvolved.json'
    with open(path_deconv, 'r') as json_file:
        deconvolved = json.load(json_file)
    deconvolved = np.array(deconvolved)
    path_accept = path_mouse + 'accepted.json'
    if os.path.isfile(path_accept):
        with open(path_accept, 'r') as json_file:
            accepted = json.load(json_file)
            # save onoly the accepted neurons
            neu_traces = neu_traces[accepted,:]
            deconvolved = deconvolved[accepted,:]

    # normalize the traces

    for trace_n in np.arange(len(neu_traces)):
        trace = neu_traces[trace_n]
        trace_z = stats.zscore(trace)
        trace_z_min = min(trace_z)
        trace_z0 = [x - trace_z_min for x in trace_z]
        neu_traces[trace_n] = trace_z0
        # smoothen for deconvolved
        

        # interpolate to correct the timing to make it to 100 ms
    real_ts = frame_ts['real_ts'].tolist()
    new_ts = np.arange(real_ts[0], real_ts[-1], 100)
    t_diffs = frame_ts['t_diff'].tolist()
    cursed = [t_diff > 330 for t_diff in t_diffs]
    frame_ts_new = {'real_ts': new_ts, 't_diff': [100 for n in np.arange(len(new_ts))]}
    frame_ts_new = pd.DataFrame(frame_ts_new)

    # Interpolated traces
    interpolated_traces = np.zeros((neu_traces.shape[0], len(new_ts)))
    interpolated_deconv_traces = np.zeros((deconvolved.shape[0], len(new_ts)))
    traces_smooth = np.zeros((neu_traces.shape[0], len(new_ts)))
    # Interpolate each neuron's activity
    for neuron_idx in range(neu_traces.shape[0]):
        interpolator = interp1d(real_ts, neu_traces[neuron_idx], kind='linear', fill_value="extrapolate")
        interpolated_traces[neuron_idx] = interpolator(new_ts)
        # interpolate deconvolved
        interpolator_deconv = interp1d(real_ts, deconvolved[neuron_idx], kind='linear', fill_value="extrapolate")
        interpolated_deconv_traces[neuron_idx] = interpolator_deconv(new_ts)
        
        trace_d = interpolated_deconv_traces[neuron_idx]
        trace_smooth = np.zeros_like(trace_d) 
        trace_smooth[0] = (trace_d[0] + trace_d[1]) / 2
        trace_smooth[-1] = (trace_d[-2] + trace_d[-1]) / 2
        for i in np.arange(1, len(trace_d)-1):
            trace_smooth[i] = (trace_d[i-1] + trace_d[i] + trace_d[i+1]) / 3
        traces_smooth[trace_n] = trace_smooth
        


# Add col to behav about ca imaging quality (we need to avoid periods of time with freezes)
    # Iterate over each trial
    freeze_threshold = 330
    behav['ca_well'] = True
    rec_len = interpolated_traces.shape[1]*100
    trim = False
    for index, row in behav.iterrows():
        tone_ts = row['tone_ts']
        start_window = tone_ts - 2000  
        end_window = tone_ts + 5000     
    
        # Find frames within the time window
        frames_in_window = frame_ts[(frame_ts['real_ts'] >= start_window) &
                                        (frame_ts['real_ts'] <= end_window)]
    
        # Check if there are any freezes in this window
        if any(frames_in_window['t_diff'] > freeze_threshold):
            behav.at[index, 'ca_well'] = False
        elif (end_window >= rec_len) & trim == False:
            trim_idx = index
            trim = True
        # specifically for os89
        # 610985 end of the 1st rec,  start of the 2nd
        
        if (id == 'os89'):
            start_bad = 610985; end_bad = 722355; tr_dur = 6000
            if (behav.at[index, 'laser_ts'] > (start_bad-tr_dur)) & (behav.at[index, 'laser_ts'] < (end_bad+tr_dur)):
                behav.at[index-1, 'ca_well'] = False
                behav.at[index, 'ca_well'] = False
                behav.at[index+1, 'ca_well'] = False
    behav = behav[behav['ca_well']].reset_index(drop=True)
            

    if trim:
        behav = behav.iloc[:trim_idx]
        
    # remove few useless columns
    behav.drop(columns=['right_crrect', 'left_crrect', 'ca_well'], inplace=True)
    
    experiment['behav'] = behav
    experiment['licks_L'] = licks_L
    experiment['licks_R'] = licks_R
    
    # get evaluations of the neurons
    inside_neurons = pd.read_csv(path_mouse + 'inside_neurons.csv')
    good_neurons = pd.read_csv(path_mouse + 'good_neurons.csv')
    fair_neurons = pd.read_csv(path_mouse + 'fair_neurons.csv')
    inside_neurons = inside_neurons['Inside'].to_list()
    good_neurons = good_neurons['Status'].to_list()
    fair_neurons = fair_neurons['Fair'].to_list()
    

    # make a big list of all good / fair neurons
    all_good_neurons = []
    all_fair_neurons = []
    usable_neurons = []
    idx = 0
    for neuron in inside_neurons:
        if neuron:  # If the neuron is inside
            if (good_neurons[idx] == 1) & (fair_neurons[idx] == 0):
                all_good_neurons.append(1) 
                all_fair_neurons.append(0)
                usable_neurons.append(1) 
            elif (good_neurons[idx] == 0) & (fair_neurons[idx] == 1):
                all_good_neurons.append(0) 
                all_fair_neurons.append(1)
                usable_neurons.append(1) 
            elif (good_neurons[idx] == 1) & (fair_neurons[idx] == 1):
             #   print(f'WTF {id}')
                all_good_neurons.append(1) 
                all_fair_neurons.append(1)
                usable_neurons.append(1) 
            else:
                all_good_neurons.append(0) 
                all_fair_neurons.append(0)
                usable_neurons.append(0) 
            idx += 1 
        else: 
            all_good_neurons.append(0) 
            all_fair_neurons.append(0) 
            usable_neurons.append(0) 

    within_ca_img = behav['laser_ts']<neu_traces.shape[1]*100
    non_missed = behav[within_ca_img & behav['missed']==0].shape[0]
    correct_left = behav[within_ca_img & (behav['side']==1) & (behav['choice']==1)].shape[0]
    correct_right = behav[within_ca_img & (behav['side']==-1) & (behav['choice']==-1)].shape[0]
    print(id)
    print(f'{non_missed} nomiss, correct: {correct_left} L, {correct_right} R;\
    neurons: {sum(all_good_neurons)} good, {sum(all_fair_neurons)} fair')
    
    #neu_traces = pd.read_csv(path_neu)
  #  neu_traces = neu_traces.to_numpy()
    experiment['good_neurons'] = all_good_neurons
    experiment['fair_neurons'] = all_fair_neurons
    experiment['usable_neurons'] = usable_neurons
    experiment['smooth'] = traces_smooth
    experiment['ca'] = interpolated_traces
    experiment['deconv'] = interpolated_deconv_traces
    experiment['frame_ts_og'] = frame_ts
    experiment['frame_ts'] = frame_ts_new

    # add all the experiment data 
    data[id] = experiment


In [ ]:
# add pre-computed AUCs if available
for id in mouse_ids:
    print(f"::: Mouse  {id} :::")
    if os.path.isfile(path + id +'\\aucs.pkl'):
        with open(path + id +'\\aucs.pkl', 'rb') as f:
            aucs = pickle.load(f)
    data[id]['aucs'] = aucs

In [ ]:
# smoothen up the traces
for id in mouse_ids:
    print(f"::: Mouse  {id} :::")
    ew = data[id]['deconv']
    smoothies = np.zeros_like(ew)
    for n in np.arange(ew.shape[0]):
        trace = pd.Series(ew[n])
        smooth = trace.rolling(window=5).mean()
        smoothies[n] = smooth
    data[id]['smoothies'] = smoothies
fig, axs = plt.subplots(1, 1, figsize=(15, 3))        
axs.plot(smoothies[n][:200])
axs.plot(trace[:200])
plt.show()


## A bunch of sanity checks

In [ ]:
# Test how well the timing looks
need_timing_test = True
test_range = np.arange(-15, 50, 1)

if need_timing_test:
    # For each mouse
    for id in mouse_ids:
        print(f"::: Mouse  {id} :::")
        prophet_neus = []
        data_exp = data[id]
        traces = data_exp['ca']
        deconvolveds = data_exp['deconv']
        behav = data_exp['behav']
        behav = behav[behav['missed']==0]
        real_ts = data_exp['frame_ts']['real_ts'].tolist()
        behav_l = behav[behav['side'] == 1]
        behav_r = behav[behav['side'] == -1]
        ts_l = behav_l['tone_ts'].values
        ts_r = behav_r['tone_ts'].values
        idx_ls = []
        for t in ts_l:
            # Find the index in the ts array closest to the tone_time
            idx_l = (np.abs(real_ts - t)).argmin()
            if idx_l + test_range[-1] < deconvolveds.shape[1]:
                idx_ls.append(idx_l)
        idx_rs = []
        for t in ts_r:
            # Find the index in the ts array closest to the tone_time
            idx_r = (np.abs(real_ts - t)).argmin()
            if idx_r + test_range[-1] < deconvolveds.shape[1]:
                idx_rs.append(idx_r)
    
        for test_time in test_range:
      #      print(f":::== test time {test_time} ==:::")
            
            n_selective = 0
            neu_total = traces.shape[0]

            for n in np.arange(neu_total):
                # if the neurons is not good or fair, skip it
              #  if data_exp['fair_neurons'][n] + data_exp['good_neurons'][n] == 0:
                #    continue
                
                trace  = traces[n][:]
                deconvolved = deconvolveds[n][:]
                
                # effects of the side
                params = ['missed', 'side']
                side_l = [trace[i + test_time] for i in idx_ls]
                side_r = [trace[i + test_time] for i in idx_rs]

                # Perform Wilcoxon signed-rank test
                stat, p_val = stats.ttest_ind(side_l, side_r)            
        
                if p_val < 0.05:
                    n_selective += 1
            rate_selective = n_selective/neu_total
            prophet_neus.append(n_selective)

        #    print(f" Prophet neurons: {n_selective} ")

        plt.plot(test_range,prophet_neus)
    plt.axvline(x=0, linestyle=':')
    plt.axvline(x=3, linestyle=':')
    plt.legend(mouse_ids)
    plt.show()

In [ ]:
sanity_check89 = False
if sanity_check89:
    trace = data['os89']['ca'][0]
    deconvolved = data['os89']['deconv'][0]
    ts = np.arange(len(trace))
    fig, axs = plt.subplots(4, 1, figsize=(15, 5))
    chunk = len(trace)//3
    axs[0].plot(ts[0:2000],trace[0:2000])
    axs[1].plot(ts[0:2000],deconvolved[0:2000])
    axs[2].plot(ts[6000:7500], trace[6000:7500])
    axs[3].plot(ts[6000:7500], deconvolved[6000:7500])
    #axs[2].plot(ts[chunk*2:], trace[chunk*2:])
    plt.show()

In [ ]:
deconv_check = False
id = 'os88'
if deconv_check:
    find_fair = True
    n = 50
    while find_fair:
        if data[id]['fair_neurons'][n] != 1:
            n += 1
        else:
            find_fair = False
    trace = data[id]['ca'][n]
    deconvolved = data[id]['deconv'][n]
    ts = np.arange(len(trace))
    fig, axs = plt.subplots(4, 1, figsize=(15, 5))
    chunk = len(trace)//3
    axs[0].plot(ts[2000:3000],trace[2000:3000])
    axs[1].plot(ts[2000:3000],deconvolved[2000:3000])
    axs[2].plot(ts[chunk:chunk*2], trace[chunk:chunk*2])
    axs[3].plot(ts[chunk:chunk*2], deconvolved[chunk:chunk*2])
    #axs[2].plot(ts[chunk*2:], trace[chunk*2:])
    plt.show()

## Functions for data selection and plotting

## New soon-to-be canon functions

In [ ]:
# Function to get all the needed event times from the behavioral data
# for the specified attributes
def get_behav_ts(behav, attr, attr_vals, event_type='tone_ts'):
    
    # select the behav trials according to attributes & values
    for a, v, in zip(attr, attr_vals):
        behav = behav[behav[a]==v]

    ts = behav[event_type]
    ts = ts.tolist()
    
    return ts
    

In [ ]:
def fdr_correction(p_values, alpha=0.05):
    """
    Perform FDR correction using the Benjamini-Hochberg method.

    Parameters:
    p_values (list or numpy array): Array of p-values.
    alpha (float): Desired significance level.

    Returns:
    significant (numpy array): Boolean array indicating which p-values are significant after FDR correction.
    """
    p_values = np.array(p_values)
    m = len(p_values)  # Number of tests
    sorted_indices = np.argsort(p_values)  # Indices that would sort the p-values
    sorted_p_values = p_values[sorted_indices]  # Sorted p-values
    adjusted_p_values = np.zeros(m)  # Array to hold adjusted p-values

    # Calculate the adjusted p-values
    for i in range(m):
        adjusted_p_values[i] = sorted_p_values[i] * m / (i + 1)

    # Ensure that adjusted p-values are non-decreasing
    adjusted_p_values = np.minimum.accumulate(adjusted_p_values[::-1])[::-1]

    # Determine which p-values are significant
    significant = adjusted_p_values < alpha

    # Return the significant p-values in the original order
    significant_corrected = np.zeros(m, dtype=bool)
    significant_corrected[sorted_indices] = significant

    return significant_corrected, adjusted_p_values[sorted_indices]

# Example usage:
# Let's assume p_values is a list or numpy array of p-values from your permutation tests.
p_values = [0.01, 0.03, 0.2, 0.5, 0.04, 0.002, 0.07]
alpha = 0.05

significant, adjusted_p_values = fdr_correction(p_values, alpha=alpha)

print("Significant neurons after FDR correction:", significant)
print("Adjusted p-values:", adjusted_p_values)


In [ ]:
# ============ NEW ONE WITH FRAME TIMINGS =============================
# default span
pre_event_ts = 2000
post_event_ts = 6000
# Function to extract data around some type of event in 'behav'
def get_segments(behav, trace, frame_ts, attrs, attr_vals, trace_type=False, 
                 span = [pre_event_ts,post_event_ts]):
    
    for a_n in np.arange(len(attrs)):
        behav = behav[behav[attrs[a_n]] == attr_vals[a_n] ]
    ts = behav['tone_ts']
    real_ts = frame_ts['real_ts']
    fr_before = int(span[0]//100)
    fr_after = int(span[1]//100)
    
    segments = []
    segments_ts = []
    # choose which neural signal to use
    for t in ts:
        # Find the closest index in frame_ts to the event time 't'
        idx = (np.abs(real_ts - t)).argmin()
        start = idx - fr_before
        end =  idx + fr_after

        good_timings = True
        for t_diff in frame_ts['t_diff'][start:end]:
            if t_diff > 300:
                good_timings = False

        if (start > 0) and (end < len(trace)) and good_timings:
            segment = trace[start:end]
            segment_ts = real_ts[start:end]
            segment_ts = [time - t for time in segment_ts]
            
            
            segments.append(segment)
            segments_ts.append(segment_ts)
    
    return segments, segments_ts 


    


In [ ]:
## =========================== HOPEFULLY GOOD ENOUGH LATEST VERSION FOR ALL SCRIPTS ==================================
# Function to extract data around some type of event in 'behav'
pre_event_points = 1000
post_event_points = 5000
def get_segments_canon(trace, ts, frame_ts, span = [pre_event_points,post_event_points]):

    real_ts = frame_ts['real_ts']
    fr_before = int(span[0]//100)
    fr_after = int(span[1]//100)
    
    segments = []
    segments_ts = []
    # choose which neural signal to use
    for t in ts:
        # Find the closest index in frame_ts to the event time 't'
        idx = (np.abs(real_ts - t)).argmin()
        start = idx - fr_before
        end =  idx + fr_after

        good_timings = True
        for t_diff in frame_ts['t_diff'][start:end]:
            if t_diff > 300:
                good_timings = False

        if (start > 0) and (end < len(trace)) and good_timings:
            segment = trace[start:end]
            segment_ts = real_ts[start:end]
            segment_ts = [time - t for time in segment_ts]
            segments.append(segment)
            segments_ts.append(segment_ts)   
    
    return segments, segments_ts 

In [ ]:
# standard timepoints to use
# MB DELETE THOSE COMMENTED LINES
#pre_event_points = 10
#post_event_points = 30
#sampling_rate=10

# function to plot neuronal average traces
def plot_aver_sem(traces, ts, clr, ghost_traces=False, linestyle=None, ax_n = 0):
    # Time points for x-axis 
    #time_points = np.arange(-pre_event_points, post_event_points) * (1000 / sampling_rate)
    # Do 2 subplots if ghost traces
   
    # Convert to numpy array and calculate mean and SEM
    mean_trace = np.nanmean(traces, axis=0)
    std_trace = sem(traces, axis=0, nan_policy='omit')

    # Plot ghost traces if needed
    if ghost_traces:
        for trace, t in zip(traces, ts):
            axs[ax_n + 1].plot(t, trace, clr, alpha=0.2)
    
    # Plot mean and SEM
    axs[ax_n].plot(ts[0], mean_trace, clr, linestyle=linestyle)
    axs[ax_n].fill_between(ts[0], mean_trace - std_trace, mean_trace + std_trace, color=clr, alpha=0.2)
  

In [ ]:
# standard timepoints to use

# function to plot neuronal average traces
def plot_aver_sem_canon(traces, times, colr, ghost_traces=False, axn=0, linestyle=None, 
                        span = [pre_event_points,post_event_points], sampling_rate = 100):
    # Time points for x-axis
    pre_event_points = span[0]
    post_event_points = span[1]
    time_points = np.arange(-pre_event_points, post_event_points, sampling_rate) 
    # Do 2 subplots if ghost traces
    if ghost_traces:
        #fig, axs = plt.subplots(1, 2, figsize=(10, 3)) 
        for trace, ts in zip(traces, times):
            axs[axn+1].plot(time_points, trace, colr, alpha=0.5)
   # else:
    #    fig, axs = plt.subplots(1, 1, figsize=(10, 3))
    
    # Mean and std
    mean_trace = np.mean(traces, axis=0)
    std_trace = sem(traces, axis=0)
    
    # Plot
    axs[axn].plot(time_points, mean_trace, colr, linestyle=linestyle)
    axs[axn].fill_between(time_points, mean_trace - std_trace, mean_trace + std_trace, color=colr, alpha=0.2)
    

In [ ]:
def get_auc(event_ts, trace, frame_ts, t1, t2):
    """
    Calculate the area under the curve (AUC) of neural activity traces 
    around specific event times

    """
    # get the number of frames needed to be fetched
    fr_n = int((t2-t1)/100)

    # get ts from the frame_ts dataframe
    ts = frame_ts['real_ts'] 
    
    auc_values = []
    
    for event_t in event_ts:
        # Calculate the start and end indices based on t1 and t2
        start_time = event_t + t1
        
        start_idx = (np.abs(ts - start_time)).argmin()
        end_idx = start_idx + fr_n
                
        # Ensure the indices are within the range of the trace
        if (start_idx < 0) or (end_idx >= (len(trace)-1)):
          #  print(end_idx)
            continue
        
        # Extract the segment of the trace
        segment = trace[start_idx:end_idx + 1]
        
        # Calculate the AUC by simply summing it all haha
        auc = sum(segment)
        auc_values.append(auc)
    
    return auc_values


In [ ]:


def permutation_test(data1, data2, n_permutations=10000):
    """
    Perform a permutation test on two datasets.
    
    Parameters:
    - data1, data2: Arrays containing the data for the two groups.
    - n_permutations: Number of permutations to perform.
    
    Returns:
    - p-value: The proportion of permuted statistics more extreme than the observed.
    """
    # Calculate the observed difference in means
    observed_diff = np.mean(data1) - np.mean(data2)
    
    # Concatenate the datasets
    combined_data = np.concatenate([data1, data2])
    
    # Initialize a counter for permutations where the permuted difference is more extreme than observed
    count_extreme = 0
    
    # Perform permutations
    for _ in range(n_permutations):
        # Shuffle the combined data
        np.random.shuffle(combined_data)
        
        # Split the permuted dataset
        permuted_data1 = combined_data[:len(data1)]
        permuted_data2 = combined_data[len(data1):]
        
        # Calculate the difference in means for the permuted datasets
        permuted_diff = np.mean(permuted_data1) - np.mean(permuted_data2)
        
        # Check if the permuted difference is as extreme as the observed difference
        if abs(permuted_diff) >= abs(observed_diff):
            count_extreme += 1
    
    # Calculate the p-value
    p_value = count_extreme / n_permutations
    
    return p_value


In [ ]:
# test the new functions
code = {'l': 1, 'r': -1}
t1 = 0; t2 = 1200; t3 = 2500
time_periods = [ [0, 1200], [1200, 2400], [2400, 3600] ]
# For each mouse
for id in mouse_ids[6:]:

    print(f"::: Mouse  {id} :::")
    print(f":::============:::")
    data_exp = data[id]
   # traces = data_exp['ca']
    traces = data_exp['deconv']
    behav = data_exp['behav']
    frame_ts = data_exp['frame_ts']
    n_tr = behav.shape[0]
    for neu in np.arange(traces.shape[0]):
        # if the neurons is not good or fair, skip it
        if data_exp['fair_neurons'][neu] + data_exp['good_neurons'][neu] == 0:
            continue
            
        trace  = traces[neu][:]
        disp_fig = False
        aucs = {}
        params = ['missed', 'laser', 'choice', 'side']
        p_vals = {}
        for t1,t2 in time_periods:
            for side in ['l', 'r']:
                event_ts = get_behav_ts(behav, params, [0, 0, code[side], code[side]])
                aucs[side] = get_auc(event_ts, trace, frame_ts, t1, t2)
            p_value_side = permutation_test(aucs['l'], aucs['r'], 1000)
            
            if p_value_side < 0.02:
                disp_fig = True

        if disp_fig:
            # prep the figure
            fig, axs = plt.subplots(1, 2, figsize=(15, 3))

            for cond in ['side']:
                params = [cond, 'missed']

                ts_l = get_behav_ts(behav, params, [1, 0])
                ts_r = get_behav_ts(behav, params, [-1, 0])
                for event_t in ts_r:
                    if (event_t < 720000) & (event_t > 600083):
                        print(event_t)

                segms_l, times_l = get_segments_canon(trace, ts_l, frame_ts)
                segms_r, times_r = get_segments_canon(trace, ts_r, frame_ts)

                plot_aver_sem_canon(segms_l, times_l, 'b', ghost_traces=True)
                plot_aver_sem_canon(segms_r, times_r, 'r', ghost_traces=True)



                for ax_n in [0,1]:
                    axs[ax_n].axvline(x=0)
                    axs[ax_n].axvline(x=1150)

                plt.show()


In [ ]:

# distance btw points in component space

def distance_btw_comp(condtions, data_exp, ca_df_time_points, w=[9, 17], span=[0, 30]):
    coords_av = {condtions[0]:[], condtions[1]:[]}
    

    cas = ca_df_time_points.keys()
    
    for key in coords_av.keys():
        params = param_val_keys[key]['params']
        vals = param_val_keys[key]['vals']
        coords_cond = []
        
        for ca_n in cas:
            comp = ca_df_time_points[ca_n]
            traces = get_segments_old(data_exp, params, vals, n, frame_type='frame_tone', span=span, is_pca=True, pc=comp)
            coords_n = [np.mean(trace[span[0]+w[0]:span[0]+w[1]]) for trace in traces]
            coord_n_av = np.mean(coords_n)
            coords_cond.append(coord_n_av)
        
        coords_av[key] = coords_cond
    
    distance = np.sqrt(np.sum((np.array(coords_av[condtions[0]]) - np.array(coords_av[condtions[1]]))**2))
    return distance




# All stats to extract

### Get all AUCs

In [ ]:

# Add to all experiemnt a tables the auc for pre, tone, and resp
code = {'pre': [-1000, 0], 'tone': [0, 1000], 'resp': [1000, 2000], 'resp2': [2000, 3000], 'anticip': [3000, 4000]}
time_periods = ['pre', 'tone', 'resp', 'resp2', 'anticip']
# For each mouse
for id in mouse_ids:

    if 'aucs' in data[id].keys():
        print(f'aucs for {id} done')
        continue
    elif os.path.isfile(path + id +'\\aucs.json'):
        with open(path + id +'\\aucs.json', 'r') as f:
            aucs = json.load(f)
        data[id]['aucs'] = aucs

    print(f"::: Mouse  {id} :::")
    print(f":::============:::")
    data_exp = data[id] 
    behav = data_exp['behav'] 
    all_ts = get_behav_ts(behav, [], [])
    frame_ts = data_exp['frame_ts']
    ts = frame_ts['real_ts'].tolist()
    n_tr = behav.shape[0] 
    aucs = {}
    for trace_type in ['ca', 'deconv']:
        period_aucs = {}
        period_aucs_to_save = {}
        for time_period in time_periods:
            print(f'{trace_type} {time_period}')
            neu_aucs = {}
            for n in np.arange(data_exp[trace_type].shape[0]):
                trace = data_exp[trace_type][n]
                t1, t2 = code[time_period]
                
                neu_aucs[n] = get_auc(all_ts, trace, frame_ts, t1, t2)
                
            # here we add period aucs
            neu_aucs_df = pd.DataFrame(neu_aucs)
            period_aucs[time_period] = neu_aucs_df
        # here we add trace types aucs
        aucs[trace_type] = period_aucs

    print(behav.shape, neu_aucs_df.shape, data_exp[trace_type].shape)
    data[id]['aucs'] = aucs
    with open(path + id +'\\aucs.pkl', 'wb') as f:
        pickle.dump(aucs, f)

### Add specifically laser AUCs

In [ ]:

# Add to all experiemnt a tables the auc for pre, tone, and resp
code = {'pre': [-1000, 0], 'tone': [0, 1000], 'resp': [1000, 2000], 'resp2': [2000, 3000], 'anticip': [3000, 4000]}
time_periods = ['pre', 'tone', 'resp', 'resp2', 'anticip']
# For each mouse
for id in mouse_ids:

    aucs = data[id]['aucs'] 

    print(f"::: Mouse  {id} :::")
    print(f":::============:::")
    data_exp = data[id] 
    behav = data_exp['behav'] 
    all_ts = get_behav_ts(behav, [], [])
    frame_ts = data_exp['frame_ts']
    ts = frame_ts['real_ts'].tolist()
    n_tr = behav.shape[0] 
    aucs = {}
    for trace_type in ['ca', 'deconv']:
        period_aucs = {}
        period_aucs_to_save = {}
        print(f'{trace_type} ')
  
        neu_aucs = {}
        for n in np.arange(data_exp[trace_type].shape[0]):
            trace = data_exp[trace_type][n]
            t1 = 1250
            t2 = 2250
            
            neu_aucs[n] = get_auc(all_ts, trace, frame_ts, t1, t2)
            
        # here we add period aucs
        neu_aucs_df = pd.DataFrame(neu_aucs)

    print(behav.shape, neu_aucs_df.shape, data_exp[trace_type].shape)
    data[id]['aucs'][trace_type]['polas'] = neu_aucs_df
    with open(path + id +'\\aucs.pkl', 'wb') as f:
        pickle.dump(aucs, f)

## Major stats: side

In [ ]:
#code = {'l': 1, 'r': -1, 
    #    'pre': [-1000, 0], 'tone': [0, 1000], 
     #   'resp': [1000, 2000], 'resp2': [2000, 3000], 
     #   'anticip': [3000, 4000]}
time_periods = ['pre', 'tone', 'resp', 'resp2', 'anticip']
trace_types = ['ca', 'deconv']
           
# number of permutations to perform; ideally 10000
n_permut = 10000

# For each mouse
for id in mouse_ids:
    print(f"::: Mouse  {id} :::")
    print(f":::============:::")
    '''
    if os.path.isfile(path + id + '\\stats.csv'):
        stats_mouse = pd.read_csv(path + id + '\\stats.csv')
        data[id]['stats'] = stats_mouse
        print(f'stats loaded for {id}')
        continue
    '''
    data_exp = data[id] 
    behav = data_exp['behav']
    frame_ts = data_exp['frame_ts']
    ts = frame_ts['real_ts'].tolist()
    aucs = data_exp['aucs']
    n_neu = len(data_exp['ca'])
    n_tr = behav.shape[0] 
   # if 'stats' not in data[id].keys():
    if True:
        neu_ids = np.arange(n_neu)
        usable = data_exp['usable_neurons']
        data[id]['stats'] = {
                             'neu_n': neu_ids,
                             'usable': usable,
                            }
    # from behav, make the masks for the selection of aucs
    correct_l = (behav['side'] == 1) & (behav['choice'] == 1) & (behav['laser'] == 0)
    correct_r = (behav['side'] ==-1) & (behav['choice'] ==-1) & (behav['laser'] == 0)
    prev_l = (behav['prev_reward'] == 1) & (behav['laser'] == 0)
    prev_r = (behav['prev_reward'] ==-1) & (behav['laser'] == 0)
    same_l = (behav['side'] == 1) & (behav['choice'] == 1) & (behav['prev_reward'] == 1) & (behav['laser'] == 0)
    diff_l = (behav['side'] == 1) & (behav['choice'] == 1) & (behav['prev_reward'] ==-1) & (behav['laser'] == 0)
    same_r = (behav['side'] ==-1) & (behav['choice'] ==-1) & (behav['prev_reward'] ==-1) & (behav['laser'] == 0)
    diff_r = (behav['side'] ==-1) & (behav['choice'] ==-1) & (behav['prev_reward'] == 1) & (behav['laser'] == 0)
    stats_mouse = {}

    for trace_type, time_period in itertools.product(trace_types, time_periods):
        for metric in ['p_', 'diff_']:
            stats_mouse[metric + 'side_' + time_period + '_' + trace_type] = []
            stats_mouse[metric + 'oomf_l_' + time_period + '_' + trace_type] = []
            stats_mouse[metric + 'oomf_r_' + time_period + '_' + trace_type] = []
            stats_mouse[metric + 'prew_' + time_period + '_' + trace_type] = []
            
    for trace_type, time_period in itertools.product(trace_types, time_periods):
 #   for trace_type in ['ca', 'deconv']: 
   #     for time_period in time_periods:
        aucs_neu_list = aucs[trace_type][time_period]
        aucs_neu_np = np.array(aucs_neu_list)
        for n in np.arange(n_neu):
            if n % 100 == 0:
                print(n, trace_type, time_period)
            # select the neuron's aucs
           # aucs_neu_n = [sublist[n] for sublist in aucs_neu_list]
            aucs_neu_n = aucs_neu_np[:,n]
            # Stats for side preference (all correct)
            aucs_l = [elem for i, elem in enumerate(aucs_neu_n) if correct_l.iloc[i]]
            aucs_r = [elem for i, elem in enumerate(aucs_neu_n) if correct_r.iloc[i]]
            p_side = permutation_test(aucs_l, aucs_r, n_permut) 
            diff_side = np.mean(aucs_l) - np.mean(aucs_r) 
            stats_mouse['p_side_' + time_period + '_' + trace_type].append(p_side)
            stats_mouse['diff_side_' + time_period + '_' + trace_type].append(diff_side)
            # Stats for prev reward
            aucs_l = [elem for i, elem in enumerate(aucs_neu_n) if prev_l.iloc[i]]
            aucs_r = [elem for i, elem in enumerate(aucs_neu_n) if prev_r.iloc[i]]
            p = permutation_test(aucs_l, aucs_r, n_permut) 
            diff = np.mean(aucs_l) - np.mean(aucs_r) 
            stats_mouse['p_prew_' + time_period + '_' + trace_type].append(p)
            stats_mouse['diff_prew_' + time_period + '_' + trace_type].append(diff)
            # oomfs
            # left
            aucs_same = [elem for i, elem in enumerate(aucs_neu_n) if same_l.iloc[i]]
            aucs_diff = [elem for i, elem in enumerate(aucs_neu_n) if diff_l.iloc[i]]
            p = permutation_test(aucs_same, aucs_diff, n_permut) 
            diff = np.mean(aucs_diff) - np.mean(aucs_same) 
            stats_mouse['p_oomf_l_' + time_period + '_' + trace_type].append(p)
            stats_mouse['diff_oomf_l_' + time_period + '_' + trace_type].append(diff)
            # right
            aucs_same = [elem for i, elem in enumerate(aucs_neu_n) if same_r.iloc[i]]
            aucs_diff = [elem for i, elem in enumerate(aucs_neu_n) if diff_r.iloc[i]]
            p = permutation_test(aucs_same, aucs_diff, n_permut) 
            diff = np.mean(aucs_diff) - np.mean(aucs_same) 
            stats_mouse['p_oomf_r_' + time_period + '_' + trace_type].append(p)
            stats_mouse['diff_oomf_r_' + time_period + '_' + trace_type].append(diff)
    # append everything to the main mouse stats data
    for key in stats_mouse.keys():
        data[id]['stats'][key] = stats_mouse[key]
   
for id in mouse_ids:
    stats_exp = data[id]['stats']
    stats_exp.to_csv(path + id + '\\stats.csv', index=False)

### Save the stats

In [ ]:
### Save the stats
for id in mouse_ids:
    stats_exp = data[id]['stats']
    stats_exp = pd.DataFrame(stats_exp)
    data[id]['stats'] = stats_exp
    stats_exp.to_csv(path + id + '\\stats.csv', index=False)

In [ ]:
# Test if the scipy permutation test is faster


In [ ]:
import time
from scipy.stats import permutation_test as permutt

# Assuming aucs_y and aucs_n are your two lists

def diff_stat(x, y):
    """Function to calculate the difference in means."""
    return np.mean(x) - np.mean(y)

# Start the timer
start_time = time.time()
for tt in np.arange(100):
    p_value = permutation_test(aucs_l, aucs_r, n_permut)
    p_value = permutation_test(aucs_same, aucs_diff, n_permut)
# End the timer
end_time = time.time()

# Calculate the duration
execution_time = end_time - start_time
print(f"Execution time with my own lil permutation test: {execution_time} seconds")

# Start the timer
start_time = time.time()
for tt in np.arange(100):
    p_value = permutt((aucs_l, aucs_r), diff_stat, permutation_type='independent', n_resamples=n_permut)
    p_value = permutt((aucs_same, aucs_diff), diff_stat, permutation_type='independent', n_resamples=n_permut)
# End the timer
end_time = time.time()

# Calculate the duration
execution_time = end_time - start_time
print(f"Execution time with Scipy permutation test: {execution_time} seconds")

### Add laser stats = FOR NOW ONLY DECONVOLVED!!!!

In [ ]:
#code = {'l': 1, 'r': -1, 
    #    'pre': [-1000, 0], 'tone': [0, 1000], 
     #   'resp': [1000, 2000], 'resp2': [2000, 3000], 
     #   'anticip': [3000, 4000]}
#time_periods = ['pre', 'tone', 'resp', 'resp2', 'anticip']
time_periods = ['resp', 'resp2', 'post_laser']
trace_types = ['deconv'] # HERE ONLY DECONVOLVED FOR NOW< BUT LATER ADD CA AS WELL
           
# number of permutations to perform; ideally 10000
n_permut = 2000

# For each mouse
for id in mouse_ids:
    print(f"::: Mouse  {id} :::")
    print(f":::============:::")
    '''
    if os.path.isfile(path + id + '\\stats.csv'):
        stats_mouse = pd.read_csv(path + id + '\\stats.csv')
        data[id]['stats'] = stats_mouse
        print(f'stats loaded for {id}')
        continue
    '''
    data_exp = data[id] 
    behav = data_exp['behav']
    frame_ts = data_exp['frame_ts']
    ts = frame_ts['real_ts'].tolist()
    aucs = data_exp['aucs']
    n_neu = len(data_exp['ca'])
    n_tr = behav.shape[0] 
   # if 'stats' not in data[id].keys():

    # from behav, make the masks for the selection of aucs

    las_l = (behav['side'] == 1) & (behav['laser'] == 1)
    las_l0 = (behav['side'] == 1) &  (behav['laser'] == 0)
    las_r = (behav['side'] == -1) &  (behav['laser'] == -1)
    las_r0 = (behav['side'] == -1) & (behav['laser'] == 0)
    
    # here goes nothing
    las_l_same = (behav['side'] == 1)  & (behav['laser'] == 1) & (behav['prev_reward'] == 1)
    las_l0_same = (behav['side'] == 1)  & (behav['laser'] == 0) & (behav['prev_reward'] == 1)
    las_l_diff = (behav['side'] == 1)  & (behav['laser'] == 1) & (behav['prev_reward'] == -1)
    las_l0_diff = (behav['side'] == 1)  & (behav['laser'] == 0) & (behav['prev_reward'] == -1)
    
    las_r_same = (behav['side'] == -1) & (behav['laser'] == -1) & (behav['prev_reward'] == -1)
    las_r0_same = (behav['side'] == -1) & (behav['laser'] == 0) & (behav['prev_reward'] == -1)
    las_r_diff = (behav['side'] == -1) & (behav['laser'] == -1) & (behav['prev_reward'] == 1)
    las_r0_diff = (behav['side'] == -1) & (behav['laser'] == 0) & (behav['prev_reward'] == 1)

    
    stats_add= {}

    for trace_type, time_period in itertools.product(trace_types, time_periods):
        for metric in ['p_', 'diff_']:
            for side in ['l_', 'r_']:
                stats_add[metric + 'lass_' + side + time_period + '_' + trace_type] = []
             #   stats_add[metric + 'las_same_' + side + time_period + '_' + trace_type] = []
             #   stats_add[metric + 'las_diff_' + side + time_period + '_' + trace_type] = []
            
    for trace_type, time_period in itertools.product(trace_types, time_periods):
 #   for trace_type in ['ca', 'deconv']: 
   #     for time_period in time_periods:
        aucs_neu_list = aucs[trace_type][time_period]
        aucs_neu_np = np.array(aucs_neu_list)
        for n in np.arange(n_neu):
            if n % 100 == 0:
                print(n, trace_type, time_period)
            # select the neuron's aucs
            aucs_neu_n = aucs_neu_np[:,n]
            # Stats for laser effects (all correct)
            # Left
            aucs_y = [elem for i, elem in enumerate(aucs_neu_n) if las_l.iloc[i]]
            aucs_n = [elem for i, elem in enumerate(aucs_neu_n) if las_l0.iloc[i]]
            p_side = permutation_test(aucs_y, aucs_n, n_permut) 
            diff_side = np.mean(aucs_y) - np.mean(aucs_n) 
            stats_add['p_lass_l_' + time_period + '_' + trace_type].append(p_side)
            stats_add['diff_lass_l_' + time_period + '_' + trace_type].append(diff_side)
            # Right
            aucs_y = [elem for i, elem in enumerate(aucs_neu_n) if las_r.iloc[i]]
            aucs_n = [elem for i, elem in enumerate(aucs_neu_n) if las_r0.iloc[i]]
            p_side = permutation_test(aucs_y, aucs_n, n_permut) 
            diff_side = np.mean(aucs_y) - np.mean(aucs_n) 
            stats_add['p_lass_r_' + time_period + '_' + trace_type].append(p_side)
            stats_add['diff_lass_r_' + time_period + '_' + trace_type].append(diff_side)
            '''
            # Those fucking oomf lasers
            # Left
            # same
            aucs_y = [elem for i, elem in enumerate(aucs_neu_n) if las_l_same.iloc[i]]
            aucs_n = [elem for i, elem in enumerate(aucs_neu_n) if las_l0_same.iloc[i]]
            p_side = permutation_test(aucs_y, aucs_n, n_permut) 
            diff_side = np.mean(aucs_y) - np.mean(aucs_n) 
            stats_add['p_las_same_l_' + time_period + '_' + trace_type].append(p_side)
            stats_add['diff_las_same_l_' + time_period + '_' + trace_type].append(diff_side)
            # diff
            aucs_y = [elem for i, elem in enumerate(aucs_neu_n) if las_l_diff.iloc[i]]
            aucs_n = [elem for i, elem in enumerate(aucs_neu_n) if las_l0_diff.iloc[i]]
            p_side = permutation_test(aucs_y, aucs_n, n_permut) 
            diff_side = np.mean(aucs_y) - np.mean(aucs_n) 
            stats_add['p_las_diff_l_' + time_period + '_' + trace_type].append(p_side)
            stats_add['diff_las_diff_l_' + time_period + '_' + trace_type].append(diff_side)

            # Right
            # same
            aucs_y = [elem for i, elem in enumerate(aucs_neu_n) if las_r_same.iloc[i]]
            aucs_n = [elem for i, elem in enumerate(aucs_neu_n) if las_r0_same.iloc[i]]
            p_side = permutation_test(aucs_y, aucs_n, n_permut) 
            diff_side = np.mean(aucs_y) - np.mean(aucs_n) 
            stats_add['p_las_same_r_' + time_period + '_' + trace_type].append(p_side)
            stats_add['diff_las_same_r_' + time_period + '_' + trace_type].append(diff_side)
            # diff
            aucs_y = [elem for i, elem in enumerate(aucs_neu_n) if las_r_diff.iloc[i]]
            aucs_n = [elem for i, elem in enumerate(aucs_neu_n) if las_r0_diff.iloc[i]]
            p_side = permutation_test(aucs_y, aucs_n, n_permut) 
            diff_side = np.mean(aucs_y) - np.mean(aucs_n) 
            stats_add['p_las_diff_r_' + time_period + '_' + trace_type].append(p_side)
            stats_add['diff_las_diff_r_' + time_period + '_' + trace_type].append(diff_side)
    '''

    # append everything to the main mouse stats data
    for key in stats_add.keys():
        data[id]['stats'][key] = stats_add[key]
   
for id in mouse_ids:
    stats_exp = data[id]['stats']
    stats_exp.to_csv(path + id + '\\stats.csv', index=False)

In [ ]:
#code = {'l': 1, 'r': -1, 
    #    'pre': [-1000, 0], 'tone': [0, 1000], 
     #   'resp': [1000, 2000], 'resp2': [2000, 3000], 
     #   'anticip': [3000, 4000]}
time_periods = ['pre', 'tone', 'resp', 'resp2', 'anticip']
trace_types = ['deconv'] # HERE ONLY DECONVOLVED FOR NOW< BUT LATER ADD CA AS WELL
           
# number of permutations to perform; ideally 10000
n_permut = 10000

# For each mouse
for id in mouse_ids:
    print(f"::: Mouse  {id} :::")
    print(f":::============:::")
    '''
    if os.path.isfile(path + id + '\\stats.csv'):
        stats_mouse = pd.read_csv(path + id + '\\stats.csv')
        data[id]['stats'] = stats_mouse
        print(f'stats loaded for {id}')
        continue
    '''
    data_exp = data[id] 
    behav = data_exp['behav']
    frame_ts = data_exp['frame_ts']
    ts = frame_ts['real_ts'].tolist()
    aucs = data_exp['aucs']
    n_neu = len(data_exp['ca'])
    n_tr = behav.shape[0] 
   # if 'stats' not in data[id].keys():

    # from behav, make the masks for the selection of aucs

    las_l = (behav['side'] == 1) & (behav['choice'] == 1) & (behav['laser'] == 1)
    las_l0 = (behav['side'] == 1) & (behav['choice'] == 1) & (behav['laser'] == 0)
    las_r = (behav['side'] == -1) & (behav['choice'] == -1) & (behav['laser'] == -1)
    las_r0 = (behav['side'] == -1) & (behav['choice'] == -1) & (behav['laser'] == 0)

    las_ll = (behav['side'] == 1) & (behav['laser'] == 1)
    las_ll0 = (behav['side'] == 1)  & (behav['laser'] == 0)
    las_rr = (behav['side'] == -1)  & (behav['laser'] == -1)
    las_rr0 = (behav['side'] == -1) & (behav['laser'] == 0)
    
    # here goes nothing
    las_l_same = (behav['side'] == 1) & (behav['choice'] == 1) & (behav['laser'] == 1) & (behav['prev_reward'] == 1)
    las_l0_same = (behav['side'] == 1) & (behav['choice'] == 1) & (behav['laser'] == 0) & (behav['prev_reward'] == 1)
    las_l_diff = (behav['side'] == 1) & (behav['choice'] == 1) & (behav['laser'] == 1) & (behav['prev_reward'] == -1)
    las_l0_diff = (behav['side'] == 1) & (behav['choice'] == 1) & (behav['laser'] == 0) & (behav['prev_reward'] == -1)
    
    las_r_same = (behav['side'] == -1) & (behav['choice'] == -1) & (behav['laser'] == -1) & (behav['prev_reward'] == -1)
    las_r0_same = (behav['side'] == -1) & (behav['choice'] == -1) & (behav['laser'] == 0) & (behav['prev_reward'] == -1)
    las_r_diff = (behav['side'] == -1) & (behav['choice'] == -1) & (behav['laser'] == -1) & (behav['prev_reward'] == 1)
    las_r0_diff = (behav['side'] == -1) & (behav['choice'] == -1) & (behav['laser'] == 0) & (behav['prev_reward'] == 1)

    
    stats_add= {}
    time_period = 'resp'
    for trace_type in trace_types:
        for metric in ['p_', 'diff_']:
            for side in ['l_', 'r_']:
                stats_add[metric + 'lass_' + side + 'post_laser' + '_' + trace_type] = []
               # stats_add[metric + 'las_same_' + side + time_period + '_' + trace_type] = []
              #  stats_add[metric + 'las_diff_' + side + time_period + '_' + trace_type] = []
              #  stats_add[metric + 'las_oomf_' + side + time_period + '_' + trace_type] = []
            
    for trace_type in trace_types:
        aucs_neu_list = aucs[trace_type][time_period]
        aucs_neu_np = np.array(aucs_neu_list)
        for n in np.arange(n_neu):
            if n % 100 == 0:
                print(n, trace_type, time_period)
            # select the neuron's aucs
            aucs_neu_n = aucs_neu_np[:,n]
            # Stats for laser effects (all correct)
            # Left
            aucs_y = [elem for i, elem in enumerate(aucs_neu_n) if las_ll.iloc[i]]
            aucs_n = [elem for i, elem in enumerate(aucs_neu_n) if las_ll0.iloc[i]]
            p_side = permutation_test(aucs_y, aucs_n, n_permut) 
            diff_side = np.mean(aucs_y) - np.mean(aucs_n) 
            stats_add['p_lass_l_' + time_period + '_' + trace_type].append(p_side)
            stats_add['diff_lass_l_' + time_period + '_' + trace_type].append(diff_side)
            
            # Right
            aucs_y = [elem for i, elem in enumerate(aucs_neu_n) if las_rr.iloc[i]]
            aucs_n = [elem for i, elem in enumerate(aucs_neu_n) if las_rr0.iloc[i]]
            p_side = permutation_test(aucs_y, aucs_n, n_permut) 
            diff_side = np.mean(aucs_y) - np.mean(aucs_n) 
            stats_add['p_lass_r_' + time_period + '_' + trace_type].append(p_side)
            stats_add['diff_lass_r_' + time_period + '_' + trace_type].append(diff_side)
            '''
            # Those fucking oomf lasers
            # Left
            # same
            aucs_y = [elem for i, elem in enumerate(aucs_neu_n) if las_l_same.iloc[i]]
            aucs_n = [elem for i, elem in enumerate(aucs_neu_n) if las_l0_same.iloc[i]]
            p_side = permutation_test(aucs_y, aucs_n, n_permut) 
            diff_side = np.mean(aucs_y) - np.mean(aucs_n) 
            stats_add['p_las_same_l_' + time_period + '_' + trace_type].append(p_side)
            stats_add['diff_las_same_l_' + time_period + '_' + trace_type].append(diff_side)
            # diff
            aucs_y = [elem for i, elem in enumerate(aucs_neu_n) if las_l_diff.iloc[i]]
            aucs_n = [elem for i, elem in enumerate(aucs_neu_n) if las_l0_diff.iloc[i]]
            p_side = permutation_test(aucs_y, aucs_n, n_permut) 
            diff_side = np.mean(aucs_y) - np.mean(aucs_n) 
            stats_add['p_las_diff_l_' + time_period + '_' + trace_type].append(p_side)
            stats_add['diff_las_diff_l_' + time_period + '_' + trace_type].append(diff_side)

            # Right
            # same
            aucs_y = [elem for i, elem in enumerate(aucs_neu_n) if las_r_same.iloc[i]]
            aucs_n = [elem for i, elem in enumerate(aucs_neu_n) if las_r0_same.iloc[i]]
            p_side = permutation_test(aucs_y, aucs_n, n_permut) 
            diff_side = np.mean(aucs_y) - np.mean(aucs_n) 
            stats_add['p_las_same_r_' + time_period + '_' + trace_type].append(p_side)
            stats_add['diff_las_same_r_' + time_period + '_' + trace_type].append(diff_side)
            # diff
            aucs_y = [elem for i, elem in enumerate(aucs_neu_n) if las_r_diff.iloc[i]]
            aucs_n = [elem for i, elem in enumerate(aucs_neu_n) if las_r0_diff.iloc[i]]
            p_side = permutation_test(aucs_y, aucs_n, n_permut) 
            diff_side = np.mean(aucs_y) - np.mean(aucs_n) 
            stats_add['p_las_diff_r_' + time_period + '_' + trace_type].append(p_side)
            stats_add['diff_las_diff_r_' + time_period + '_' + trace_type].append(diff_side)

            # Those fucking ELABORATE oomf lasers
            # Left oomf laser
            aucs_y = [elem for i, elem in enumerate(aucs_neu_n) if las_l_diff.iloc[i]]
            aucs_n = [elem for i, elem in enumerate(aucs_neu_n) if las_l_same.iloc[i]]
            p_side = permutation_test(aucs_y, aucs_n, n_permut) 
            diff_side = np.mean(aucs_y) - np.mean(aucs_n) 
            stats_add['p_las_oomf_l_' + time_period + '_' + trace_type].append(p_side)
            stats_add['diff_las_oomf_l_' + time_period + '_' + trace_type].append(diff_side)
            # Right oomf laser
            aucs_y = [elem for i, elem in enumerate(aucs_neu_n) if las_r_diff.iloc[i]]
            aucs_n = [elem for i, elem in enumerate(aucs_neu_n) if las_r_same.iloc[i]]
            p_side = permutation_test(aucs_y, aucs_n, n_permut) 
            diff_side = np.mean(aucs_y) - np.mean(aucs_n) 
            stats_add['p_las_oomf_r_' + time_period + '_' + trace_type].append(p_side)
            stats_add['diff_las_oomf_r_' + time_period + '_' + trace_type].append(diff_side)
            '''


    # append everything to the main mouse stats data
    for key in stats_add.keys():
        data[id]['stats'][key] = stats_add[key]
   
for id in mouse_ids:
    stats_exp = data[id]['stats']
    stats_exp.to_csv(path + id + '\\stats.csv', index=False)

In [ ]:
aucs 

In [ ]:
### Plot the laser affected neurons

In [ ]:
stats_add['p_las_oomf_l_' + time_period + '_' + trace_type]

In [ ]:
### More elaborate laser stats

In [ ]:
#code = {'l': 1, 'r': -1, 
    #    'pre': [-1000, 0], 'tone': [0, 1000], 
     #   'resp': [1000, 2000], 'resp2': [2000, 3000], 
     #   'anticip': [3000, 4000]}
time_periods = ['pre', 'tone', 'resp', 'resp2', 'anticip']
trace_types = ['deconv'] # HERE ONLY DECONVOLVED FOR NOW< BUT LATER ADD CA AS WELL
           
# number of permutations to perform; ideally 10000
n_permut = 10000

# For each mouse
for id in mouse_ids:
    print(f"::: Mouse  {id} :::")
    print(f":::============:::")
    '''
    if os.path.isfile(path + id + '\\stats.csv'):
        stats_mouse = pd.read_csv(path + id + '\\stats.csv')
        data[id]['stats'] = stats_mouse
        print(f'stats loaded for {id}')
        continue
    '''
    data_exp = data[id] 
    behav = data_exp['behav']
    frame_ts = data_exp['frame_ts']
    ts = frame_ts['real_ts'].tolist()
    aucs = data_exp['aucs']
    n_neu = len(data_exp['ca'])
    n_tr = behav.shape[0] 
   # if 'stats' not in data[id].keys():

    # from behav, make the masks for the selection of aucs
    
    # here goes nothing
    las_l_same = (behav['side'] == 1) & (behav['choice'] == 1) & (behav['laser'] == 1) & (behav['prev_reward'] == 1)
    las_l_diff = (behav['side'] == 1) & (behav['choice'] == 1) & (behav['laser'] == 1) & (behav['prev_reward'] == -1)    
    las_r_same = (behav['side'] == -1) & (behav['choice'] == -1) & (behav['laser'] == -1) & (behav['prev_reward'] == -1)
    las_r_diff = (behav['side'] == -1) & (behav['choice'] == -1) & (behav['laser'] == -1) & (behav['prev_reward'] == 1)


    
    stats_add= {}

    for trace_type, time_period in itertools.product(trace_types, time_periods):
        for metric in ['p_', 'diff_']:
            for side in ['l_', 'r_']:
                stats_add[metric + 'las_oomf_' + side + time_period + '_' + trace_type] = []
            
    for trace_type, time_period in itertools.product(trace_types, time_periods):
 #   for trace_type in ['ca', 'deconv']: 
   #     for time_period in time_periods:
        aucs_neu_list = aucs[trace_type][time_period]
        aucs_neu_np = np.array(aucs_neu_list)
        for n in np.arange(n_neu):
            if n % 100 == 0:
                print(n, trace_type, time_period)
            # select the neuron's aucs
            aucs_neu_n = aucs_neu_np[:,n]

            # Those fucking ELABORATE oomf lasers
            # Left oomf laser
            aucs_y = [elem for i, elem in enumerate(aucs_neu_n) if las_l_diff.iloc[i]]
            aucs_n = [elem for i, elem in enumerate(aucs_neu_n) if las_l_same.iloc[i]]
            p_side = permutation_test(aucs_y, aucs_n, n_permut) 
            diff_side = np.mean(aucs_y) - np.mean(aucs_n) 
            stats_add['p_las_oomf_l_' + time_period + '_' + trace_type].append(p_side)
            stats_add['diff_las_oomf_l_' + time_period + '_' + trace_type].append(diff_side)
            # Right oomf laser
            aucs_y = [elem for i, elem in enumerate(aucs_neu_n) if las_r_diff.iloc[i]]
            aucs_n = [elem for i, elem in enumerate(aucs_neu_n) if las_r_same.iloc[i]]
            p_side = permutation_test(aucs_y, aucs_n, n_permut) 
            diff_side = np.mean(aucs_y) - np.mean(aucs_n) 
            stats_add['p_las_oomf_r_' + time_period + '_' + trace_type].append(p_side)
            stats_add['diff_las_oomf_r_' + time_period + '_' + trace_type].append(diff_side)
        


    # append everything to the main mouse stats data
    for key in stats_add.keys():
        data[id]['stats'][key] = stats_add[key]
   
for id in mouse_ids:
    stats_exp = data[id]['stats']
    stats_exp.to_csv(path + id + '\\stats.csv', index=False)

In [ ]:
# summary stats for laser
p_thr = 0.05
fig, axs = plt.subplots(1, 2, figsize=(15, 3))
trace_type = 'deconv'
all_ls = []; all_rs = []
for id in mouse_ids:
    ls = []; rs = []
    print(f"::: Mouse  {id} :::")
    stats_side = data[id]['stats']
    for time_period in time_periods:
        condition = time_period + '_' + trace_type
        neu_l = stats_side[ (stats_side['diff_las_l_' + condition] > 0) &
                               (stats_side['p_las_l_' + condition] < p_thr)  ].shape[0]
        neu_r = stats_side[(stats_side['diff_las_l_' + condition] < 0) &
                               (stats_side['p_las_l_' + condition] < p_thr)  ].shape[0]
      #  print(f'{time_period} L: {neu_l}, R: {neu_r}')
        ls.append(neu_l); rs.append(neu_r)
    all_ls.append(ls); all_rs.append(rs); times = range(5)
    
  #  axs.plot(range(5), ls, 'b')
   # axs.plot(range(5), rs, 'r')
  #  axs.set_xticks(range(5), ['pre', 'tone', 'resp', 'resp2', 'anticip'])
plot_aver_sem_canon(all_ls, times, 'b', ghost_traces=True,
                       span = [0,5], sampling_rate = 1)   
plot_aver_sem_canon(all_rs, times, 'g', ghost_traces=True,
                       span = [0,5], sampling_rate = 1)   
axs[0].set_xticks(range(5), ['pre', 'tone', 'resp', 'resp2', 'anticip'])    
plt.show()

In [ ]:
stats_side

In [ ]:
# summary stats for laser
p_thr = 0.05
fig, axs = plt.subplots(1, 2, figsize=(15, 3))
trace_type = 'deconv'
all_ls = []; all_rs = []
all_dls = []; all_drs = []
for id in mouse_ids:
    ls = []; rs = []
    dls = []; drs = []
    print(f"::: Mouse  {id} :::")
    stats_side = data[id]['stats']
    for time_period in time_periods:
        condition = time_period + '_' + trace_type
        neu_l = stats_side[ (stats_side['diff_las_same_r_' + condition] > 0) &
                               (stats_side['p_las_same_r_' + condition] < p_thr)  ].shape[0]
        neu_r = stats_side[(stats_side['diff_las_same_r_' + condition] < 0) &
                               (stats_side['p_las_same_r_' + condition] < p_thr)  ].shape[0]
        neu_dl = stats_side[ (stats_side['diff_las_diff_r_' + condition] > 0) &
                               (stats_side['p_las_diff_r_' + condition] < p_thr)  ].shape[0]
        neu_dr = stats_side[(stats_side['diff_las_diff_r_' + condition] < 0) &
                               (stats_side['p_las_diff_r_' + condition] < p_thr)  ].shape[0]
      #  print(f'{time_period} L: {neu_l}, R: {neu_r}')
        ls.append(neu_l); rs.append(neu_r)
        dls.append(neu_dl); drs.append(neu_dr)
    all_ls.append(ls); all_rs.append(rs)
    all_dls.append(dls); all_drs.append(drs)
    
  #  axs.plot(range(5), ls, 'b')
   # axs.plot(range(5), rs, 'r')
  #  axs.set_xticks(range(5), ['pre', 'tone', 'resp', 'resp2', 'anticip'])
plot_aver_sem_canon(all_ls, times, 'b', ghost_traces=True,
                       span = [0,5], sampling_rate = 1)   
plot_aver_sem_canon(all_rs, times, 'k', linestyle=':', ghost_traces=True,
                       span = [0,5], sampling_rate = 1)   
plot_aver_sem_canon(all_dls, times, 'c', ghost_traces=True,
                       span = [0,5], sampling_rate = 1)   
plot_aver_sem_canon(all_drs, times, 'g', linestyle=':', ghost_traces=True,
                       span = [0,5], sampling_rate = 1)   
axs[0].set_xticks(range(5), ['pre', 'tone', 'resp', 'resp2', 'anticip'])    
plt.show()

In [ ]:
# intersections in neurons' properties

In [ ]:
# Check if the laser active neurons are

### All the stats at once

In [ ]:
'''
code = {'l': 1, 'r': -1, 
        'pre': [-1000, 0], 'tone': [0, 1000], 
        'resp': [1000, 2000], 'resp2': [2000, 3000], 
        'anticip': [3000, 4000]}
time_periods = ['pre', 'tone', 'resp', 'resp2', 'anticip']
# P values to gather
stats_all = {'mouse': [], 'neu_n': [], 'usable': []}


for trace_type in ['ca', 'deconv']: 
    stats_all['p_prev_pre_' + trace_type] = [] # for previous reward
    stats_all['diff_prev_pre_' + trace_type] = [] 
    for time_period in time_periods:
        stats_all['p_side_' + time_period + '_' + trace_type] = []
        stats_all['diff_side_' + time_period + '_' + trace_type] = [] # difference of left - right; positive means selective for left
        for side in ['l', 'r']:
            stats_all['p_oomf_' + side + '_' + time_period + '_' + trace_type] = []
            stats_all['diff_oomf_' + side + '_' + time_period + '_' + trace_type] = [] # positive means oomf, negative - anti-oomf
            
# number of permutations to perform; ideally 10000
n_permut = 10000

# For each mouse
for id in mouse_ids:
    
    print(f"::: Mouse  {id} :::")
    print(f":::============:::")
    data_exp = data[id] 
    traces = data_exp['ca'] 
    deconvolveds = data_exp['deconv'] 
    behav = data_exp['behav'] 
    frame_ts = data_exp['frame_ts']
    ts = frame_ts['real_ts'].tolist()
    aucs = data_exp['aucs']
    n_tr = behav.shape[0] 
    for n in np.arange(traces.shape[0]):
        if n % 100 == 0:
            print(n)
        # get the traces
        traces_both  = {'ca': traces[n][:], 'deconv': deconvolveds[n][:]}
        stats_all['mouse'].append(id)
        stats_all['neu_n'].append(n)
        stats_all['usable'].append( (data_exp['fair_neurons'][n] + data_exp['good_neurons'][n]) != 0)
        for time_period in time_periods:
            for trace_type in ['ca', 'deconv']: 
                # select all aucs for the relevant neuron, time period trace type
                aucs_neu_list = aucs[trace_type][time_period]
                aucs_neu = pd.DataFrame(aucs_neu_list)
                aucs_neu_n = aucs_neu[n]
                # calculate stats for all conditions and fill out the table
                # side
                aucs_l = aucs_neu[ (behav['side']==1) & (behav['choice']==1) &  (behav['free']==0)]
                aucs_r = aucs_neu[ (behav['side']==-1) & (behav['choice']==-1)  &  (behav['free']==0)]
                p_side = permutation_test(aucs_l, aucs_r, n_permut) 
                diff_side = np.mean(aucs_l) - np.mean(aucs_r)         
                stats_all['p_side_' + time_period + '_' + trace_type].append(p_side)
                stats_all['diff_side_' + time_period + '_' + trace_type].append(diff_side)
                # this is ugly but oh well
                # add the pre trial activity stats based on prev reward
                if time_period == 'pre':
                    aucs_l = aucs_neu[ (behav['prev_reward']==1)   ]
                    aucs_r = aucs_neu[ (behav['prev_reward']==-1)  ]
                    p_side_prev = permutation_test(aucs_l, aucs_r, n_permut) 
                    diff_side_prev = np.mean(aucs_l) - np.mean(aucs_r)   
                    stats_all['p_prev_pre_' + trace_type].append(p_side_prev)
                    stats_all['diff_prev_pre_' + trace_type].append(diff_side_prev)
                # add the oomfs for left and right
                for side in ['l', 'r']:
                    s = code[side]
                    aucs_same = aucs_neu[ (behav['side']==s) & (behav['choice']==s) & (behav['prev_reward']==s) & 
                                            (behav['free']==0)]
                    aucs_diff = aucs_neu[ (behav['side']==s) & (behav['choice']==s) & (behav['prev_reward']==-s) & 
                                            (behav['free']==0)]
                    p_oomf = permutation_test(aucs_same, aucs_diff, n_permut) 
                    diff_oomf = np.mean(aucs_diff) - np.mean(aucs_same)   
                    stats_all['p_oomf_' + side + '_' + time_period + '_' + trace_type].append(p_oomf)
                    stats_all['diff_oomf_' + side + '_' + time_period + '_' + trace_type].append(diff_oomf)
    stats_all_df = pd.DataFrame(stats_all)
    stats_exp = stats_all_df[stats_all_df['mouse']==id]
    data[id]['stats'] = stats_exp
    stats_exp.to_csv(path + id + '\\stats.csv', index=False)
stats_all_df = pd.DataFrame(stats_all)
'''
print('decided not to do all the stats at once')

# Main effects, saved stats

### Prev trial aftereffects

In [ ]:

n_permut = 2000
# For each mouse
for id in mouse_ids:
    # P values to gather
    print(f"::: Mouse  {id} :::")
    data_exp = data[id]
   # if 'stats' in data_exp.keys():
  #      continue
    behav = data_exp['behav'].reset_index(drop=True)
    n_tr = behav.shape[0]
    trace_type = 'deconv'
    aucs_trace_type = data_exp['aucs'][trace_type]
    neu_n = data_exp['ca'].shape[0]
    neus = np.arange(neu_n);

                         
    for time_period in ['pre']:
        aucs_period = pd.DataFrame(aucs_trace_type[time_period])
        p_vals = []; diffs = []

        for n in neus:
            aucs_neu = aucs_period[n]
            # Select auc subsets for conditions
            auc_l = aucs_neu[(behav['prev_reward']==1) ]
            auc_r = aucs_neu[(behav['prev_reward']==-1) ]
            
            p_val = permutation_test(auc_l, auc_r, n_permut)  
            diff = np.mean(auc_l) - np.mean(auc_r)
            p_vals.append(p_val)
            diffs.append(diff)


        data[id]['stats']['p_prev_' + time_period + '_' + trace_type] = p_vals
        data[id]['stats']['diff_prev_' + time_period + '_' + trace_type] = diffs
        # save them stat
    with open(path + id + '\\stats\\stats.pkl', 'wb') as f:
        pickle.dump(stats_side_mouse, f)
    

## Side preference

In [ ]:
code_cond = {'l': 1, 'r': -1, 'same': 1, 'diff': -1}
time_periods = {'pre': [-1000, 0], 'tone': [0, 1000], 
                'resp': [1000, 2000], 'resp2': [2000, 3000], 
                'anticip': [3000, 4000]}
n_permut = 2000
# For each mouse
for id in mouse_ids:
    # P values to gather
    print(f"::: Mouse  {id} :::")
    
    data_exp = data[id]
   # if 'stats' in data_exp.keys():
  #      continue 
    behav = data_exp['behav'].reset_index(drop=True)
    n_tr = behav.shape[0]
    trace_type = 'deconv'
    aucs_trace_type = data_exp['aucs'][trace_type]
    usable = data_exp['usable_neurons']; 
    neu_n = data_exp['ca'].shape[0]
    neus = np.arange(neu_n);
    stats_side_mouse = pd.DataFrame({ 'neu': neus, 'usable': usable})
                         
    for time_period in time_periods.keys():
        aucs_period = pd.DataFrame(aucs_trace_type[time_period])
        p_vals = []; diffs = []
        for n in neus:
            aucs_neu = aucs_period[n]
            # Select auc subsets for conditions
            auc_conds = {}
            for side in ['l', 'r']:
                side_val = code_cond[side]
                auc_conds[side] = aucs_neu[(behav['side']==side_val) & 
                                           (behav['missed'] == 0) & (behav['laser']==0) ]
            p_val = permutation_test(auc_conds['l'], auc_conds['r'], n_permut)  
            diff = np.mean(auc_conds['l']) - np.mean(auc_conds['r'])
            p_vals.append(p_val)
            diffs.append(diff)

        stats_side_mouse['p_side_' + time_period+ '_' + trace_type] = p_vals
        stats_side_mouse['diff_' + time_period+ '_' + trace_type] = diffs
        # save them stats
    with open(path + id + '\\stats\\stats_side.pkl', 'wb') as f:
        pickle.dump(stats_side_mouse, f)
    data[id]['stats'] = stats_side_mouse

## Display based on stats

In [ ]:
trace_type = 'deconv'
time_periods = ['post_laser']
all_diffs_l = []
all_diffs_r = []
# For each mouse
mice_ids = mouse_ids
for id in ['os87']:
    print(f"::: Mouse {id}  :::")
    print(f":::=============:::")
    data_exp = data[id]
    traces = data_exp['ca']
    deconvolveds = data_exp['deconv']
    behav = data_exp['behav']
    frame_ts = data_exp['frame_ts']
    ts = frame_ts['real_ts'].tolist()
    n_tr = behav.shape[0]
    correct_l_ts = []
    correct_r_ts = []
    stats_mouse = data_exp['stats']
    n_to_disp = 10
    for window in time_periods:
        print(f":::==== {window} ====:::")
        condition = window + '_' + trace_type
        pref_l = stats_mouse[stats_mouse['p_las_l_' + condition] < p_thr]
        pref_r = stats_mouse[stats_mouse['p_las_r_' + condition] < p_thr]
       # pref_l = stats_mouse[stats_mouse['diff_side_' + window + '_deconv'] > 0]
      #  pref_r = stats_mouse[stats_mouse['diff_side_' + window + '_deconv'] < 0]
        diffs_l = pref_l.nsmallest(n_to_disp, 'p_las_l_' + condition)['diff_las_l_' + condition].tolist()
        diffs_r = pref_l.nsmallest(n_to_disp, 'p_las_r_' + condition)['diff_las_r_' + condition].tolist()
        all_diffs_l += diffs_l; all_diffs_r += diffs_r;
        pref_l_neu = pref_l.nsmallest(n_to_disp, 'p_las_l_' + condition)['neu_n'].tolist()
        pref_r_neu = pref_r.nsmallest(n_to_disp, 'p_las_r_' + condition)['neu_n'].tolist()
        
        pref_l_neu.extend(pref_r_neu)

        for n in pref_l_neu:
            # prep the figure
            fig, axs = plt.subplots(1, 2, figsize=(12, 3))
            
            # side
          #  deconvolved = traces[n]
            deconvolved = deconvolveds[n]
            params = ['side',  'laser']
            segms_same, segm_ts_same = get_segments(behav, deconvolved, frame_ts, params, [-1,0], span=[1000,4000])
            segms_diff, segm_ts_diff = get_segments(behav, deconvolved, frame_ts, params, [-1,-1], span=[1000,4000])
            segms_same_r, segm_ts_same_r = get_segments(behav, deconvolved, frame_ts, params, [1,0], span=[1000,4000])
            segms_diff_r, segm_ts_diff_r = get_segments(behav, deconvolved, frame_ts, params, [1,1], span=[1000,4000])
            
            plot_aver_sem(segms_same, segm_ts_same, 'r', ghost_traces=True)
            plot_aver_sem(segms_diff, segm_ts_diff, 'r', linestyle=':', ghost_traces=True)
            plot_aver_sem(segms_same_r, segm_ts_same_r, 'b', ghost_traces=True)
            plot_aver_sem(segms_diff_r, segm_ts_diff_r, 'b', linestyle=':', ghost_traces=True)

            goodfont=14
            for ax_n in [0,1]:
                axs[ax_n].axvline(x=0, linestyle=':')
                axs[ax_n].axvline(x=1150, linestyle=':')
                axs[ax_n].set_xlabel('time, ms', fontsize=goodfont)
                axs[ax_n].set_ylabel('FRE', fontsize=goodfont)
                axs[ax_n].set_title(id + ' ' + str(n), fontsize=goodfont)
            print(f' mouse {id}, neuron {n} ')
            plt.tight_layout()
            if n in to_plot_las[id]:
                plt.savefig('D:\\papers_thesis\\figs\\' + 'las_effect_' + id + '_' + str(n) + '.png', dpi=300)
        
            #axs[0].set_yticks(np.arange(0, 3, .5), np.arange(0, 300, 50))

          #  plt.ylabel('PC')

            plt.show()


In [ ]:
to_plot_las = {'os85a':[141], 'os85b':[], 'os86a':[39, 503, 61], 'os86b':[], 'os87':[], 'os88':[], 'os89':[]}

In [ ]:
pos_diffs_l = [a>0 for a in all_diffs_l]
pos_diffs_r = [a>0 for a in all_diffs_r]
print(sum(pos_diffs_l),sum(pos_diffs_r))

### Summary of laser effects

In [ ]:
stats_side['p_las_l_post_laser_deconv']

In [ ]:
stats_side[stats_side['usable']!=0] 
all_diff_ls

In [ ]:
mice_ids = ['os85b', 'os86a', 'os86b', 'os87']

In [ ]:

# Creating the subplots
fig, axes = plt.subplots(1, 2, figsize=(12, 6), gridspec_kw={'width_ratios': [1, 2]})
width_bar = 0.5
labels_2 = ['Left laser', 'Right laser']
means_2 = [125, 64] 
std_1 = [42.2, 36]
# Second subplot
bars = axes[0].bar(labels_2, means_2,  yerr=std_1, capsize=10, color=['blue', 'red'], width=width_bar)

#axes[0].set_title('A', loc='left')
axes[0].set_ylabel('Accuracy', fontsize=14)
axes[0].set_xlabel('Conditions', fontsize=14)

# First subplot
labels_1 = ['Left Excit', 'Left Inhib', 'Right Excit', 'Right Inhib']
means_1 = [77, 48, 49, 15]
bars2 = axes[1].bar(labels_1, means_1, capsize=10, color=['blue', 'blue', 'red', 'red'], width=width_bar)
axes[1].set_title('B', loc='left')

axes[1].set_xlabel('Conditions', fontsize=14)
for ax_n in [0,1]:
    axes[ax_n].set_ylabel('Number of neurons', fontsize=14)
   # axes[ax_n].set_ylim([0, 1])
  #  axes[ax_n].set_xticks(fontsize=12)
  #  axes[ax_n].set_yticks(fontsize=12)
    axes[ax_n].spines['right'].set_visible(False)
    axes[ax_n].spines['top'].set_visible(False)
axes[0].set_xticklabels(labels_2, fontsize=14)
# Adjust layout
plt.tight_layout()
plt.savefig('D:\\papers_thesis\\figs\\' + 'laser_accuracy_prev.png', dpi=300)

plt.show()



In [ ]:

# Creating the subplots
fig, axes = plt.subplots(1, 2, figsize=(12, 6), gridspec_kw={'width_ratios': [1, 2]})
width_bar = 0.5
labels_2 = ['Left laser', 'Right laser']
means_2 = [125, 64] 
std_1 = [53.2
# Second subplot
bars = axes[0].bar(labels_2, means_2,  yerr=std_1 capsize=10, color=['blue', 'red'], width=width_bar)

axes[0].set_title('A', loc='left')
axes[0].set_ylabel('Accuracy')
axes[0].set_xlabel('Conditions')

# First subplot
labels_1 = ['Left Excit', 'Left Inhib', 'Right Excit', 'Right Inhib']
means_1 = [77, 48, 49, 15]
bars2 = axes[1].bar(labels_1, means_1, capsize=10, color=['blue', 'blue', 'red', 'red'], width=width_bar)
axes[1].set_title('B', loc='left')

axes[1].set_xlabel('Conditions')
for ax_n in [0,1]:
    axes[ax_n].set_ylabel('Number of neurons')
   # axes[ax_n].set_ylim([0, 1])
    axes[ax_n].spines['right'].set_visible(False)
    axes[ax_n].spines['top'].set_visible(False)

# Adjust layout
plt.tight_layout()
plt.savefig('D:\\papers_thesis\\figs\\' + 'laser_accuracy_prev.png', dpi=300)

plt.show()



In [ ]:
pos_diff_ls = [a>0 for a in all_diff_ls]
pos_diff_rs = [a>0 for a in all_diff_rs]
print(sum(pos_diff_ls), 'of', len(pos_diff_ls), 'left laser positive')
print(sum(pos_diff_rs), 'of', len(pos_diff_rs), 'right laser positive')

In [ ]:
17/(89+36)

In [ ]:
ls = []; rs = []
for id in mouse_ids:
    molas = mouse_las[id]
    if id == 'os87':
        ls.append(25); rs.append(len(molas['r']))
    else:
        ls.append(len(molas['l'])); rs.append(len(molas['r']))
    print(id, 'l:', len(molas['l']), 'r:', len(molas['r']))
print(np.std(ls), np.mean(ls), np.std(rs), np.mean(rs))

In [ ]:
# summary stats
p_thr = 0.05
#fig, axs = plt.subplots(1, 2, figsize=(15, 3))
trace_type = 'deconv'
all_ls = []; all_rs = []
all_diff_ls = []; all_diff_rs = []
mouse_las = {}
all_side_guys = 0; all_oomf_guys = 0; all_prew_guys = 0
for id in mouse_ids:
    side_guys = []; oomf_guys = []; prew_guys = []
    ls = []; rs = []
    mouse_las_l = []
    mouse_las_r = []
    print(f"::: Mouse  {id} :::")
    stats_og = data[id]['stats']
    stats_side = stats_og[stats_og['usable']!=0]
    for time_period in ['resp', 'resp2']:
        condition = time_period + '_' + trace_type
        p_neu_l = stats_side['p_las_l_' + condition]
        p_neu_r = stats_side['p_las_r_' + condition]
        sign_l, adjusted_p_values = fdr_correction(p_neu_l, alpha=p_thr)
        sign_r, adjusted_p_values = fdr_correction(p_neu_r, alpha=p_thr)
        neu_n_all =  stats_side['neu_n']
        neu_n_l = neu_n_all[sign_l].tolist()
        neu_n_r = neu_n_all[sign_r].tolist()
        stats_l = stats_og.iloc[neu_n_l]
        stats_r = stats_og.iloc[neu_n_r]
        if stats_l.shape[0] != 0:
            for col in stats_l.keys():
                if (col[0] == 'p') & (col[-1] == 'v') & ('las' not in col):
                    sign_vals = stats_l[stats_l[col]<0.01]
                    if sign_vals.shape[0] != 0:
                     #   print(col, sign_vals.shape[0], 
                         #     sign_vals['neu_n'].tolist(), sign_vals[col].tolist())
                        if 'side' in col:
                            side_guys += sign_vals['neu_n'].tolist()
                        elif 'nomf' in col:
                            oomf_guys += sign_vals['neu_n'].tolist()
                        elif 'prew' in col: 
                            prew_guys += sign_vals['neu_n'].tolist()
            
        
        p_l = stats_side[stats_side['p_las_l_' + condition]<p_thr].shape[0]
        p_r = stats_side[stats_side['p_las_r_' + condition]<p_thr].shape[0]
        diffs_l = stats_side['diff_las_l_' + condition][sign_l].tolist()
        diffs_r = stats_side['diff_las_r_' + condition][sign_r].tolist()
        ls+= diffs_l; rs+= diffs_r; 
        all_diff_ls += diffs_l; all_diff_rs += diffs_r
      #  print(sum(sign_l), sum(sign_r))
    side_guys = list(set(side_guys)); all_side_guys += len(side_guys)
    oomf_guys = list(set(oomf_guys)); all_oomf_guys += len(oomf_guys)
    prew_guys = list(set(prew_guys)); all_prew_guys += len(prew_guys)
    print(side_guys, oomf_guys, prew_guys)
    mouse_las[id] = {'l': ls, 'r': rs}
        #print(p_l, p_r)
        #stats_side[sign_l]
      #  print(f'{time_period} L: {neu_l}, R: {neu_r}')
      #  ls.append(neu_l); rs.append(neu_r)
  #  all_ls.append(ls); all_rs.append(rs); times = range(5)
    
  #  axs.plot(range(5), ls, 'b')
   # axs.plot(range(5), rs, 'r')
  #  axs.set_xticks(range(5), ['pre', 'tone', 'resp', 'resp2', 'anticip'])
#plot_aver_sem_canon(all_ls, times, 'b', ghost_traces=True,
#                       span = [0,5], sampling_rate = 1)   
#plot_aver_sem_canon(all_rs, times, 'r', ghost_traces=True,
 #                      span = [0,5], sampling_rate = 1)   
#axs[0].set_xticks(range(5), ['pre', 'tone', 'resp', 'resp2', 'anticip'])    
#plt.show()
print(all_side_guys, all_oomf_guys,all_prew_guys)

## Summary oomf

In [ ]:
### # DIFFERENT WAY
p_thr = 0.2
oomf_neus_all = {'oomf': {'l': [], 'r':[]}, 'nomf': {'l': [], 'r':[]}}
trace_type = 'deconv'
all_ls = []; all_rs = []
all_nls = []; all_nrs = []
#fig, axs = plt.subplots(1, len(mouse_ids), figsize=(15, 2))
plot_n = 0
times = [np.arange(5)]*len(mouse_ids)
oomfs_l = []
oomfs_r = []
all_o = 0; all_n = 0
all_os = []; all_ns = []
k=1
time_periods = ['pre', 'tone', 'resp', 'resp2', 'anticip']
oomph = '_oomf_'
time_periods = time_periods[1:]
for id in mouse_ids:
    ls = 0; rs = 0
    nls = 0; nrs = 0
    print(f"::: Mouse  {id} :::")
    stats_side = data[id]['stats']
    n_neus = stats_side[stats_side['usable']!=0].shape[0]
    stats_side = stats_side[ stats_side['usable']!=0 ]
                       #  & (stats_side['p_prew_pre_' + trace_type] > p_thr)
                       #  & (stats_side['p_prew_tone_' + trace_type] > p_thr*k)
                        # & (stats_side['p_oomf_l_pre_' + trace_type] > p_thr*k)
                       #  & (stats_side['p_oomf_r_pre_' + trace_type] > p_thr*k)
   #                         ]
    for time_period in time_periods:
        condition = time_period + '_' + trace_type
        p_oomf_l_all = stats_side['p' + oomph + 'l_' + condition] 
        p_oomf_r_all = stats_side['p' + oomph + 'r_' + condition] 
        signif_l, p_vals_adj_l = fdr_correction(p_oomf_l_all, alpha=p_thr)
        signif_r, p_vals_adj_r = fdr_correction(p_oomf_r_all, alpha=p_thr)
        neu_ol = stats_side[ (stats_side['diff' + oomph + 'l_' + condition] > 0) & signif_l ].shape[0]
        neu_or = stats_side[ (stats_side['diff' + oomph + 'r_' + condition] > 0) & signif_r ].shape[0]
        neu_nl = stats_side[ (stats_side['diff' + oomph + 'l_' + condition] < 0) & signif_l ].shape[0]
        neu_nr = stats_side[ (stats_side['diff' + oomph + 'r_' + condition] < 0) & signif_r ].shape[0]
        all_os.append(neu_ol); all_os.append(neu_or); 
        all_ns.append(neu_nl); all_ns.append(neu_nr); 
        all_o += neu_ol + neu_or; all_n += neu_nl + neu_nr;
        print((neu_ol + neu_or), 'vs', (neu_nl + neu_nr))
        ls += neu_ol; rs += neu_or; 
        nls += neu_nl; nrs += neu_nr; 

    oomf_neus_all['oomf']['l'].append(ls)
    oomf_neus_all['oomf']['r'].append(rs)
    oomf_neus_all['nomf']['l'].append(nls)
    oomf_neus_all['nomf']['r'].append(nrs)

      
oomf_neus_all


In [ ]:
oomf_neus_all

In [ ]:
stats_side[stats_side['usable']!=0]

In [ ]:
# summary stats
p_thr = 0.05
fig, axs = plt.subplots(1, 2, figsize=(15, 3))
trace_type = 'deconv'
all_ls = []; all_rs = []
for id in mouse_ids:
    ls = []; rs = []
    print(f"::: Mouse  {id} :::")
    stats_side = data[id]['stats']
    for time_period in time_periods:
        condition = time_period + '_' + trace_type
        neu_l = stats_side[ (stats_side['diff_' + condition] > 0) &
                               (stats_side['p_side_' + condition] < p_thr)  ].shape[0]
        neu_r = stats_side[(stats_side['diff_' + condition] < 0) &
                               (stats_side['p_side_' + condition] < p_thr)  ].shape[0]
      #  print(f'{time_period} L: {neu_l}, R: {neu_r}')
        ls.append(neu_l); rs.append(neu_r)
    all_ls.append(ls); all_rs.append(rs); times = range(5)
    
  #  axs.plot(range(5), ls, 'b')
   # axs.plot(range(5), rs, 'r')
  #  axs.set_xticks(range(5), ['pre', 'tone', 'resp', 'resp2', 'anticip'])
plot_aver_sem_canon(all_ls, times, 'b', ghost_traces=True,
                       span = [0,5], sampling_rate = 1)   
plot_aver_sem_canon(all_rs, times, 'r', ghost_traces=True,
                       span = [0,5], sampling_rate = 1)   
axs[0].set_xticks(range(5), ['pre', 'tone', 'resp', 'resp2', 'anticip'])    
plt.show()

In [ ]:
# Laser effects
code_cond = {'l': 1, 'r': -1, 'same': 1, 'diff': -1}
time_periods = {'pre': [-1000, 0], 'tone': [0, 1000], 
                'resp': [1000, 2000], 'resp2': [2000, 3000], 
                'anticip': [3000, 4000]}
n_permut = 2000
# For each mouse
for id in mouse_ids:
    # P values to gather
    stats_side_mouse = {'neu_n': [], 'usable': []}
    print(f"::: Mouse  {id} :::")
    
    data_exp = data[id]
   # if 'stats' in data_exp.keys():
  #      continue
    behav = data_exp['behav'].reset_index(drop=True)
    n_tr = behav.shape[0]
    trace_type = 'deconv'
    aucs_trace_type = data_exp['aucs'][trace_type]
    neu_n = data_exp['ca'].shape[0]
    neus = np.arange(neu_n);

                         
    for time_period in time_periods.keys():
        aucs_period = pd.DataFrame(aucs_trace_type[time_period])
        p_vals = {'l':[],'r':[]}; diffs = {'l':[],'r':[]}

        for n in neus:
            aucs_neu = aucs_period[n]
            # Select auc subsets for conditions
            auc_conds = {}
            for side in ['l', 'r']:
                side_val = code_cond[side]
                auc_las = aucs_neu[(behav['side']==side_val) & 
                                           (behav['missed'] == 0) & (behav['laser']==side_val) ]
                auc_nolas = aucs_neu[(behav['side']==side_val) & 
                                           (behav['missed'] == 0) & (behav['laser']==0) ]
                
                p_val = permutation_test(auc_las, auc_nolas, n_permut)  
                diff = np.mean(auc_las) - np.mean(auc_nolas)
                p_vals[side].append(p_val)
                diffs[side].append(diff)
        for side in ['l', 'r']:

            data[id]['stats']['p_las_' + side + '_' + time_period + '_' + trace_type] = p_vals[side]
            data[id]['stats']['diff_las_' + side + '_' + time_period + '_' + trace_type] = diffs[side]
        # save them stats
    with open(path + id + '\\stats\\stats.pkl', 'wb') as f:
        pickle.dump(stats_side_mouse, f)
    

In [ ]:
# summary stats
p_thr = 0.01
fig, axs = plt.subplots(1, 2, figsize=(15, 3))
trace_type = 'deconv'
all_ls = []; all_rs = []
all_dls = []; all_drs = []
times = [range(4)]*len(mouse_ids)
for id in mouse_ids:
    ls = []; rs = []
    dls = []; drs = []
    print(f"::: Mouse  {id} :::")
    stats_side = data[id]['stats']
    for time_period in time_periods:
        condition =  time_period + '_' + trace_type
        neu_l = stats_side[ 
                               (stats_side['p_las_same_l_' + condition] < p_thr)  ].shape[0]
        neu_r = stats_side[
                               (stats_side['p_las_same_r_' + condition] < p_thr)  ].shape[0]
        neu_dl = stats_side[ 
                               (stats_side['p_las_diff_l_' + condition] < p_thr)  ].shape[0]
        neu_dr = stats_side[
                               (stats_side['p_las_diff_r_' + condition] < p_thr)  ].shape[0]
      #  print(f'{time_period} L: {neu_l}, R: {neu_r}')
        ls.append(neu_l); rs.append(neu_r)
        dls.append(neu_dl); drs.append(neu_dr)
    all_ls.append(ls); all_rs.append(rs); times = range(5)
    all_dls.append(dls); all_drs.append(drs); times = range(5)
    
  #  axs.plot(range(5), ls, 'b')
   # axs.plot(range(5), rs, 'r')
  #  axs.set_xticks(range(5), ['pre', 'tone', 'resp', 'resp2', 'anticip'])
plot_aver_sem_canon(all_ls, times, 'b', ghost_traces=True,
                       span = [0,5], sampling_rate = 1)   
plot_aver_sem_canon(all_rs, times, 'r', ghost_traces=True,
                       span = [0,5], sampling_rate = 1)   
plot_aver_sem_canon(all_dls, times, 'c', ghost_traces=True,
                       span = [0,5], sampling_rate = 1)   
plot_aver_sem_canon(all_drs, times, 'm', ghost_traces=True,
                       span = [0,5], sampling_rate = 1)   
axs[0].set_xticks(range(5), ['pre', 'tone', 'resp', 'resp2', 'anticip'])
plt.show()

In [ ]:
# summary stats
p_thr = 0.05
fig, axs = plt.subplots(1, 2, figsize=(15, 3))
trace_type = 'deconv'
all_ls = []; all_rs = []
times = [range(4)]*len(mouse_ids)
for id in mouse_ids:
    ls = []; rs = []
    print(f"::: Mouse  {id} :::")
    stats_side = data[id]['stats']
    for time_period in time_periods:
        condition =  time_period + '_' + trace_type
        neu_l = stats_side[ 
                               (stats_side['p_las_same_l_' + condition] < p_thr)  ].shape[0]
        neu_r = stats_side[
                               (stats_side['p_las_same_r_' + condition] < p_thr)  ].shape[0]
      #  print(f'{time_period} L: {neu_l}, R: {neu_r}')
        ls.append(neu_l); rs.append(neu_r)
    all_ls.append(ls); all_rs.append(rs); times = range(5)
    
  #  axs.plot(range(5), ls, 'b')
   # axs.plot(range(5), rs, 'r')
  #  axs.set_xticks(range(5), ['pre', 'tone', 'resp', 'resp2', 'anticip'])
plot_aver_sem_canon(all_ls, times, 'b', ghost_traces=True,
                       span = [0,5], sampling_rate = 1)   
plot_aver_sem_canon(all_rs, times, 'r', ghost_traces=True,
                       span = [0,5], sampling_rate = 1)   
axs[0].set_xticks(range(5), ['pre', 'tone', 'resp', 'resp2', 'anticip'])
plt.show()

## Oomph stats

In [ ]:
# Laser effects
code_cond = {'l': 1, 'r': -1, 'same': 1, 'diff': -1}
time_periods = {'pre': [-1000, 0], 'tone': [0, 1000], 
                'resp': [1000, 2000], 'resp2': [2000, 3000], 
                'anticip': [3000, 4000]}
n_permut = 2000
# For each mouse
for id in mouse_ids:
    # P values to gather
    print(f"::: Mouse  {id} :::")  
    data_exp = data[id]
   # if 'stats' in data_exp.keys():
  #      continue
    behav = data_exp['behav'].reset_index(drop=True)
    n_tr = behav.shape[0]
    trace_type = 'deconv'
    aucs_trace_type = data_exp['aucs'][trace_type]
    neu_n = data_exp['ca'].shape[0]
    neus = np.arange(neu_n);
                         
    for time_period in time_periods.keys():
        aucs_period = pd.DataFrame(aucs_trace_type[time_period])
        p_vals = {'l':[],'r':[]}; diffs = {'l':[],'r':[]}

        for n in neus:
            aucs_neu = aucs_period[n]
            # Select auc subsets for conditions
            auc_conds = {}
            for side in ['l', 'r']:
                side_val = code_cond[side]
                auc_same = aucs_neu[(behav['side']==side_val) & (behav['choice']==side_val) & 
                                    (behav['prev_reward']==side_val) &
                                    (behav['missed'] == 0) & (behav['laser']==0) ]
                auc_diff = aucs_neu[(behav['side']==side_val) & (behav['choice']==side_val) & 
                                    (behav['prev_reward']== -side_val) &
                                    (behav['missed'] == 0) & (behav['laser']==0) ]
                
                p_val = permutation_test(auc_same, auc_diff, n_permut)  
                diff = np.mean(auc_diff) - np.mean(auc_same)
                p_vals[side].append(p_val)
                diffs[side].append(diff)
        for side in ['l', 'r']:
            data[id]['stats']['p_oomf_' + side + '_' + time_period + '_' + trace_type] = p_vals[side]
            data[id]['stats']['diff_oomf_' + side + '_' + time_period + '_' + trace_type] = diffs[side]
        # save them stats
    with open(path + id + '\\stats\\stats_oomph.pkl', 'wb') as f:
        pickle.dump(stats_side_mouse, f)
    

In [ ]:
# Laser effects
code_cond = {'l': 1, 'r': -1, 'same': 1, 'diff': -1}
time_periods = {'pre': [-1000, 0], 'tone': [0, 1000], 
                'resp': [1000, 2000], 'resp2': [2000, 3000], 
                'anticip': [3000, 4000]}
n_permut = 2000
# For each mouse
for id in mouse_ids:
    # P values to gather
    stats_side_mouse = {'neu_n': [], 'usable': []}
    print(f"::: Mouse  {id} :::")
    
    data_exp = data[id]
   # if 'stats' in data_exp.keys():
  #      continue
    behav = data_exp['behav'].reset_index(drop=True)
    n_tr = behav.shape[0]
    trace_type = 'ca'
    aucs_trace_type = data_exp['aucs'][trace_type]
    neu_n = data_exp['ca'].shape[0]
    neus = np.arange(neu_n);
                         
    for time_period in time_periods.keys():
        aucs_period = pd.DataFrame(aucs_trace_type[time_period])
        p_vals = {'l':[],'r':[]}; diffs = {'l':[],'r':[]}

        for n in neus:
            aucs_neu = aucs_period[n]
            # Select auc subsets for conditions
            auc_conds = {}
            for side in ['l', 'r']:
                side_val = code_cond[side]
                auc_same = aucs_neu[(behav['side']==side_val) & (behav['choice']==side_val) & 
                                    (behav['prev_reward']==side_val) &
                                    (behav['missed'] == 0) & (behav['laser']==0) ]
                auc_diff = aucs_neu[(behav['side']==side_val) & (behav['choice']==side_val) & 
                                    (behav['prev_reward']== -side_val) &
                                    (behav['missed'] == 0) & (behav['laser']==0) ]
                
                p_val = permutation_test(auc_same, auc_diff, n_permut)  
                diff = np.mean(auc_diff) - np.mean(auc_same)
                p_vals[side].append(p_val)
                diffs[side].append(diff)
        for side in ['l', 'r']:
            data[id]['stats']['p_oomf_' + side + '_' + time_period + '_' + trace_type] = p_vals[side]
            data[id]['stats']['diff_oomf_' + side + '_' + time_period + '_' + trace_type] = diffs[side]
        # save them stats
    with open(path + id + '\\stats\\stats_oomph.pkl', 'wb') as f:
        pickle.dump(stats_side_mouse, f)
    

In [ ]:
stats_side

In [ ]:


# summary stats
p_thr = 0.05
tps = ['pre', 'tone', 'resp', 'resp2', 'anticip']
fig, axs = plt.subplots(1, 2, figsize=(15, 3))
trace_type = 'deconv'
all_oomf_ls = []; all_oomf_rs = []
all_nomf_ls = []; all_nomf_rs = []
times = [range(4)]*len(mouse_ids)
for id in mouse_ids:
    oomf_ls = []; oomf_rs = []
    nomf_ls = []; nomf_rs = []
    print(f"::: Mouse  {id} :::")
    stats_side = data[id]['stats']
    for time_period in tps[1:]:
        oomf_l = stats_side[ (stats_side['diff_oomf_l_' + time_period + '_' + trace_type] > 0) &
                             (stats_side['p_oomf_l_' + time_period + '_' + trace_type] < p_thr) &
                             (stats_side['usable']==1) &
                             (stats_side['p_prew_pre_deconv'] > p_thr) ].shape[0]
        nomf_l = stats_side[ (stats_side['diff_oomf_l_' + time_period + '_' + trace_type] < 0) &
                             (stats_side['p_oomf_l_' + time_period + '_' + trace_type] < p_thr) &
                             (stats_side['usable']==1) &
                             (stats_side['p_prew_pre_deconv'] > p_thr)].shape[0]
        oomf_r = stats_side[ (stats_side['diff_oomf_r_' + time_period + '_' + trace_type] > 0) &
                             (stats_side['p_oomf_r_' + time_period + '_' + trace_type] < p_thr) & 
                             (stats_side['usable']==1) &
                             (stats_side['p_prew_pre_deconv'] > p_thr)].shape[0]
        nomf_r = stats_side[ (stats_side['diff_oomf_r_' + time_period + '_' + trace_type] < 0) &
                             (stats_side['p_oomf_r_' + time_period + '_' + trace_type] < p_thr) &
                             (stats_side['usable']==1) &
                             (stats_side['p_prew_pre_deconv'] > p_thr)].shape[0]
      #  print(f'{time_period} L: {neu_l}, R: {neu_r}')
        oomf_ls.append(oomf_l); oomf_rs.append(oomf_r)
        nomf_ls.append(nomf_l); nomf_rs.append(nomf_r)
    all_oomf_ls.append(oomf_ls); all_oomf_rs.append(oomf_rs); 
    all_nomf_ls.append(nomf_ls); all_nomf_rs.append(nomf_rs); 
    
  #  axs.plot(range(5), ls, 'b')
   # axs.plot(range(5), rs, 'r')
  #  axs.set_xticks(range(5), ['pre', 'tone', 'resp', 'resp2', 'anticip'])
plot_aver_sem_canon(all_oomf_ls, times, 'c', ghost_traces=True,
                       span = [0,4], sampling_rate = 1)   

plot_aver_sem_canon(all_oomf_rs, times, 'm', ghost_traces=True,
                       span = [0,4], sampling_rate = 1)   
plot_aver_sem_canon(all_nomf_ls, times, 'g', ghost_traces=True,
                       span = [0,4], sampling_rate = 1)   
plot_aver_sem_canon(all_nomf_rs, times, 'orange', ghost_traces=True,
                       span = [0,4], sampling_rate = 1)   
                     
axs[0].set_xticks(range(4), ['tone', 'resp', 'resp2', 'anticip'])
plt.show()

In [ ]:
stats_side

In [ ]:
# summary stats
p_thr = 0.025

# summary stats
p_thr = 0.01
tps = ['pre', 'tone', 'resp', 'resp2', 'anticip']

trace_type = 'deconv'
all_oomf_ls = []; all_oomf_rs = []
all_nomf_ls = []; all_nomf_rs = []
times = [range(4)]*len(mouse_ids)
fig, axs = plt.subplots(1, len(mouse_ids), figsize=(15, 2))
plot_n = 0
for id in mouse_ids:
    oomf_ls = []; oomf_rs = []
    nomf_ls = []; nomf_rs = []
    print(f"::: Mouse  {id} :::")
    stats_side = data[id]['stats']
    neu_n = stats_side.shape[0]
    for time_period in tps[1:]:
        oomf_l = stats_side[ (stats_side['diff_oomf_l_' + time_period + '_' + trace_type] > 0) &
                             (stats_side['p_oomf_l_' + time_period + '_' + trace_type] < p_thr) &
                             (stats_side['usable']==1) &
                             (stats_side['p_prew_pre_deconv'] > p_thr)  &
                             (stats_side['p_prew_tone_deconv'] > p_thr)].shape[0] / neu_n
        nomf_l = stats_side[ (stats_side['diff_oomf_l_' + time_period + '_' + trace_type] < 0) &
                             (stats_side['p_oomf_l_' + time_period + '_' + trace_type] < p_thr) &
                             (stats_side['usable']==1) &
                             (stats_side['p_prew_pre_deconv'] > p_thr) &
                             (stats_side['p_prew_tone_deconv'] > p_thr)].shape[0] / neu_n
        oomf_r = stats_side[ (stats_side['diff_oomf_r_' + time_period + '_' + trace_type] > 0) &
                             (stats_side['p_oomf_r_' + time_period + '_' + trace_type] < p_thr) & 
                             (stats_side['usable']==1) &
                             (stats_side['p_prew_pre_deconv'] > p_thr) &
                             (stats_side['p_prew_tone_deconv'] > p_thr)].shape[0] / neu_n
        nomf_r = stats_side[ (stats_side['diff_oomf_r_' + time_period + '_' + trace_type] < 0) &
                             (stats_side['p_oomf_r_' + time_period + '_' + trace_type] < p_thr) &
                             (stats_side['usable']==1) &
                             (stats_side['p_prew_pre_deconv'] > p_thr) &
                             (stats_side['p_prew_tone_deconv'] > p_thr)].shape[0] / neu_n
      #  print(f'{time_period} L: {neu_l}, R: {neu_r}')
        oomf_ls.append(oomf_l); oomf_rs.append(oomf_r)
        nomf_ls.append(nomf_l); nomf_rs.append(nomf_r)
    all_oomf_ls.append(oomf_ls); all_oomf_rs.append(oomf_rs); 
    all_nomf_ls.append(nomf_ls); all_nomf_rs.append(nomf_rs); 
    
    axs[plot_n].plot(range(4), oomf_ls, 'c')
    axs[plot_n].plot(range(4), oomf_rs, 'm')
    axs[plot_n].plot(range(4), nomf_ls, 'g')
    axs[plot_n].plot(range(4), nomf_rs, 'darkorange')
    axs[plot_n].set_xticks(range(4), ['tone', 'resp', 'resp2', 'anticip'])
    
    plot_n += 1
   # axs.plot(range(5), rs, 'r')
  #  axs.set_xticks(range(5), ['pre', 'tone', 'resp', 'resp2', 'anticip'])
plt.show()
fig, axs = plt.subplots(1, 2, figsize=(15, 3))
plot_aver_sem_canon(all_oomf_ls, times, 'c', ghost_traces=True,
                       span = [0,4], sampling_rate = 1)   

plot_aver_sem_canon(all_oomf_rs, times, 'm', ghost_traces=True,
                       span = [0,4], sampling_rate = 1)   
plot_aver_sem_canon(all_nomf_ls, times, 'g', ghost_traces=True,
                       span = [0,4], sampling_rate = 1)   
plot_aver_sem_canon(all_nomf_rs, times, 'orange', ghost_traces=True,
                       span = [0,4], sampling_rate = 1)   
axs[0].set_xticks(range(4), ['tone', 'resp', 'resp2', 'anticip'])
plt.show()

In [ ]:
# summary stats
p_thr = 0.025

# summary stats
p_thr = 0.02
tps = ['pre', 'tone', 'resp', 'resp2', 'anticip']

trace_type = 'ca'
all_oomf_ls = []; all_oomf_rs = []
all_nomf_ls = []; all_nomf_rs = []
times = [range(4)]*len(mouse_ids)
fig, axs = plt.subplots(1, len(mouse_ids), figsize=(15, 2))
plot_n = 0
for id in mouse_ids:
    oomf_ls = []; oomf_rs = []
    nomf_ls = []; nomf_rs = []
    print(f"::: Mouse  {id} :::")
    stats_side = data[id]['stats']
    neu_n = data_exp['ca'].shape[0]
    for time_period in tps[1:]:
        oomf_l = stats_side[ (stats_side['diff_oomf_l_' + time_period + '_' + trace_type] > 0) &
                             (stats_side['p_oomf_l_' + time_period + '_' + trace_type] < p_thr) &
        (stats_side['diff_oomf_l_' + time_period + '_' + trace_type] > stats_side['diff_oomf_l_pre_' + trace_type]) &
                             (stats_side['p_side_pre_deconv'] > p_thr) ].shape[0] / neu_n
        nomf_l = stats_side[ (stats_side['diff_oomf_l_' + time_period + '_' + trace_type] < 0) &
                             (stats_side['p_oomf_l_' + time_period + '_' + trace_type] < p_thr) &
        (stats_side['diff_oomf_l_' + time_period + '_' + trace_type] < stats_side['diff_oomf_l_pre_' + trace_type]) &
                             (stats_side['p_side_pre_deconv'] > p_thr)].shape[0] / neu_n
        oomf_r = stats_side[ (stats_side['diff_oomf_r_' + time_period + '_' + trace_type] > 0) &
                             (stats_side['p_oomf_r_' + time_period + '_' + trace_type] < p_thr) & 
        (stats_side['diff_oomf_r_' + time_period + '_' + trace_type] > stats_side['diff_oomf_r_pre_' + trace_type]) &
                             (stats_side['p_side_pre_deconv'] > p_thr)].shape[0] / neu_n
        nomf_r = stats_side[ (stats_side['diff_oomf_r_' + time_period + '_' + trace_type] < 0) &
                             (stats_side['p_oomf_r_' + time_period + '_' + trace_type] < p_thr) &
        (stats_side['diff_oomf_r_' + time_period + '_' + trace_type] < stats_side['diff_oomf_r_pre_' + trace_type]) &
                             (stats_side['p_side_pre_deconv'] > p_thr)].shape[0] / neu_n
      #  print(f'{time_period} L: {neu_l}, R: {neu_r}')
        oomf_ls.append(oomf_l); oomf_rs.append(oomf_r)
        nomf_ls.append(nomf_l); nomf_rs.append(nomf_r)
    all_oomf_ls.append(oomf_ls); all_oomf_rs.append(oomf_rs); 
    all_nomf_ls.append(nomf_ls); all_nomf_rs.append(nomf_rs); 
    
    axs[plot_n].plot(range(4), oomf_ls, 'c')
    axs[plot_n].plot(range(4), oomf_rs, 'm')
    axs[plot_n].plot(range(4), nomf_ls, 'g')
    axs[plot_n].plot(range(4), nomf_rs, 'darkorange')
    axs[plot_n].set_xticks(range(4), ['tone', 'resp', 'resp2', 'anticip'])
    
    plot_n += 1
   # axs.plot(range(5), rs, 'r')
  #  axs.set_xticks(range(5), ['pre', 'tone', 'resp', 'resp2', 'anticip'])
plt.show()
fig, axs = plt.subplots(1, 2, figsize=(15, 3))
plot_aver_sem_canon(all_oomf_ls, times, 'c', ghost_traces=True,
                       span = [0,4], sampling_rate = 1)   

plot_aver_sem_canon(all_oomf_rs, times, 'm', ghost_traces=True,
                       span = [0,4], sampling_rate = 1)   
plot_aver_sem_canon(all_nomf_ls, times, 'g', ghost_traces=True,
                       span = [0,4], sampling_rate = 1)   
plot_aver_sem_canon(all_nomf_rs, times, 'orange', ghost_traces=True,
                       span = [0,4], sampling_rate = 1)   
axs[0].set_xticks(range(4), ['tone', 'resp', 'resp2', 'anticip'])
plt.show()

### Plot the oomfs and nomfs

In [ ]:

# summary stats
p_thr = 0.01
tps = ['pre', 'tone', 'resp', 'resp2', 'anticip']
trace_type = 'deconv'
times = [range(4)]*len(mouse_ids)
plot_n = 0
for id in mouse_ids:
    oomf_l_neus = []; oomf_r_neus = []
    nomf_l_neus = []; nomf_r_neus = []
    print(f"::: Mouse  {id} :::")
    stats_side = data[id]['stats']
    frame_ts = data[id]['frame_ts']
    behav =  data[id]['behav']
    for time_period in tps[1:]:
        oomf_l_neu = stats_side[  (stats_side['diff_oomf_l_' + time_period + '_' + trace_type] > 0) &
                             (stats_side['p_oomf_l_' + time_period + '_' + trace_type] < p_thr) &
                             (stats_side['p_side_pre_' + trace_type] > p_thr) 
        ]['neu'].to_numpy().tolist()
        nomf_l_neu = stats_side[ (stats_side['diff_oomf_l_' + time_period + '_' + trace_type] < 0) &
                             (stats_side['p_oomf_l_' + time_period + '_' + trace_type] < p_thr) &
                             (stats_side['p_side_pre_' + trace_type] > p_thr)]['neu']
        oomf_r_neu = stats_side[ # (stats_side['diff_oomf_r_' + time_period + '_' + trace_type] > 0) &
                             (stats_side['p_oomf_r_' + time_period + '_' + trace_type] < p_thr) & 
                             (stats_side['p_side_pre_' + trace_type] > p_thr)]['neu'].to_numpy().tolist()
        nomf_r_neu = stats_side[ (stats_side['diff_oomf_r_' + time_period + '_' + trace_type] < 0) &
                             (stats_side['p_oomf_r_' + time_period + '_' + trace_type] < p_thr) &
                             (stats_side['p_side_pre_' + trace_type] > p_thr)]['neu']
        oomf_l_neus.extend(oomf_l_neu); oomf_r_neus.extend(oomf_r_neu)
        nomf_l_neus.extend(nomf_l_neu); nomf_r_neus.extend(nomf_r_neu)
    oomf_l_neus = list(set(oomf_l_neus)); oomf_r_neus = list(set(oomf_r_neus))
    nomf_l_neus = list(set(nomf_l_neus)); nomf_r_neus = list(set(nomf_r_neus))
    print(f"====================")
    print(f"::: OOMF NEURONS :::")
    for n in oomf_r_neus:
        if data[id]['usable_neurons'][n] == False:
            continue
       # trace = data[id]['smoothies'][n]
        trace = data[id]['deconv'][n]
        # prep the figure
        fig, axs = plt.subplots(1, 2, figsize=(15, 3))
        
        colors = {'same': {1:'b', -1:'r'},
                  'diff': {1:'c', -1:'m'} }
        for side in [1]:
            params = ['side', 'choice', 'prev_side', 'missed']
            ts_same = get_behav_ts(behav, params, [side, side, side, 0])
            ts_diff = get_behav_ts(behav, params, [side, side, -side, 0])

            segms_same, times_same = get_segments_canon(trace, ts_same, frame_ts)
            segms_diff, times_diff = get_segments_canon(trace, ts_diff, frame_ts)

            plot_aver_sem_canon(segms_same, times_same, colors['same'][side], ghost_traces=True)
            plot_aver_sem_canon(segms_diff, times_diff, colors['diff'][side], ghost_traces=True)
            '''

            ts_same = get_behav_ts(behav, params, [side, side, side, 0, side])
            ts_diff = get_behav_ts(behav, params, [side, side, -side, 0, side])

            segms_same, times_same = get_segments_canon(trace, ts_same, frame_ts)
            segms_diff, times_diff = get_segments_canon(trace, ts_diff, frame_ts)

            plot_aver_sem_canon(segms_same, times_same, colors['same'][side], ghost_traces=True, linestyle=':')
            plot_aver_sem_canon(segms_diff, times_diff, colors['diff'][side], ghost_traces=True, linestyle=':')
            '''

            for ax_n in [0,1]:
                axs[ax_n].axvline(x=0)
                axs[ax_n].axvline(x=955, linestyle=':')
                axs[ax_n].axvline(x=1150)

        plt.show()

        


In [ ]:

# summary stats
p_thr = 0.01
tps = ['pre', 'tone', 'resp', 'resp2', 'anticip']
trace_type = 'deconv'
times = [range(4)]*len(mouse_ids)
plot_n = 0
for id in mouse_ids:
    oomf_l_neus = []; oomf_r_neus = []
    nomf_l_neus = []; nomf_r_neus = []
    print(f"::: Mouse  {id} :::")
    stats_side = data[id]['stats']
    frame_ts = data[id]['frame_ts']
    behav =  data[id]['behav']
    for time_period in tps[1:]:
        oomf_l_neu = stats_side[ # (stats_side['diff_oomf_l_' + time_period + '_' + trace_type] > 0) &
                             (stats_side['p_oomf_l_' + time_period + '_' + trace_type] < p_thr) &
                             (stats_side['p_side_pre_' + trace_type] > p_thr) 
        ]['neu'].to_numpy().tolist()
        nomf_l_neu = stats_side[ (stats_side['diff_oomf_l_' + time_period + '_' + trace_type] < 0) &
                             (stats_side['p_oomf_l_' + time_period + '_' + trace_type] < p_thr) &
                             (stats_side['p_side_pre_' + trace_type] > p_thr)]['neu']
        oomf_r_neu = stats_side[ # (stats_side['diff_oomf_r_' + time_period + '_' + trace_type] > 0) &
                             (stats_side['p_oomf_r_' + time_period + '_' + trace_type] < p_thr) & 
                             (stats_side['p_side_pre_' + trace_type] > p_thr)]['neu'].to_numpy().tolist()
        nomf_r_neu = stats_side[ (stats_side['diff_oomf_r_' + time_period + '_' + trace_type] < 0) &
                             (stats_side['p_oomf_r_' + time_period + '_' + trace_type] < p_thr) &
                             (stats_side['p_side_pre_' + trace_type] > p_thr)]['neu']
        oomf_l_neus.extend(oomf_l_neu); oomf_r_neus.extend(oomf_r_neu)
        nomf_l_neus.extend(nomf_l_neu); nomf_r_neus.extend(nomf_r_neu)
    oomf_l_neus = list(set(oomf_l_neus)); oomf_r_neus = list(set(oomf_r_neus))
    nomf_l_neus = list(set(nomf_l_neus)); nomf_r_neus = list(set(nomf_r_neus))
    print(f"====================")
    print(f"::: OOMF NEURONS :::")
    for n in oomf_r_neus:
        if data[id]['usable_neurons'][n] == False:
            continue
        trace = data[id]['ca'][n]
        # prep the figure
        fig, axs = plt.subplots(1, 2, figsize=(15, 3))
        
        colors = {'same': {1:'b', -1:'r'},
                  'diff': {1:'c', -1:'m'} }
        for side in [-1]:
            params = ['side', 'choice', 'prev_side', 'missed', 'laser']
            ts_same = get_behav_ts(behav, params, [side, side, side, 0, 0])
            ts_diff = get_behav_ts(behav, params, [side, side, -side, 0, 0])

            segms_same, times_same = get_segments_canon(trace, ts_same, frame_ts)
            segms_diff, times_diff = get_segments_canon(trace, ts_diff, frame_ts)

            plot_aver_sem_canon(segms_same, times_same, colors['same'][side], ghost_traces=True)
            plot_aver_sem_canon(segms_diff, times_diff, colors['diff'][side], ghost_traces=True)

            for ax_n in [0,1]:
                axs[ax_n].axvline(x=0)
                axs[ax_n].axvline(x=955, linestyle=':')
                axs[ax_n].axvline(x=1150)

        plt.show()

        


## YEAH let's do SQL

# Population activity analysis
* Plan

# PCA

## Run PCA for all mice

In [ ]:
pca_n = 16
trace_type = 'ca'
# For each mouse
for id in mouse_ids:
    print(f"::: Mouse  {id} :::")
    data_exp = data[id]
    usable = [ data_exp['good_neurons'][n]==1 or data_exp['fair_neurons'][n]==1 for n in np.arange(len(data_exp['good_neurons']))]
    traces = data_exp[trace_type]
    traces = traces[np.array(usable)]
    # Reshape data for PCA (each row is a time point, each column is a neuron)
    traces = traces.T
    # Perform PCA on the reshaped data
    pca = PCA(n_components=pca_n)
    pca.fit(traces)

    # Get the variance explained by each component
    explained_variance = pca.explained_variance_ratio_

    # To see the cumulative explained variance:
    cumulative_explained_variance = explained_variance.cumsum()
    pca_result = pca.fit_transform(traces)
    print("Variance explained by each component:", explained_variance)
    print("Cumulative variance explained:", cumulative_explained_variance)
    
    # Create a DataFrame for easier handling
    columns = ['PC' + str(n+1) for n in np.arange(pca_n)]
    pca_df_time_points = pd.DataFrame(pca_result, columns=columns)
    data[id]['PCA'] = pca_df_time_points
    data[id]['PCA_explained'] = explained_variance

In [ ]:
n_components = 10

# For each mouse
for id in mouse_ids:
    print(f"::: Mouse  {id} :::")
    data_exp = data[id]
    traces = data_exp['ca']
    if id == 'os89':
        start_bad = 6108; end_bad = 7224
        traces = np.hstack((traces[:, :start_bad], traces[:, end_bad:]))
    usable_neurons = data_exp['usable_neurons']
    traces = [traces[n] for n in np.arange(traces.shape[0]) if usable_neurons[n] == 1]
    traces = np.array(traces)
    behav = data_exp['behav']  

    # Reshape data for PCA (each row is a time point, each column is a neuron)
    traces = traces.T
    
    # Perform PCA on the reshaped data
    pca = PCA(n_components=n_components)
    pca_result = pca.fit_transform(traces)


    
    if id == 'os89':
        # Create a new array with the original shape (15110, 6)
        new_data = np.zeros((15110, 6))
        new_data[:start_bad, :] = pca_result[:start_bad, :]
        new_data[end_bad:, :] = pca_result[start_bad:, :]
        pca_result = new_data
    
    # Create a DataFrame for easier handling
    columns = ['PC' + str(n+1) for n in np.arange(n_components)]
    pca_df_time_points = pd.DataFrame(pca_result, columns=columns)
    data[id]['PCA'] = pca_df_time_points

### Add explained variance

In [ ]:
n_components = 10

# For each mouse
for id in mouse_ids:
    print(f"::: Mouse  {id} :::")
    data_exp = data[id]
    traces = data_exp['ca']
    if id == 'os89':
        start_bad = 6108; end_bad = 7224
        traces = np.hstack((traces[:, :start_bad], traces[:, end_bad:]))
    usable_neurons = data_exp['usable_neurons']
    traces = [traces[n] for n in np.arange(traces.shape[0]) if usable_neurons[n] == 1]
    traces = np.array(traces)
    behav = data_exp['behav']  

    # Reshape data for PCA (each row is a time point, each column is a neuron)
    traces = traces.T
    
    # Perform PCA on the reshaped data
    pca = PCA(n_components=n_components)
    pca.fit(traces)

    # Get the variance explained by each component
    explained_variance = pca.explained_variance_ratio_

    # To see the cumulative explained variance:
    cumulative_explained_variance = explained_variance.cumsum()
    print("Variance explained by each component:", explained_variance)
    print("Cumulative variance explained:", cumulative_explained_variance)

    
    pca_result = pca.fit_transform(traces)


    
    if id == 'os89':
        # Create a new array with the original shape (15110, 6)
        new_data = np.zeros((15110, n_components))
        new_data[:start_bad, :] = pca_result[:start_bad, :]
        new_data[end_bad:, :] = pca_result[start_bad:, :]
        pca_result = new_data
    
    # Create a DataFrame for easier handling
    columns = ['PC' + str(n+1) for n in np.arange(n_components)]
    pca_df_time_points = pd.DataFrame(pca_result, columns=columns)
    data[id]['PCA'] = pca_df_time_points
    data[id]['PCA_explained'] = explained_variance

## Classsify the components

In [ ]:
cross_pcs = {'os85a': [], 'os85b': [], 'os86a': [], 'os86b': [], 
                     'os87': [], 'os88': [], 'os89': []}
from sklearn.linear_model import LogisticRegression
comps_prev = {}
comps_resp = {}
for id in mouse_ids:
    print(f"::: Mouse  {id} :::")
    data_exp = data[id]
    behav = data_exp['behav']
    frame_ts = data_exp['frame_ts']
    comps = data[id]['ICA']
    comp_cols = comps.keys()
    params = [ 'side', 'choice', 'missed', 'free']
    params_prev = ['prev_reward']
    t1 = 1200; t2 = 3000
    t_pr1 =  -1000; t_pr2 = 0
    all_aucs_prev = []
    all_aucs_resp = []
    behav_l = get_behav_ts(behav, params, [1, 1, 0, 0])
    behav_r = get_behav_ts(behav, params, [-1, -1, 0, 0])
    behav_l_prev = get_behav_ts(behav, params_prev, [1])
    behav_r_prev = get_behav_ts(behav, params_prev, [-1])
    for comp_n in comp_cols:
        compon = comps[comp_n]
        get_behav_ts(behav, [], [])
        auc_l = get_auc(behav_l, compon, frame_ts, t1,  t2)
        auc_r = get_auc(behav_r, compon, frame_ts, t1,  t2)
        all_aucs_resp.append(auc_l+auc_r)
        auc_l_pr = get_auc(behav_l_prev, compon, frame_ts, t_pr1,  t_pr2)
        auc_r_pr = get_auc(behav_r_prev, compon, frame_ts, t_pr1,  t_pr2)
        all_aucs_prev.append(auc_l_pr+auc_r_pr)
    labels_resp = [1]*len(auc_l) + [-1]*len(auc_r)
    labels_prev = [1]*len(auc_l_pr) + [-1]*len(auc_r_pr)
    
    # For response
    y = np.array(labels_resp)
    all_aucs_resp = np.array(all_aucs_resp)
    X = all_aucs_resp.T
    # Initialize LogisticRegression
    log_reg = LogisticRegression()
    # Fit the model
    log_reg.fit(X, y)
    # Get the coefficients (weights) assigned to each component
    coefficients = log_reg.coef_[0]
    # Sort components by their absolute coefficient value to find the most important ones
    sorted_indices = np.argsort(np.abs(coefficients))[::-1]
    sorted_components = coefficients[sorted_indices]
    
    print("Most important components in separating the RESPONSE:", sorted_indices+1)
    cross_pcs[id].append(sorted_indices[0]+1)
    print("Corresponding coefficients:", sorted_components)
    comps_resp[id] = sorted_indices+1

    # for prev 
    y = np.array(labels_prev)
    all_aucs_prev = np.array(all_aucs_prev)
    X = all_aucs_prev.T
    # Initialize LogisticRegression
    log_reg = LogisticRegression()
    # Fit the model
    log_reg.fit(X, y)
    # Get the coefficients (weights) assigned to each component
    coefficients = log_reg.coef_[0]
    # Sort components by their absolute coefficient value to find the most important ones
    sorted_indices = np.argsort(np.abs(coefficients))[::-1]
    sorted_components = coefficients[sorted_indices]
    
    print("Most important components in separating the PREV:", sorted_indices+1)
    if sorted_indices[0]+1!=cross_pcs[id][0]:
        cross_pcs[id].append(sorted_indices[0]+1)
    else:
        cross_pcs[id].append(sorted_indices[1]+1)
    print("Corresponding coefficients:", sorted_components)
    comps_prev[id] = sorted_indices+1


cross_pcs


## Plot components

### Side and choice across components

In [ ]:
pca_cols = columns
# For each mouse
for id in mouse_ids:
    print(f"::: Mouse  {id} :::")
    data_exp = data[id]
    pcas = data[id]['PCA']
    frame_ts = data_exp['frame_ts']
    behav = data_exp['behav']

    # go through the 1st n components
    for pca_comp in pca_cols:
        n=0
        pc = pcas[pca_comp]
        params = ['side', 'choice', 'missed', 'laser']
        ts_l = get_behav_ts(behav, params, [1, 1, 0, 0])
        ts_r = get_behav_ts(behav, params, [-1, -1, 0, 0])
        segms_l, times_l = get_segments_canon(pc, ts_l, frame_ts)
        segms_r, times_r = get_segments_canon(pc, ts_r, frame_ts)
        # prep figure
        fig, axs = plt.subplots(1, 2, figsize=(15, 3))
        plot_aver_sem_canon(segms_l, times_l, 'b', ghost_traces=True)
        plot_aver_sem_canon(segms_r, times_r, 'r', ghost_traces=True)
    
        for ax_n in [0,1]:
            axs[ax_n].axvline(x=-200)
            axs[ax_n].axvline(x=950)
    
        plt.title(pca_comp)
        plt.xlabel('time')
        plt.ylabel('au')
        plt.show()

## Prev choice

In [ ]:
pca_cols = columns
# For each mouse
for id in mouse_ids:
    print(f"::: Mouse  {id} :::")
    data_exp = data[id]
    pcas = data[id]['PCA']
    frame_ts = data_exp['frame_ts']
    behav = data_exp['behav']

    # go through the 1st n components
    for pca_comp in pca_cols:
        n=0
        pc = pcas[pca_comp]
        params = ['prev_reward']
        ts_l = get_behav_ts(behav, params, [1, 1, 0, 0])
        ts_r = get_behav_ts(behav, params, [-1, -1, 0, 0])
        segms_l, times_l = get_segments_canon(pc, ts_l, frame_ts)
        segms_r, times_r = get_segments_canon(pc, ts_r, frame_ts)
        # prep figure
        fig, axs = plt.subplots(1, 2, figsize=(15, 3))
        plot_aver_sem_canon(segms_l, times_l, 'b', ghost_traces=True)
        plot_aver_sem_canon(segms_r, times_r, 'r', ghost_traces=True)
    
        for ax_n in [0,1]:
            axs[ax_n].axvline(x=-200)
            axs[ax_n].axvline(x=950)
    
        plt.title(pca_comp)
        plt.xlabel('time')
        plt.ylabel('au')
        plt.show()

### Laser

In [ ]:
pca_cols = columns
# For each mouse
for id in mouse_ids:
    print(f"::: Mouse  {id} :::")
    data_exp = data[id]
    pcas = data[id]['PCA']
    frame_ts = data_exp['frame_ts']
    behav = data_exp['behav']

    # go through the 1st n components
    for pca_comp in pca_cols:
        n=0
        pc = pcas[pca_comp]
        params = ['side', 'choice', 'missed', 'laser']
        ts_l = get_behav_ts(behav, params, [1, 1, 0, 0])
        ts_r = get_behav_ts(behav, params, [-1, -1, 0, 0])
        segms_l, times_l = get_segments_canon(pc, ts_l, frame_ts)
        segms_r, times_r = get_segments_canon(pc, ts_r, frame_ts)
        # prep figure
        fig, axs = plt.subplots(1, 2, figsize=(15, 3))
        plot_aver_sem_canon(segms_l, times_l, 'b', ghost_traces=True)
        plot_aver_sem_canon(segms_r, times_r, 'r', ghost_traces=True)
        
        # LASER 
        ts_l = get_behav_ts(behav, params, [1, 1, 0, 1])
        ts_r = get_behav_ts(behav, params, [-1, -1, 0, -1])
        segms_l, times_l = get_segments_canon(pc, ts_l, frame_ts)
        segms_r, times_r = get_segments_canon(pc, ts_r, frame_ts)
        # add to figure
        plot_aver_sem_canon(segms_l, times_l, 'b', linestyle=':', ghost_traces=True)
        plot_aver_sem_canon(segms_r, times_r, 'r', linestyle=':', ghost_traces=True)
    
        for ax_n in [0,1]:
            axs[ax_n].axvline(x=-200)
            axs[ax_n].axvline(x=950)
    
        plt.title(pca_comp)
        plt.xlabel('time')
        plt.ylabel('au')
        plt.show()

### OOMF

In [ ]:
pca_cols = columns
cross_pcs = {'os85a': [1,2], 'os85b': [1,3], 'os86a': [2,5], 'os86b': [1,2], 
                     'os87': [1,2], 'os88': [1,3], 'os89': [1,2]}

# For each mouse
for id in mouse_ids:
    print(f"::: Mouse  {id} :::")
    data_exp = data[id]
    pcas = data[id]['PCA']
    frame_ts = data_exp['frame_ts']
    behav = data_exp['behav']
    pca_ns = cross_pcs[id]
    pca_cols = ['PC' + str(n) for n in pca_ns]
    pca_cols = columns

    # go through the 1st n components
    for pca_comp in pca_cols:
        n=0
        pc = pcas[pca_comp]
        params = ['prev_side', 'side', 'choice', 'missed', 'laser']
        # Same
        ts_l = get_behav_ts(behav, params, [1, 1, 1, 0, 0])
        ts_r = get_behav_ts(behav, params, [-1,-1, -1, 0, 0])      
        segms_l, times_l = get_segments_canon(pc, ts_l, frame_ts)
        segms_r, times_r = get_segments_canon(pc, ts_r, frame_ts)

        # prep figure
        fig, axs = plt.subplots(1, 2, figsize=(15, 3))
        plot_aver_sem_canon(segms_l, times_l, 'b', ghost_traces=True)
        plot_aver_sem_canon(segms_r, times_r, 'r', ghost_traces=True)
        
        # DIFFERENT 
        ts_l = get_behav_ts(behav, params, [-1, 1, 1, 0, 0])
        ts_r = get_behav_ts(behav, params, [1,-1, -1, 0, 0])
        segms_l, times_l = get_segments_canon(pc, ts_l, frame_ts)
        segms_r, times_r = get_segments_canon(pc, ts_r, frame_ts)
        # add to figure
        plot_aver_sem_canon(segms_l, times_l, 'c', ghost_traces=True)
        plot_aver_sem_canon(segms_r, times_r, 'm', ghost_traces=True)
    
        for ax_n in [0,1]:
            axs[ax_n].axvline(x=-200)
            axs[ax_n].axvline(x=950)
        axs[0].set_title(id)
        plt.title(pca_comp)
        plt.xlabel('time')
        plt.ylabel('au')
        plt.show()

In [ ]:

# left same vs different

    
pc = pca_df_time_points['PC3']
n=99
params = ['side', 'laser']
traces_l1 = get_segments_old(data_exp, params, [1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
traces_r1 = get_segments_old(data_exp, params, [1,1], n, frame_type='frame_tone', is_pca=True, pc=pc)
pc = pca_df_time_points['PC4']
traces_l2 = get_segments_old(data_exp, params, [1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
traces_r2 = get_segments_old(data_exp, params, [1,1], n, frame_type='frame_tone', is_pca=True, pc=pc)

t_windows = [ [0,5], [6,10], [11,15], [16,20]]
# setup the plot
fig, axs = plt.subplots(1, 4, figsize=(15, 3))
ax_n = 0


for w in t_windows:
    # plot all the left 
    for segment_n in np.arange(len(traces_l1)):
        segment_pc1 = traces_l1[segment_n]
        segment_pc2 = traces_l2[segment_n]
        av_pc1 = np.mean(segment_pc1[w[0]:w[1]])
        av_pc2 = np.mean(segment_pc2[w[0]:w[1]])
        axs[ax_n].plot(av_pc1,av_pc2, 'b*')
        
    # plot all the right
    for segment_n in np.arange(len(traces_r1)):
        segment_pc1 = traces_r1[segment_n]
        segment_pc2 = traces_r2[segment_n]
        av_pc1 = np.mean(segment_pc1[w[0]:w[1]])
        av_pc2 = np.mean(segment_pc2[w[0]:w[1]])
        axs[ax_n].plot(av_pc1,av_pc2, 'c*')
 #   axs[ax_n].set_title(f't window {str(w[0]*100)} to {str(w[1]*100)} ms')
   #axs[ax_n].xlabel('PC1')
  #  axs[ax_n].ylabel('PC2')
    
    ax_n = ax_n+1
    
plt.show()


In [ ]:
# go through the 1st n components
for pca_comp in ['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6']:
    n=0
    pc = pca_df_time_points[pca_comp]
    traces_l = get_segments_old(data_exp, ['side', 'laser'], [1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
    traces_r = get_segments_old(data_exp, ['side', 'laser'],[-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
    
    fig, axs = plt.subplots(1, 2, figsize=(15, 3))
    # side  
    plot_aver_se_old(traces_l, 'b',ghost_traces=False, axn=0)
    plot_aver_se_old(traces_r, 'r',ghost_traces=False, axn=0)

    # choice
    traces_l = get_segments_old(data_exp, ['choice', 'laser'], [1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
    traces_r = get_segments_old(data_exp, ['choice', 'laser'], [-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
    traces_0 = get_segments_old(data_exp, ['choice', 'laser'], [0,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
    plot_aver_se_old(traces_l, 'b',ghost_traces=False, axn=1)
    plot_aver_se_old(traces_r, 'r',ghost_traces=False, axn=1)
    plot_aver_se_old(traces_0, 'k',ghost_traces=False, axn=1)

    for ax_n in [0,1]:
        axs[ax_n].axvline(x=-200)
        axs[ax_n].axvline(x=950)

    plt.title(pca_comp)
    plt.xlabel('time')
    plt.ylabel('au')
    plt.show()

## Distances with PCA

In [ ]:

# Distances for 2 selected components
# For each mouse
for id in mouse_ids:
    dates = data[id].keys()
    # and date
    for date in dates:
        print(f"::: Mouse  {id}, {date} :::")
        data_exp = data[id][date]
        traces = data_exp['ca']
        behav = data_exp['behav']
        pca_df_time_points = data[id][date]['PCA']
    
        pc = pca_df_time_points['PC2']
        n=99
        params = ['side', 'prev_reward', 'laser']
        span=[20,40]
        traces_l_same_1 = get_segments_old(data_exp, params, [1,1,0], n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
        traces_r_same_1 = get_segments_old(data_exp, params,[-1,-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
        traces_l_diff_1 = get_segments_old(data_exp, params, [1,-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
        traces_r_diff_1 = get_segments_old(data_exp, params,[-1,1,0], n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
        pc = pca_df_time_points['PC3']
        traces_l_same_2 = get_segments_old(data_exp, params, [1,1,0], n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
        traces_r_same_2 = get_segments_old(data_exp, params,[-1,-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
        traces_l_diff_2 = get_segments_old(data_exp, params, [1,-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
        traces_r_diff_2 = get_segments_old(data_exp, params,[-1,1,0], n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
        
        # t_windows = [ [0,5], [6,10], [11,15], [16,20]]
        window_width = 10
        t_windows = [ ]
        for t in np.arange(0,55,window_width):
            t_windows.append([t,t+window_width-1])
        # setup the plot
        fig, axs = plt.subplots(1, len(t_windows), figsize=(15, 2))
        ax_n = 0
        
        for w in t_windows:
            # plot all the left same
            segms_l_same_1 = []
            segms_l_same_2 = []
            for segment_n in np.arange(len(traces_l_same_1)):
                segment_pc1 = traces_l_same_1[segment_n]
                segment_pc2 = traces_l_same_2[segment_n]
                av_pc1 = np.mean(segment_pc1[w[0]:w[1]])
                av_pc2 = np.mean(segment_pc2[w[0]:w[1]])
                segms_l_same_1.append(av_pc1)
                segms_l_same_2.append(av_pc2)
                axs[ax_n].plot(av_pc1, av_pc2, 'b*', markersize=3, alpha=0.2)
        
            # plot all the left diff
            segms_l_diff_1 = []
            segms_l_diff_2 = []
            for segment_n in np.arange(len(traces_l_diff_1)):
                segment_pc1 = traces_l_diff_1[segment_n]
                segment_pc2 = traces_l_diff_2[segment_n]
                av_pc1 = np.mean(segment_pc1[w[0]:w[1]])
                av_pc2 = np.mean(segment_pc2[w[0]:w[1]])
                segms_l_diff_1.append(av_pc1)
                segms_l_diff_2.append(av_pc2)
                axs[ax_n].plot(av_pc1, av_pc2, 'c*', markersize=3, alpha=0.2)
        
            # plot all the right same
            segms_r_same_1 = []
            segms_r_same_2 = []
            for segment_n in np.arange(len(traces_r_same_1)):
                segment_pc1 = traces_r_same_1[segment_n]
                segment_pc2 = traces_r_same_2[segment_n]
                av_pc1 = np.mean(segment_pc1[w[0]:w[1]])
                av_pc2 = np.mean(segment_pc2[w[0]:w[1]])
                segms_r_same_1.append(av_pc1)
                segms_r_same_2.append(av_pc2)
                axs[ax_n].plot(av_pc1, av_pc2, 'r*', markersize=3, alpha=0.2)
        
            # plot all the right diff
            segms_r_diff_1 = []
            segms_r_diff_2 = []
            for segment_n in np.arange(len(traces_r_diff_1)):
                segment_pc1 = traces_r_diff_1[segment_n]
                segment_pc2 = traces_r_diff_2[segment_n]
                av_pc1 = np.mean(segment_pc1[w[0]:w[1]])
                av_pc2 = np.mean(segment_pc2[w[0]:w[1]])
                segms_r_diff_1.append(av_pc1)
                segms_r_diff_2.append(av_pc2)
                axs[ax_n].plot(av_pc1, av_pc2, 'm*', markersize=3, alpha=0.2)
        
            av_segms_l_same_1 = np.mean(segms_l_same_1)
            av_segms_l_same_2 = np.mean(segms_l_same_2)
            av_segms_l_diff_1 = np.mean(segms_l_diff_1)
            av_segms_l_diff_2 = np.mean(segms_l_diff_2)
            av_segms_r_same_1 = np.mean(segms_r_same_1)
            av_segms_r_same_2 = np.mean(segms_r_same_2)
            av_segms_r_diff_1 = np.mean(segms_r_diff_1)
            av_segms_r_diff_2 = np.mean(segms_r_diff_2)
        
            axs[ax_n].plot(av_segms_l_same_1, av_segms_l_same_2, color='b', marker='o', markersize=12, markeredgewidth=2, fillstyle='none')
            axs[ax_n].plot(av_segms_l_diff_1, av_segms_l_diff_2, color='c', marker='o', markersize=12, markeredgewidth=2, fillstyle='none')
            axs[ax_n].plot(av_segms_r_same_1, av_segms_r_same_2, color='r', marker='o', markersize=12, markeredgewidth=2, fillstyle='none')
            axs[ax_n].plot(av_segms_r_diff_1, av_segms_r_diff_2, color='m', marker='o', markersize=12, markeredgewidth=2, fillstyle='none')
        
            ax_n += 1
        
        plt.show()


In [ ]:

# Distances for 2 selected components
# For each mouse
for id in mouse_ids:
    dates = data[id].keys()
    # and date
    for date in dates:
        print(f"::: Mouse  {id}, {date} :::")
        data_exp = data[id][date]
        traces = data_exp['ca']
        behav = data_exp['behav']
        pca_df_time_points = data[id][date]['PCA']
    
        pc = pca_df_time_points['PC2']
        n=99
        params = ['side', 'prev_reward', 'laser']
        span=[10,40]
        traces_l_same_1 = get_segments_old(data_exp, params, [1,1,1], n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
        traces_r_same_1 = get_segments_old(data_exp, params,[-1,-1,-1], n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
        traces_l_diff_1 = get_segments_old(data_exp, params, [1,-1,1], n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
        traces_r_diff_1 = get_segments_old(data_exp, params,[-1,1,-1], n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
        pc = pca_df_time_points['PC3']
        traces_l_same_2 = get_segments_old(data_exp, params, [1,1,1], n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
        traces_r_same_2 = get_segments_old(data_exp, params,[-1,-1,-1], n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
        traces_l_diff_2 = get_segments_old(data_exp, params, [1,-1,1], n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
        traces_r_diff_2 = get_segments_old(data_exp, params,[-1,1,-1], n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
        
        # t_windows = [ [0,5], [6,10], [11,15], [16,20]]
        window_width = 10
        t_windows = [ ]
        for t in np.arange(0,45,window_width):
            t_windows.append([t,t+window_width-1])
        # setup the plot
        fig, axs = plt.subplots(1, len(t_windows), figsize=(15, 3))
        ax_n = 0
        
        for w in t_windows:
            # plot all the left same
            segms_l_same_1 = []
            segms_l_same_2 = []
            for segment_n in np.arange(len(traces_l_same_1)):
                segment_pc1 = traces_l_same_1[segment_n]
                segment_pc2 = traces_l_same_2[segment_n]
                av_pc1 = np.mean(segment_pc1[w[0]:w[1]])
                av_pc2 = np.mean(segment_pc2[w[0]:w[1]])
                segms_l_same_1.append(av_pc1)
                segms_l_same_2.append(av_pc2)
                axs[ax_n].plot(av_pc1, av_pc2, 'b*', markersize=3, alpha=0.2)
        
            # plot all the left diff
            segms_l_diff_1 = []
            segms_l_diff_2 = []
            for segment_n in np.arange(len(traces_l_diff_1)):
                segment_pc1 = traces_l_diff_1[segment_n]
                segment_pc2 = traces_l_diff_2[segment_n]
                av_pc1 = np.mean(segment_pc1[w[0]:w[1]])
                av_pc2 = np.mean(segment_pc2[w[0]:w[1]])
                segms_l_diff_1.append(av_pc1)
                segms_l_diff_2.append(av_pc2)
                axs[ax_n].plot(av_pc1, av_pc2, 'c*', markersize=3, alpha=0.2)
        
            # plot all the right same
            segms_r_same_1 = []
            segms_r_same_2 = []
            for segment_n in np.arange(len(traces_r_same_1)):
                segment_pc1 = traces_r_same_1[segment_n]
                segment_pc2 = traces_r_same_2[segment_n]
                av_pc1 = np.mean(segment_pc1[w[0]:w[1]])
                av_pc2 = np.mean(segment_pc2[w[0]:w[1]])
                segms_r_same_1.append(av_pc1)
                segms_r_same_2.append(av_pc2)
                axs[ax_n].plot(av_pc1, av_pc2, 'r*', markersize=3, alpha=0.2)
        
            # plot all the right diff
            segms_r_diff_1 = []
            segms_r_diff_2 = []
            for segment_n in np.arange(len(traces_r_diff_1)):
                segment_pc1 = traces_r_diff_1[segment_n]
                segment_pc2 = traces_r_diff_2[segment_n]
                av_pc1 = np.mean(segment_pc1[w[0]:w[1]])
                av_pc2 = np.mean(segment_pc2[w[0]:w[1]])
                segms_r_diff_1.append(av_pc1)
                segms_r_diff_2.append(av_pc2)
                axs[ax_n].plot(av_pc1, av_pc2, 'm*', markersize=3, alpha=0.2)
        
            av_segms_l_same_1 = np.mean(segms_l_same_1)
            av_segms_l_same_2 = np.mean(segms_l_same_2)
            av_segms_l_diff_1 = np.mean(segms_l_diff_1)
            av_segms_l_diff_2 = np.mean(segms_l_diff_2)
            av_segms_r_same_1 = np.mean(segms_r_same_1)
            av_segms_r_same_2 = np.mean(segms_r_same_2)
            av_segms_r_diff_1 = np.mean(segms_r_diff_1)
            av_segms_r_diff_2 = np.mean(segms_r_diff_2)
        
            axs[ax_n].plot(av_segms_l_same_1, av_segms_l_same_2, color='b', marker='o', markersize=12, markeredgewidth=2, fillstyle='none')
            axs[ax_n].plot(av_segms_l_diff_1, av_segms_l_diff_2, color='c', marker='o', markersize=12, markeredgewidth=2, fillstyle='none')
            axs[ax_n].plot(av_segms_r_same_1, av_segms_r_same_2, color='r', marker='o', markersize=12, markeredgewidth=2, fillstyle='none')
            axs[ax_n].plot(av_segms_r_diff_1, av_segms_r_diff_2, color='m', marker='o', markersize=12, markeredgewidth=2, fillstyle='none')
            axs[ax_n].set_ylabel('PC2')
            axs[ax_n].set_xlabel('PC1')
        
            ax_n += 1
        plt.tight_layout()
        plt.show()


## trajectories

In [ ]:
mouse_ids

In [ ]:
cross_pcs = {'os85a': [1,2], 'os85b': [1,4], 'os86a': [2,5], 'os86b': [1,2], 
                     'os87': [1,2], 'os88': [1,4], 'os89': [1,2]}
# Trajectories for 2 selected components
# For each mouse
condition_pairs = [ ['left_side_same', 'left_side_diff'],
                   ['right_side_same', 'right_side_diff'] ]
sides_code = {'l': 1, 'r': -1, 1: 'l', -1: 'r', 'same':1, 'diff':-1}
#mice_ids = ['os85a',  'os86a',  'os87', 'os88', 'os89']
pc_sets = 
for id in mouse_ids[1:2]:
    print(f"::: Mouse  {id} :::")
    data_exp = data[id]
    behav = data_exp['behav']
    pcas = data[id]['PCA']
    frame_ts = data_exp['frame_ts']
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    ax_n = 0
    comp_ns = cross_pcs[id]
    pc1 = pcas['PC' + str(comp_ns[0])]; pc2 = pcas['PC' + str(comp_ns[1])]
    params = ['prev_side', 'side', 'choice', 'missed', 'laser']
    span = [0, 4000]
    mouse_ts = {'same': {}, 'diff': {}}
    mouse_max_vals = {'same': {}, 'diff': {}}
    for sameness in ['same', 'diff']:
        for side in ['l', 'r']:
            side_val = sides_code[side]
            modif = sides_sode[sameness]
            peak_ts = []; max_vals = []
            # same left 
            segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [modif*side_val,side_val,side_val,0,0], span=span)
            segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [modif*side_val,side_val,side_val,0,0], span=span)
            # each segment has to start from 0 and end at the peak
            for s_n in np.arange(len(segms1)):
                segm1 = segms1[s_n]; segm2 = segms2[s_n]
                segm1 = segm1.tolist(); segm2 = segm2.tolist() 
                # Calculate the distance from the start (first point)
                distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)
                # Find the index of the maximum distance
                max_distance_idx = np.argmax(distances)
                # The time when the line is furthest away   
                peak_t = max_distance_idx; # abs_segm.index(max(abs_segm))
                
                #segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
                axs[ax_n].plot(segm1, segm2, 'b', alpha=0.5)
               # axs[ax_n].plot(segm1[0], segm2[0], 'b', marker='o', alpha=0.5)
                axs[ax_n].plot(segm1[-1], segm2[-1], 'b', marker='D', alpha=0.5)
                # Assuming segm1 and segm2 are lists or arrays of coordinates [x1, y1] and [x2, y2]
                segm1 = np.array(segm1); segm2 = np.array(segm2)
                # Calculate the Euclidean distance
                distance = np.linalg.norm(segm2 - segm1); # dist_same.append(distance)
                # our values
                peak_ts.append(peak_t); max_vals.append(max(distances))
            mouse_ts[sameness][side] = peak_ts
            mouse_max_vals[sameness][side] = max_vals
        ax_n+=1
    data[id]['peak_ts'] = mouse_ts
    data[id]['max_vals'] = mouse_max_vals



    all_same_ts = mouse_ts['same']['l'] + mouse_ts['same']['r']
    all_diff_ts = mouse_ts['diff']['l'] + mouse_ts['diff']['r']
    all_same_maxs = mouse_max_vals['same']['l'] + mouse_max_vals['same']['r']
    all_diff_maxs = mouse_max_vals['diff']['l'] + mouse_max_vals['diff']['r']
    print('ts same: ', round(np.mean(all_same_ts),2), 
          'diff: ', round(np.mean(all_diff_ts),2)) 
    stat, p_val = stats.ttest_ind(all_same_ts, all_diff_ts)    
    print('p for time to peak', p_val)
    print('max vals same: ', round(np.mean(all_same_maxs),2), 
          'diff: ', round(np.mean(all_diff_maxs),2)) 
    stat, p_val = stats.ttest_ind(all_same_maxs, all_diff_maxs)    
    print('p for time to peak', p_val)
    '''
    print(np.mean(dist_same), np.mean(dist_diff)) 
    stat, p_val = stats.ttest_ind(dist_same, dist_diff)    
    print('p for distances', p_val)
    '''


    
    plt.show()
    

### Peak value

In [ ]:
cross_pcs = {'os85a': [5,3], 'os85b': [1,3], 'os86a': [2,5], 'os86b': [1,2], 
                     'os87': [1,2], 'os88': [1,4], 'os89': [1,2]}
comps_resp
all_max_same = []
all_max_diff = []
for id in mouse_ids:
    mouse_max_same = []
    mouse_max_diff = []
    print(f"::: Mouse  {id} :::")
    data_exp = data[id]
    behav = data_exp['behav']
    pcas = data[id]['PCA']
    frame_ts = data_exp['frame_ts']
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    ax_n = 0
    comp_resp_mouse = comps_resp[id]
    comp_prev_mouse = comps_prev[id]
    c1 = comp_resp_mouse[0]
    if c1 == comp_prev_mouse[0]:
        c2 = comp_prev_mouse[1]
    else:
        c2 =  comp_prev_mouse[0]
        
    
    pc1 = pcas['PC' + str(c1)]; pc2 = pcas['PC' + str(c2)]
    params = ['prev_side', 'side', 'missed', 'laser', 'choice']
    span = [0, 3000]
    # ==== LEFT ====
    peak_vals_same = []; dist_same = []
    # same left 
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [1,1,0,0,1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [1,1,0,0,1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
       # segm1 = [s-segm1[0] for s in segm1]
      #  segm2 = [s-segm2[0] for s in segm2]

        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
     #   time_furthest_away = time[max_distance_idx]     
        peak_t = max_distance_idx; # abs_segm.index(max(abs_segm))
        peak_vals_same.append(max(distances))
        #segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'b', alpha=0.5)
       # axs[ax_n].plot(segm1[0], segm2[0], 'b', marker='o', alpha=0.5)
        axs[ax_n].plot(segm1[-1], segm2[-1], 'b', marker='D', alpha=0.5)
        # Assuming segm1 and segm2 are lists or arrays of coordinates [x1, y1] and [x2, y2]
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_same.append(distance)

    # same left 
    peak_vals_diff = []; dist_diff = []
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [-1,1,0,0,1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [-1,1,0,0,1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
      #  segm1 = [s-segm1[0] for s in segm1]
      #  segm2 = [s-segm2[0] for s in segm2]
        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
      #  time_furthest_away = time[max_distance_idx]

        
        peak_t = max_distance_idx #abs_segm.index(max(abs_segm))
        peak_vals_diff.append(max(distances))
        segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'c', alpha=0.5)
        #axs[ax_n].plot(segm1[0], segm2[0], 'c', marker='o', alpha=0.5)
        axs[ax_n].plot(segm1[-1], segm2[-1], 'c', marker='D', alpha=0.5)
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_diff.append(distance)
    
    print(np.mean(peak_vals_same), np.mean(peak_vals_diff)) 
    stat, p_val = stats.ttest_ind(peak_vals_same, peak_vals_diff)    
    print('p for max val', p_val)
    all_max_same += peak_vals_same
    all_max_diff += peak_vals_diff
    mouse_max_same += peak_vals_same
    mouse_max_diff += peak_vals_diff
    '''
    print(np.mean(dist_same), np.mean(dist_diff)) 
    stat, p_val = stats.ttest_ind(dist_same, dist_diff)    
    print('p for distances', p_val)
    '''
    # ==== RIGHT =====
    ax_n += 1
    peak_vals_same = []; dist_same = []
    # same left 
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [-1,-1,0,0,-1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [-1,-1,0,0,-1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
      #  segm1 = [s-segm1[-1] for s in segm1]
      #  segm2 = [s-segm2[-1] for s in segm2]

        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
     #   time_furthest_away = time[max_distance_idx]


        
        peak_t = max_distance_idx; # abs_segm.index(max(abs_segm))
        peak_vals_same.append(max(distances))
        segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'r', alpha=0.5)
        axs[ax_n].plot(segm1[0], segm2[0], 'r', marker='o', alpha=0.5)
       # axs[ax_n].plot(segm1[-1], segm2[-1], 'r', marker='D', alpha=0.5)
        # Assuming segm1 and segm2 are lists or arrays of coordinates [x1, y1] and [x2, y2]
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_same.append(distance)

    # same left 
    #ax_n += 1
    peak_vals_diff = []; dist_diff = []
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [1,-1,0,0,-1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [1,-1,0,0,-1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
     #   segm1 = [s-segm1[-1] for s in segm1]
      #  segm2 = [s-segm2[-1] for s in segm2]
        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
      #  time_furthest_away = time[max_distance_idx]

        
        peak_t = max_distance_idx #abs_segm.index(max(abs_segm))
        peak_vals_diff.append(max(distances))
        segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'm', alpha=0.5)
        axs[ax_n].plot(segm1[0], segm2[0], 'm', marker='o', alpha=0.5)
       # axs[ax_n].plot(segm1[-1], segm2[-1], 'm', marker='D', alpha=0.5)
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_diff.append(distance)
    
    print(np.mean(peak_vals_same), np.mean(peak_vals_diff)) 
    stat, p_val = stats.ttest_ind(peak_vals_same, peak_vals_diff)    
    print('p for max val', p_val)
    all_max_same += peak_vals_same
    all_max_diff += peak_vals_diff
    mouse_max_same += peak_vals_same
    mouse_max_diff += peak_vals_diff
    data[id]['max_comp'] = {'same': mouse_max_same, 'diff': mouse_max_diff}
    '''
    print(np.mean(dist_same), np.mean(dist_diff)) 
    stat, p_val = stats.ttest_ind(dist_same, dist_diff)    
    print('p for distances', p_val)
    '''


    
    plt.show()
    

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

for id in mouse_ids:
    peak_ts_same = data[id]['max_comp']['same']
    peak_ts_diff = data[id]['max_comp']['diff']
    
    # Combine data for boxplot
    to_plot = [peak_ts_same, peak_ts_diff]
    # Create boxplot
    from scipy.stats import mannwhitneyu
    
    u_stat, p_value = mannwhitneyu(peak_ts_same, peak_ts_diff)
    print(f"Mann-Whitney U test p-value: {p_value}")
    plt.figure(figsize=(8, 6))
    plt.boxplot(to_plot, patch_artist=True, showmeans=True)
    
    # Add labels and title
    plt.xticks([1, 2], ['Peak TS Same', 'Peak TS Diff'])
   # plt.yticks(np.arange (10, 40, 5), np.arange (1000, 4000, 500))
    plt.title('Peak Times (Previous Same vs. Diff)')
    plt.ylabel('Peak Times')
    
    # Add standard deviation (as text) for each group
    same_std = np.std(peak_ts_same)
    diff_std = np.std(peak_ts_diff)
    plt.text(1, np.max(peak_ts_same), f'Std: {same_std:.2f}', ha='center')
    plt.text(2, np.max(peak_ts_diff), f'Std: {diff_std:.2f}', ha='center')
    
    
    # Show plot
    plt.show()



In [ ]:
u_stat, p_value = mannwhitneyu(all_max_same, all_max_diff)
print(f"Mann-Whitney U test p-value: {p_value}, stats: {u_stat}")

### Just for the beauty, PC1 and PC2

In [ ]:
cross_pcs = {'os85a': [1,2], 'os85b': [1,3], 'os86a': [2,5], 'os86b': [1,2], 
                     'os87': [1,2], 'os88': [1,4], 'os89': [1,2]}
# Trajectories for 2 selected components
# For each mouse
condition_pairs = [ ['left_side_same', 'left_side_diff'],
                   ['right_side_same', 'right_side_diff'] ]

for id in mouse_ids:
    print(f"::: Mouse  {id} :::")
    data_exp = data[id]
    behav = data_exp['behav']
    pcas = data[id]['PCA']
    frame_ts = data_exp['frame_ts']
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    ax_n = 0
    comp_ns = cross_pcs[id]
    pc1 = pcas['PC3']; pc2 = pcas['PC2']
    params = ['prev_side', 'side', 'missed']
    span = [0, 2600]
    # ==== LEFT ====
    peak_ts_same = []; dist_same = []
    # same left 
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [1,1,0,1,1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [1,1,0,1,1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
       # segm1 = [s-segm1[0] for s in segm1]
      #  segm2 = [s-segm2[0] for s in segm2]

        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
     #   time_furthest_away = time[max_distance_idx]     
        peak_t = max_distance_idx; # abs_segm.index(max(abs_segm))
        peak_ts_same.append(peak_t)
        #segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'b', alpha=0.5)
       # axs[ax_n].plot(segm1[0], segm2[0], 'b', marker='o', alpha=0.5)
        axs[ax_n].plot(segm1[-1], segm2[-1], 'b', marker='D', alpha=0.5)
        # Assuming segm1 and segm2 are lists or arrays of coordinates [x1, y1] and [x2, y2]
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_same.append(distance)

    # same left 
    peak_ts_diff = []; dist_diff = []
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [-1,1,0,1,1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [-1,1,0,1,1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
      #  segm1 = [s-segm1[0] for s in segm1]
      #  segm2 = [s-segm2[0] for s in segm2]
        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
      #  time_furthest_away = time[max_distance_idx]

        
        peak_t = max_distance_idx #abs_segm.index(max(abs_segm))
        peak_ts_diff.append(peak_t)
        segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'c', alpha=0.5)
        #axs[ax_n].plot(segm1[0], segm2[0], 'c', marker='o', alpha=0.5)
        axs[ax_n].plot(segm1[-1], segm2[-1], 'c', marker='D', alpha=0.5)
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_diff.append(distance)
    
    print(np.mean(peak_ts_same), np.mean(peak_ts_diff)) 
    stat, p_val = stats.ttest_ind(peak_ts_same, peak_ts_diff)    
    print('p for time to peak', p_val)
    '''
    print(np.mean(dist_same), np.mean(dist_diff)) 
    stat, p_val = stats.ttest_ind(dist_same, dist_diff)    
    print('p for distances', p_val)
    '''
    # ==== RIGHT =====
    ax_n += 1
    peak_ts_same = []; dist_same = []
    # same left 
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [-1,-1,0,-1,-1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [-1,-1,0,-1,-1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
      #  segm1 = [s-segm1[-1] for s in segm1]
      #  segm2 = [s-segm2[-1] for s in segm2]

        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
     #   time_furthest_away = time[max_distance_idx]


        
        peak_t = max_distance_idx; # abs_segm.index(max(abs_segm))
        peak_ts_same.append(peak_t)
        segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'r', alpha=0.5)
        axs[ax_n].plot(segm1[0], segm2[0], 'r', marker='o', alpha=0.5)
       # axs[ax_n].plot(segm1[-1], segm2[-1], 'r', marker='D', alpha=0.5)
        # Assuming segm1 and segm2 are lists or arrays of coordinates [x1, y1] and [x2, y2]
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_same.append(distance)

    # same left 
    #ax_n += 1
    peak_ts_diff = []; dist_diff = []
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [1,-1,0,-1,-1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [1,-1,0,-1,-1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
     #   segm1 = [s-segm1[-1] for s in segm1]
      #  segm2 = [s-segm2[-1] for s in segm2]
        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
      #  time_furthest_away = time[max_distance_idx]

        
        peak_t = max_distance_idx #abs_segm.index(max(abs_segm))
        peak_ts_diff.append(peak_t)
        segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'm', alpha=0.5)
        axs[ax_n].plot(segm1[0], segm2[0], 'm', marker='o', alpha=0.5)
       # axs[ax_n].plot(segm1[-1], segm2[-1], 'm', marker='D', alpha=0.5)
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_diff.append(distance)
    
    print(np.mean(peak_ts_same), np.mean(peak_ts_diff)) 
    stat, p_val = stats.ttest_ind(peak_ts_same, peak_ts_diff)    
    print('p for time to peak', p_val)
    '''
    print(np.mean(dist_same), np.mean(dist_diff)) 
    stat, p_val = stats.ttest_ind(dist_same, dist_diff)    
    print('p for distances', p_val)
    '''


    
    plt.show()
    

### Same but for laser

In [ ]:
cross_pcs = {'os85a': [1,2], 'os85b': [1,3], 'os86a': [2,5], 'os86b': [1,2], 
                     'os87': [1,2], 'os88': [1,4], 'os89': [1,2]}
comps_resp
all_max_same = []
all_max_diff = []
for id in mouse_ids:
    mouse_max_same = []
    mouse_max_diff = []
    print(f"::: Mouse  {id} :::")
    data_exp = data[id]
    behav = data_exp['behav']
    pcas = data[id]['PCA']
    frame_ts = data_exp['frame_ts']
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    ax_n = 0
    comp_resp_mouse = comps_resp[id]
    comp_prev_mouse = comps_prev[id]
    c1 = comp_resp_mouse[0]
    if c1 == comp_prev_mouse[0]:
        c2 = comp_prev_mouse[1]
    else:
        c2 =  comp_prev_mouse[0]
        
    
    pc1 = pcas['PC' + str(c1)]; pc2 = pcas['PC' + str(c2)]
    params = ['prev_side', 'side', 'missed', 'laser', 'choice']
    span = [0, 3000]
    # ==== LEFT ====
    peak_vals_same = []; dist_same = []
    # same left 
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [1,1,0,1,1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [1,1,0,1,1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
       # segm1 = [s-segm1[0] for s in segm1]
      #  segm2 = [s-segm2[0] for s in segm2]

        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
     #   time_furthest_away = time[max_distance_idx]     
        peak_t = max_distance_idx; # abs_segm.index(max(abs_segm))
        peak_vals_same.append(max(distances))
        #segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'b', alpha=0.5)
       # axs[ax_n].plot(segm1[0], segm2[0], 'b', marker='o', alpha=0.5)
        axs[ax_n].plot(segm1[-1], segm2[-1], 'b', marker='D', alpha=0.5)
        # Assuming segm1 and segm2 are lists or arrays of coordinates [x1, y1] and [x2, y2]
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_same.append(distance)

    # same left 
    peak_vals_diff = []; dist_diff = []
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [-1,1,0,1,1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [-1,1,0,1,1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
      #  segm1 = [s-segm1[0] for s in segm1]
      #  segm2 = [s-segm2[0] for s in segm2]
        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
      #  time_furthest_away = time[max_distance_idx]

        
        peak_t = max_distance_idx #abs_segm.index(max(abs_segm))
        peak_vals_diff.append(max(distances))
        segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'c', alpha=0.5)
        #axs[ax_n].plot(segm1[0], segm2[0], 'c', marker='o', alpha=0.5)
        axs[ax_n].plot(segm1[-1], segm2[-1], 'c', marker='D', alpha=0.5)
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_diff.append(distance)
    
    print(np.mean(peak_vals_same), np.mean(peak_vals_diff)) 
    stat, p_val = stats.ttest_ind(peak_vals_same, peak_vals_diff)    
    print('p for max val', p_val)
    all_max_same += peak_vals_same
    all_max_diff += peak_vals_diff
    mouse_max_same += peak_vals_same
    mouse_max_diff += peak_vals_diff
    '''
    print(np.mean(dist_same), np.mean(dist_diff)) 
    stat, p_val = stats.ttest_ind(dist_same, dist_diff)    
    print('p for distances', p_val)
    '''
    # ==== RIGHT =====
    ax_n += 1
    peak_vals_same = []; dist_same = []
    # same left 
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [-1,-1,0,-1,-1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [-1,-1,0,-1,-1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
      #  segm1 = [s-segm1[-1] for s in segm1]
      #  segm2 = [s-segm2[-1] for s in segm2]

        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
     #   time_furthest_away = time[max_distance_idx]


        
        peak_t = max_distance_idx; # abs_segm.index(max(abs_segm))
        peak_vals_same.append(max(distances))
        segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'r', alpha=0.5)
        axs[ax_n].plot(segm1[0], segm2[0], 'r', marker='o', alpha=0.5)
       # axs[ax_n].plot(segm1[-1], segm2[-1], 'r', marker='D', alpha=0.5)
        # Assuming segm1 and segm2 are lists or arrays of coordinates [x1, y1] and [x2, y2]
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_same.append(distance)

    # same left 
    #ax_n += 1
    peak_vals_diff = []; dist_diff = []
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [1,-1,0,-1,-1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [1,-1,0,-1,-1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
     #   segm1 = [s-segm1[-1] for s in segm1]
      #  segm2 = [s-segm2[-1] for s in segm2]
        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
      #  time_furthest_away = time[max_distance_idx]

        
        peak_t = max_distance_idx #abs_segm.index(max(abs_segm))
        peak_vals_diff.append(max(distances))
        segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'm', alpha=0.5)
        axs[ax_n].plot(segm1[0], segm2[0], 'm', marker='o', alpha=0.5)
       # axs[ax_n].plot(segm1[-1], segm2[-1], 'm', marker='D', alpha=0.5)
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_diff.append(distance)
    
    print(np.mean(peak_vals_same), np.mean(peak_vals_diff)) 
    stat, p_val = stats.ttest_ind(peak_vals_same, peak_vals_diff)    
    print('p for max val', p_val)
    all_max_same += peak_vals_same
    all_max_diff += peak_vals_diff
    mouse_max_same += peak_vals_same
    mouse_max_diff += peak_vals_diff
    data[id]['max_comp_laser'] = {'same': mouse_max_same, 'diff': mouse_max_diff}
    '''
    print(np.mean(dist_same), np.mean(dist_diff)) 
    stat, p_val = stats.ttest_ind(dist_same, dist_diff)    
    print('p for distances', p_val)
    '''


    
    plt.show()
    

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

for id in mouse_ids:
    peak_ts_same = data[id]['max_comp_laser']['same']
    peak_ts_diff = data[id]['max_comp_laser']['diff']
    
    # Combine data for boxplot
    to_plot = [peak_ts_same, peak_ts_diff]
    # Create boxplot
    from scipy.stats import mannwhitneyu
    
    u_stat, p_value = mannwhitneyu(peak_ts_same, peak_ts_diff)
    print(f"Mann-Whitney U test p-value: {p_value}")
    plt.figure(figsize=(8, 6))
    plt.boxplot(to_plot, patch_artist=True, showmeans=True)
    
    # Add labels and title
    plt.xticks([1, 2], ['Peak TS Same', 'Peak TS Diff'])
   # plt.yticks(np.arange (10, 40, 5), np.arange (1000, 4000, 500))
    plt.title('Peak Times (Previous Same vs. Diff)')
    plt.ylabel('Peak Times')
    
    # Add standard deviation (as text) for each group
    same_std = np.std(peak_ts_same)
    diff_std = np.std(peak_ts_diff)
    plt.text(1, np.max(peak_ts_same), f'Std: {same_std:.2f}', ha='center')
    plt.text(2, np.max(peak_ts_diff), f'Std: {diff_std:.2f}', ha='center')
    
    # Show plot
    plt.show()



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
all_las = []; all_nolas = []
all_same_las = []; all_same_nolas = []
all_diff_las = []; all_diff_nolas = []
for id in mouse_ids[:-2]:
    peak_ts_diff = data[id]['max_comp_laser']['diff'] + data[id]['max_comp_laser']['same']
    peak_ts_same = data[id]['max_comp']['diff'] + data[id]['max_comp']['same']
    all_las+=peak_ts_diff; all_nolas+=peak_ts_same
    all_same_las += data[id]['max_comp_laser']['same']
    all_same_nolas += data[id]['max_comp']['same']
    all_diff_las += data[id]['max_comp_laser']['diff']
    all_diff_nolas += data[id]['max_comp']['diff']
    # Combine data for boxplot
    to_plot = [peak_ts_same, peak_ts_diff]
    # Create boxplot
    from scipy.stats import mannwhitneyu
    
    u_stat, p_value = mannwhitneyu(peak_ts_same, peak_ts_diff)
    print(f"Mann-Whitney U test p-value: {p_value}")
    plt.figure(figsize=(8, 6))
    plt.boxplot(to_plot, patch_artist=True, showmeans=True)
    
    # Add labels and title
    plt.xticks([1, 2], ['Peak TS Same', 'Peak TS Diff'])
   # plt.yticks(np.arange (10, 40, 5), np.arange (1000, 4000, 500))
    plt.title('Peak Times (Previous Same vs. Diff)')
    plt.ylabel('Peak Times')
    
    # Add standard deviation (as text) for each group
    same_std = np.std(peak_ts_same)
    diff_std = np.std(peak_ts_diff)
    plt.text(1, np.max(peak_ts_same), f'Std: {same_std:.2f}', ha='center')
    plt.text(2, np.max(peak_ts_diff), f'Std: {diff_std:.2f}', ha='center')
    
    
    # Show plot
    plt.show()



In [ ]:
u_stat, p_value = mannwhitneyu(all_diff_las, all_diff_nolas)
print(f"Mann-Whitney U test p-value: {p_value}, {u_stat}")

### Table

In [ ]:
cross_pcs = {'os85a': [1,2], 'os85b': [1,3], 'os86a': [2,5], 'os86b': [1,2], 
                     'os87': [1,2], 'os88': [1,4], 'os89': [1,2]}
cross_pcs = {'os85a': [3,4], 'os85b': [3,6], 'os86a': [1,5], 'os86b': [2,7], 
  #                   'os87': [2,5], 'os88': [1,2], 'os89': [5,10]}
# ================ LASER NOW ===================
for id in mouse_ids:
    print(f"::: Mouse  {id} :::")
    data_exp = data[id]
    behav = data_exp['behav']
    pcas = data[id]['PCA']
    frame_ts = data_exp['frame_ts']
    fig, axs = plt.subplots(1, 4, figsize=(15, 3))
    ax_n = 0
    comp_ns = cross_pcs[id]
    pc1 = pcas['PC' + str(comp_ns[0])]; pc2 = pcas['PC' + str(comp_ns[1])]
    params = ['prev_side', 'side', 'missed', 'laser', 'choice']
    span = [-700, 4000]
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # ==== LEFT ====
    peak_vals_same = []; dist_same = []
    # same left 
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [1,1,0,0,1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [1,1,0,0,1], span=span)
    table_means = {'side': [], 'laser': [], 'is_same': [], 'mean': []}
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
       # segm1 = [s-segm1[0] for s in segm1]
      #  segm2 = [s-segm2[0] for s in segm2]

        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
     #   time_furthest_away = time[max_distance_idx]     
        peak_t = max_distance_idx; # abs_segm.index(max(abs_segm))
        peak_vals_same.append(max(distances))
        #segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'b', alpha=0.5)
       # axs[ax_n].plot(segm1[0], segm2[0], 'b', marker='o', alpha=0.5)
        axs[ax_n].plot(segm1[-1], segm2[-1], 'b', marker='D', alpha=0.5)
        # Assuming segm1 and segm2 are lists or arrays of coordinates [x1, y1] and [x2, y2]
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_same.append(distance)

    # same left 
    peak_vals_diff = []; dist_diff = []
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [-1,1,0,0,1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [-1,1,0,0,1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
      #  segm1 = [s-segm1[0] for s in segm1]
      #  segm2 = [s-segm2[0] for s in segm2]
        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
      #  time_furthest_away = time[max_distance_idx]

        
        peak_t = max_distance_idx #abs_segm.index(max(abs_segm))
        peak_vals_diff.append(max(distances))
        segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'c', alpha=0.5)
        #axs[ax_n].plot(segm1[0], segm2[0], 'c', marker='o', alpha=0.5)
        axs[ax_n].plot(segm1[-1], segm2[-1], 'c', marker='D', alpha=0.5)
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_diff.append(distance)
    
   # print(np.mean(peak_vals_same), np.mean(peak_vals_diff)) 
    stat, p_val = stats.ttest_ind(peak_vals_same, peak_vals_diff)    
   # print('p for max val L', p_val)
    table_means['side'].append(1)
    table_means['laser'].append(False)
    table_means['is_same'].append(True)
    table_means['mean'].append(np.mean(peak_vals_same))
    table_means['side'].append(1)
    table_means['laser'].append(False)
    table_means['is_same'].append(False)
    table_means['mean'].append(np.mean(peak_vals_diff))
    
    '''
    print(np.mean(dist_same), np.mean(dist_diff)) 
    stat, p_val = stats.ttest_ind(dist_same, dist_diff)    
    print('p for distances', p_val)
    '''
    # ==== RIGHT =====
    ax_n += 1
    peak_vals_same = []; dist_same = []
    # same left 
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [-1,-1,0,0,-1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [-1,-1,0,0,-1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
      #  segm1 = [s-segm1[-1] for s in segm1]
      #  segm2 = [s-segm2[-1] for s in segm2]

        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
     #   time_furthest_away = time[max_distance_idx]


        
        peak_t = max_distance_idx; # abs_segm.index(max(abs_segm))
        peak_vals_same.append(max(distances))
        segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'r', alpha=0.5)
        axs[ax_n].plot(segm1[0], segm2[0], 'r', marker='o', alpha=0.5)
       # axs[ax_n].plot(segm1[-1], segm2[-1], 'r', marker='D', alpha=0.5)
        # Assuming segm1 and segm2 are lists or arrays of coordinates [x1, y1] and [x2, y2]
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_same.append(distance)

    # same left 
    #ax_n += 1
    peak_vals_diff = []; dist_diff = []
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [1,-1,0,0,-1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [1,-1,0,0,-1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
     #   segm1 = [s-segm1[-1] for s in segm1]
      #  segm2 = [s-segm2[-1] for s in segm2]
        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
      #  time_furthest_away = time[max_distance_idx]

        
        peak_t = max_distance_idx #abs_segm.index(max(abs_segm))
        peak_vals_diff.append(max(distances))
        segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'm', alpha=0.5)
        axs[ax_n].plot(segm1[0], segm2[0], 'm', marker='o', alpha=0.5)
       # axs[ax_n].plot(segm1[-1], segm2[-1], 'm', marker='D', alpha=0.5)
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_diff.append(distance)
    
  #  print(np.mean(peak_vals_same), np.mean(peak_vals_diff)) 
    stat, p_val = stats.ttest_ind(peak_vals_same, peak_vals_diff)    
 #   print('p for max val R', p_val)
    table_means['side'].append(-1)
    table_means['laser'].append(False)
    table_means['is_same'].append(True)
    table_means['mean'].append(np.mean(peak_vals_same))
    table_means['side'].append(-1)
    table_means['laser'].append(False)
    table_means['is_same'].append(False)
    table_means['mean'].append(np.mean(peak_vals_diff))
    '''
    print(np.mean(dist_same), np.mean(dist_diff)) 
    stat, p_val = stats.ttest_ind(dist_same, dist_diff)    
    print('p for distances', p_val)
    '''
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # ==== LEFT ====
    ax_n += 1
    peak_vals_same = []; dist_same = []
    # same left 
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [1,1,0,1,1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [1,1,0,1,1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
       # segm1 = [s-segm1[0] for s in segm1]
      #  segm2 = [s-segm2[0] for s in segm2]

        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
     #   time_furthest_away = time[max_distance_idx]     
        peak_t = max_distance_idx; # abs_segm.index(max(abs_segm))
        peak_vals_same.append(max(distances))
        #segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'b', alpha=0.5)
       # axs[ax_n].plot(segm1[0], segm2[0], 'b', marker='o', alpha=0.5)
        axs[ax_n].plot(segm1[-1], segm2[-1], 'b', marker='D', alpha=0.5)
        # Assuming segm1 and segm2 are lists or arrays of coordinates [x1, y1] and [x2, y2]
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_same.append(distance)

    # same left 
    peak_vals_diff = []; dist_diff = []
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [-1,1,0,1,1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [-1,1,0,1,1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
      #  segm1 = [s-segm1[0] for s in segm1]
      #  segm2 = [s-segm2[0] for s in segm2]
        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
      #  time_furthest_away = time[max_distance_idx]

        
        peak_t = max_distance_idx #abs_segm.index(max(abs_segm))
        peak_vals_diff.append(max(distances))
        segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'c', alpha=0.5)
        #axs[ax_n].plot(segm1[0], segm2[0], 'c', marker='o', alpha=0.5)
        axs[ax_n].plot(segm1[-1], segm2[-1], 'c', marker='D', alpha=0.5)
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_diff.append(distance)
    
  #  print(np.mean(peak_vals_same), np.mean(peak_vals_diff)) 
    stat, p_val = stats.ttest_ind(peak_vals_same, peak_vals_diff)    
  #  print('p for max val L laser', p_val)
    table_means['side'].append(1)
    table_means['laser'].append(True)
    table_means['is_same'].append(True)
    table_means['mean'].append(np.mean(peak_vals_same))
    table_means['side'].append(1)
    table_means['laser'].append(True)
    table_means['is_same'].append(False)
    table_means['mean'].append(np.mean(peak_vals_diff))
    '''
    print(np.mean(dist_same), np.mean(dist_diff)) 
    stat, p_val = stats.ttest_ind(dist_same, dist_diff)    
    print('p for distances', p_val)
    '''
    # ==== RIGHT =====
    ax_n += 1
    peak_vals_same = []; dist_same = []
    # same left 
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [-1,-1,0,-1,-1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [-1,-1,0,-1,-1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
      #  segm1 = [s-segm1[-1] for s in segm1]
      #  segm2 = [s-segm2[-1] for s in segm2]

        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
     #   time_furthest_away = time[max_distance_idx]


        
        peak_t = max_distance_idx; # abs_segm.index(max(abs_segm))
        peak_vals_same.append(max(distances))
        segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'r', alpha=0.5)
        axs[ax_n].plot(segm1[0], segm2[0], 'r', marker='o', alpha=0.5)
       # axs[ax_n].plot(segm1[-1], segm2[-1], 'r', marker='D', alpha=0.5)
        # Assuming segm1 and segm2 are lists or arrays of coordinates [x1, y1] and [x2, y2]
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_same.append(distance)

    # same left 
    #ax_n += 1
    peak_vals_diff = []; dist_diff = []
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [1,-1,0,-1,-1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [1,-1,0,-1,-1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
     #   segm1 = [s-segm1[-1] for s in segm1]
      #  segm2 = [s-segm2[-1] for s in segm2]
        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
      #  time_furthest_away = time[max_distance_idx]

        
        peak_t = max_distance_idx 
        peak_vals_diff.append(max(distances))
        segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'm', alpha=0.5)
        axs[ax_n].plot(segm1[0], segm2[0], 'm', marker='o', alpha=0.5)
       # axs[ax_n].plot(segm1[-1], segm2[-1], 'm', marker='D', alpha=0.5)
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_diff.append(distance)
    
   # print(np.mean(peak_vals_same), np.mean(peak_vals_diff)) 
    stat, p_val = stats.ttest_ind(peak_vals_same, peak_vals_diff)    
   # print('p for max val R laser', p_val)
    table_means['side'].append(-1)
    table_means['laser'].append(True)
    table_means['is_same'].append(True)
    table_means['mean'].append(np.mean(peak_vals_same))
    table_means['side'].append(-1)
    table_means['laser'].append(True)
    table_means['is_same'].append(False)
    table_means['mean'].append(np.mean(peak_vals_diff))
    '''
    print(np.mean(dist_same), np.mean(dist_diff)) 
    stat, p_val = stats.ttest_ind(dist_same, dist_diff)    
    print('p for distances', p_val)
    '''
    table_means = pd.DataFrame(table_means)
  #  print(table_means)
    data[id]['table_means'] = table_means

    
    plt.show()
    

In [ ]:
all_s = []
all_d = []
for id in mouse_ids:
    table_means = data[id]['table_means']
    same_l = table_means[(table_means['is_same']==True) & (table_means['side']==1)]['mean'].tolist()
    same_r = table_means[(table_means['is_same']==True) & (table_means['side']==-1)]['mean'].tolist()
    diff_l = table_means[(table_means['is_same']==False) & (table_means['side']==1)]['mean'].tolist()
    diff_r = table_means[(table_means['is_same']==False) & (table_means['side']==-1)]['mean'].tolist()
   # print(same_las, 'vs', same_nolas)
    plt.plot([0,1], same_l, 'b')
    plt.plot([0,1], same_r, 'r')
    plt.plot([2,3], diff_l, 'c')
    plt.plot([2,3], diff_r, 'm')
    all_s.append(same_l); all_s.append(same_r); 
    all_d.append(diff_l); all_d.append(diff_r); 

#fig = plt.figure()
plt.plot([0,1], np.mean(all_s, axis=0), 'k', linewidth=6)
plt.plot([2,3], np.mean(all_d, axis=0), 'y', linewidth=6)
plt.show()

In [ ]:
all_s = []
all_d = []
for id in mouse_ids:
    table_means = data[id]['table_means']
    same_l = table_means[(table_means['is_same']==True) & (table_means['side']==1)]['mean'].tolist()
    same_r = table_means[(table_means['is_same']==True) & (table_means['side']==-1)]['mean'].tolist()
    diff_l = table_means[(table_means['is_same']==False) & (table_means['side']==1)]['mean'].tolist()
    diff_r = table_means[(table_means['is_same']==False) & (table_means['side']==-1)]['mean'].tolist()
   # print(same_las, 'vs', same_nolas)
    plt.plot([0,1], same_l, 'b')
    plt.plot([0,1], same_r, 'r')
    plt.plot([2,3], diff_l, 'c')
    plt.plot([2,3], diff_r, 'm')
    all_s.append(same_l); all_s.append(same_r); 
    all_d.append(diff_l); all_d.append(diff_r); 

#fig = plt.figure()
plt.plot([0,1], np.mean(all_s, axis=0), 'k', linewidth=6)
plt.plot([2,3], np.mean(all_d, axis=0), 'y', linewidth=6)
plt.show()

In [ ]:

# Trajectories for 2 selected components
# For each mouse
condition_pairs = [ ['left_side_same', 'left_side_diff'],
                   ['right_side_same', 'right_side_diff'] ]


for id in mouse_ids[:-1]:
    dates = data[id].keys()
    # and date
    for date in dates:
        print(f"::: Mouse  {id}, {date} :::")
        data_exp = data[id][date]
        traces = data_exp['ca']
        behav = data_exp['behav']
        pca_df_time_points = data[id][date]['PCA']
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))
        ax_n = 0
        for conditions in condition_pairs:
            print(conditions)
            params = param_val_keys[conditions[0]]['params']
            vals1 = param_val_keys[conditions[0]]['vals']
            vals2 = param_val_keys[conditions[1]]['vals']
    
            pc = pca_df_time_points['PC2']
            n=99
            span=[10,25]
            traces_l_diff_x = get_segments_old(data_exp, params, vals1, n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
            traces_r_diff_x = get_segments_old(data_exp, params, vals2, n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
            pc = pca_df_time_points['PC3']
            traces_l_diff_y = get_segments_old(data_exp, params, vals1, n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
            traces_r_diff_y = get_segments_old(data_exp, params, vals2, n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
            

            colors = [param_val_keys[conditions[0]]['color'], param_val_keys[conditions[1]]['color']]
            for tr_n in np.arange(15): #len(traces_l_diff_x)):
                axs[ax_n].plot(traces_l_diff_x[tr_n], -traces_l_diff_y[tr_n], color=colors[0], alpha=0.2)
                axs[ax_n].plot(traces_l_diff_x[tr_n][0], -traces_l_diff_y[tr_n][0], color=colors[0], marker='o', alpha=0.5)
                axs[ax_n].plot(traces_l_diff_x[tr_n][-1], -traces_l_diff_y[tr_n][-1], color=colors[0], marker='D', alpha=0.5)
          #  ax_n += 1
            for trace_n in np.arange(15): #len(traces_r_diff_x)):    
                axs[ax_n].plot(traces_r_diff_x[trace_n], -traces_r_diff_y[trace_n], color=colors[1], alpha=0.2)
                axs[ax_n].plot(traces_r_diff_x[trace_n][0], -traces_r_diff_y[trace_n][0], color=colors[1], marker='o', alpha=0.5)
                axs[ax_n].plot(traces_r_diff_x[trace_n][-1], -traces_r_diff_y[trace_n][-1], color=colors[1], marker='D', alpha=0.5)

      
            
        ax_n += 1
        
        plt.show()
        

In [ ]:

# Trajectories for 2 selected components
# For each mouse
condition_pairs = [ ['left_side_same_las', 'left_side_diff_las'],
                   ['right_side_same_las', 'right_side_diff_las'] ]


for id in mouse_ids:
    dates = data[id].keys()
    # and date
    for date in dates:
        print(f"::: Mouse  {id}, {date} :::")
        data_exp = data[id][date]
        traces = data_exp['ca']
        behav = data_exp['behav']
        pca_df_time_points = data[id][date]['PCA']
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))
        ax_n = 0
        for conditions in condition_pairs:
            print(conditions)
            params = param_val_keys[conditions[0]]['params']
            vals1 = param_val_keys[conditions[0]]['vals']
            vals2 = param_val_keys[conditions[1]]['vals']
    
            pc = pca_df_time_points['PC2']
            n=99
            span=[-5,25]
            traces_l_diff_x = get_segments_old(data_exp, params, vals1, n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
            traces_r_diff_x = get_segments_old(data_exp, params, vals2, n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
            pc = pca_df_time_points['PC3']
            traces_l_diff_y = get_segments_old(data_exp, params, vals1, n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
            traces_r_diff_y = get_segments_old(data_exp, params, vals2, n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
            

            colors = [param_val_keys[conditions[0]]['color'], param_val_keys[conditions[1]]['color']]
            for tr_n in np.arange(len(traces_l_diff_x)):
                axs[ax_n].plot(traces_l_diff_x[tr_n], -traces_l_diff_y[tr_n], color=colors[0], alpha=0.2)
                axs[ax_n].plot(traces_l_diff_x[tr_n][0], -traces_l_diff_y[tr_n][0], color=colors[0], marker='o', alpha=0.5)
                axs[ax_n].plot(traces_l_diff_x[tr_n][-1], -traces_l_diff_y[tr_n][-1], color=colors[0], marker='D', alpha=0.5)
          #  ax_n += 1
            for trace_n in np.arange(len(traces_r_diff_x)):    
                axs[ax_n].plot(traces_r_diff_x[trace_n], -traces_r_diff_y[trace_n], color=colors[1], alpha=0.2)
                axs[ax_n].plot(traces_r_diff_x[trace_n][0], -traces_r_diff_y[trace_n][0], color=colors[1], marker='o', alpha=0.5)
                axs[ax_n].plot(traces_r_diff_x[trace_n][-1], -traces_r_diff_y[trace_n][-1], color=colors[1], marker='D', alpha=0.5)

      
            
        ax_n += 1
        
        plt.show()
        

In [ ]:
len(traces_l_diff_x[0])

In [ ]:
len(t_windows)

In [ ]:
# mouse 86 on 2024-05-26
# For prev. conrtast laser vs no laser
condtions_pairs = [
                    ['left_side', 'right_side'],
                    ['left_side_las', 'right_side_las'],
     ['left_side_diff', 'right_side_diff'],

                  ]
condtions_pairs2 = [
                    ['left_side_same', 'right_side_same'],
                    ['left_side_same_las', 'right_side_same_las'],
                  ]
condtions_pairs3 = [
                    ['left_side_diff', 'right_side_diff'],
                    ['left_side_diff_las', 'right_side_diff_las'],
                  ]

t_windows = []
window_width = 2
for t in np.arange(0,30,window_width):
    t_windows.append([t,t+window_width-1])


# ok lets do ica for all the mice
for id in mouse_ids:
    dates = data[id].keys()
    #if a:
    for date in dates:
        print('=== ' + id + ' ' + date + ' ===')
        data_exp = data[id][date]
        ca_df_time_points = data_exp['PCA']
        # setup the plot
        fig, axs = plt.subplots(1, 3, figsize=(15, 2))
        ax_n = 0
        for conditions in condtions_pairs:
            dist = []
            for w in t_windows:
                distance_t = distance_btw_comp(conditions, data_exp, ca_df_time_points, w=w, span=[0,50])
                dist.append(distance_t)
            axs[ax_n].plot(dist)
        ax_n = ax_n+1
        for conditions in condtions_pairs2:
            dist = []
            for w in t_windows:
                distance_t = distance_btw_comp(conditions, data_exp, ca_df_time_points, w=w, span=[0,50])
                dist.append(distance_t)
            axs[ax_n].plot(dist)
        ax_n = ax_n+1
        for conditions in condtions_pairs3:
            dist = []
            for w in t_windows:
                distance_t = distance_btw_comp(conditions, data_exp, ca_df_time_points, w=w, span=[0,50])
                dist.append(distance_t)
            axs[ax_n].plot(dist)
        plt.show()

In [ ]:
dist

# ICA

In [ ]:

trace_type = 'ca'
# ok lets do ica for all the mice
for id in mouse_ids:
    print('=== ' + id + ' ===')
    data_exp = data[id]
    n_components = 12 #n_comps[id]
    usable = [ data_exp['good_neurons'][n]==1 or data_exp['fair_neurons'][n]==1 for n in np.arange(len(data_exp['good_neurons']))]
    traces = data_exp[trace_type]
    traces = traces[np.array(usable)]
    # Reshape data for PCA (each row is a time point, each column is a neuron)
    traces_t = traces.T
    # Perform ICA on the entire dataset
    
    ica = FastICA(n_components=n_components, random_state=0)
    ica_result = ica.fit_transform(traces_t)
    
    # Independent components (mixing matrix)
    components = ica.mixing_
    
    # Create a DataFrame for easier handling
    columns = ['IC' + str(n+1) for n in np.arange(n_components)]
    ica_df_time_points = pd.DataFrame(ica_result, columns=columns)

    data[id]['ICA'] = ica_df_time_points



## Browse ICAs

In [ ]:

trace_type = 'ca'
# ok lets do ica for all the mice
for id in mouse_ids:
    print('=== ' + id + ' ===')
    data_exp = data[id]
    behav = data_exp['behav']
    n_components = 16
    usable = [ data_exp['good_neurons'][n]==1 or data_exp['fair_neurons'][n]==1 for n in np.arange(len(data_exp['good_neurons']))]
    traces = data_exp[trace_type]
    traces = traces[np.array(usable)]
    # Reshape data for PCA (each row is a time point, each column is a neuron)
    traces_t = traces.T
    # Perform ICA on the entire dataset
    for n_components in [12]:
        print('=== ' + str(n_components) + ' ===')
        ica = FastICA(n_components=n_components, random_state=0)
        ica_result = ica.fit_transform(traces_t)
        # Independent components (mixing matrix)
        mixing_matrix = ica.mixing_
        # Create a DataFrame for easier handling
        ica_cols = ['IC' + str(n+1) for n in np.arange(n_components)]
        comps = pd.DataFrame(ica_result, columns=ica_cols)
        # This is A * S
        mixing_matrix_abs = np.abs(mixing_matrix)
        normalized_mixing = mixing_matrix_abs / mixing_matrix_abs.max(axis=0)
        
        # Create a DataFrame for easier visualization
        neuron_contributions = pd.DataFrame(
            normalized_mixing, 
            columns=['IC' + str(i+1) for i in range(n_components)],
            index=['Neuron' + str(i+1) for i in range(mixing_matrix.shape[0])]
        )
        '''
        # Optionally, sort each column by contribution to see which neurons contribute the most to each IC
        for ic in neuron_contributions.columns:
            print(f"\nTop contributors to {ic}:")
            print(neuron_contributions[ic].sort_values(ascending=False).head(10))
            '''

       
       # data[id]['ICA'] = ica_df_time_points
        # prep the figure
        n_cols_fig = 5
        rows_fig = math.ceil(n_components/n_cols_fig)
        fig, axs = plt.subplots(rows_fig, n_cols_fig, figsize=(15, rows_fig*3))
        ax_n=0
        pl_n = 0
        # go through the components
        for comp_n in ica_cols:
            ax_r = pl_n // n_cols_fig; ax_c = pl_n % n_cols_fig
            ax = ax_r, ax_c
            pl_n += 1
            comp = comps[comp_n]
            params = ['prev_reward', 'side', 'choice', 'missed', 'free']
            span = [1000,4000]
            
            segms_l, segm_ts_l = get_segments(behav, comp, frame_ts, params, [1,1,1,0,0], span=span)
            segms_r, segm_ts_r = get_segments(behav, comp, frame_ts, params, [-1,-1,-1,0,0], span=span)
     
            # side  
            plot_aver_sem(segms_l, segm_ts_l, 'b', ghost_traces=False, ax_n=ax)
            plot_aver_sem(segms_r, segm_ts_r, 'r', ghost_traces=False, ax_n=ax)
            '''
            # diff
            segms_l, segm_ts_l = get_segments(behav, comp, frame_ts, params, [1.1,1,-1,0,0], span=span)
            segms_r, segm_ts_r = get_segments(behav, comp, frame_ts, params, [2.2,-1,1,0,0], span=span)
            # side  
            plot_aver_sem(segms_l, segm_ts_l, 'b', linestyle=':', ghost_traces=False, ax_n=ax)
            plot_aver_sem(segms_r, segm_ts_r, 'r', linestyle=':', ghost_traces=False, ax_n=ax)
            '''
            
            segms_l, segm_ts_l = get_segments(behav, comp, frame_ts, params, [-1,1,1,0,0], span=span)
            segms_r, segm_ts_r = get_segments(behav, comp, frame_ts, params, [1,-1,-1,0,0], span=span)
            
     
            # side  
            plot_aver_sem(segms_l, segm_ts_l, 'c', ghost_traces=False, ax_n=ax)
            plot_aver_sem(segms_r, segm_ts_r, 'm', ghost_traces=False, ax_n=ax)
            '''
            # diff
            segms_l, segm_ts_l = get_segments(behav, comp, frame_ts, params, [-1,1,-1,0,0], span=span)
            segms_r, segm_ts_r = get_segments(behav, comp, frame_ts, params, [1,-1,1,0,0], span=span)
            # side  
            plot_aver_sem(segms_l, segm_ts_l, 'c', linestyle=':', ghost_traces=False, ax_n=ax)
            plot_aver_sem(segms_r, segm_ts_r, 'm', linestyle=':', ghost_traces=False, ax_n=ax)
            '''

            axs[ax].axvline(x=0)
            axs[ax].axvline(x=1150)
            axs[ax].set_title(comp_n)
        
            plt.title(comp_n)
            plt.xlabel('time')
            plt.ylabel('IC')
        plt.show()
        # just left vs right
        fig, axs = plt.subplots(rows_fig, n_cols_fig, figsize=(15, rows_fig*3))
        ax_n=0
        pl_n = 0
        # go through the components
        for comp_n in ica_cols:
            ax_r = pl_n // n_cols_fig; ax_c = pl_n % n_cols_fig
            ax = ax_r, ax_c
            pl_n += 1
            comp = comps[comp_n]
            params = [ 'side', 'choice', 'missed', 'free']
            span = [1000,4000]
            
            segms_l, segm_ts_l = get_segments(behav, comp, frame_ts, params, [1,1,0,0], span=span)
            segms_r, segm_ts_r = get_segments(behav, comp, frame_ts, params, [-1,-1,0,0], span=span)
     
            # side  
            plot_aver_sem(segms_l, segm_ts_l, 'b', ghost_traces=False, ax_n=ax)
            plot_aver_sem(segms_r, segm_ts_r, 'r', ghost_traces=False, ax_n=ax)

            axs[ax].axvline(x=0)
            axs[ax].axvline(x=1150)
            axs[ax].set_title(comp_n)
        
            plt.title(comp_n)
            plt.xlabel('time')
            plt.ylabel('IC')
        plt.show()

        # prev trial left over
        fig, axs = plt.subplots(rows_fig, n_cols_fig, figsize=(15, rows_fig*3))
        ax_n=0
        pl_n = 0
        # go through the components
        for comp_n in ica_cols:
            ax_r = pl_n // n_cols_fig; ax_c = pl_n % n_cols_fig
            ax = ax_r, ax_c
            pl_n += 1
            comp = comps[comp_n]
            params = [ 'prev_reward', 'missed', 'free']
            span = [1000,4000]
            
            segms_l, segm_ts_l = get_segments(behav, comp, frame_ts, params, [1,0,0], span=span)
            segms_r, segm_ts_r = get_segments(behav, comp, frame_ts, params, [-1,0,0], span=span)
     
            # side  
            plot_aver_sem(segms_l, segm_ts_l, 'b', ghost_traces=False, ax_n=ax)
            plot_aver_sem(segms_r, segm_ts_r, 'r', ghost_traces=False, ax_n=ax)

            axs[ax].axvline(x=0)
            axs[ax].axvline(x=1150)
            axs[ax].set_title(comp_n)
        
            plt.title(comp_n)
            plt.xlabel('time')
            plt.ylabel('IC')
        plt.show()


In [ ]:
from sklearn.linear_model import LogisticRegression
for id in mouse_ids:
    print(f"::: Mouse  {id} :::")
    data_exp = data[id]
    behav = data_exp['behav']
    frame_ts = data_exp['frame_ts']
    comps = data[id]['PCA']
    comp_cols = comps.keys()
    params = [ 'side', 'choice', 'missed', 'free']
    params_prev = ['prev_reward']
    t1 = -1200; t2 = 3000
    t_pr1 =  1000; t_pr2 =  1000
    all_aucs_prev = []
    all_aucs_resp = []
    for comp_n in comp_cols:
        compon = comps[comp_n]
        behav_l = get_behav_ts(behav, params, [1, 1, 0, 0])
        behav_r = get_behav_ts(behav, params, [-1, -1, 0, 0])
        auc_l = get_auc(behav_l, compon, frame_ts, t1,  t2)
        auc_r = get_auc(behav_r, compon, frame_ts, t1,  t2)
        all_aucs_resp.append(auc_l+auc_r)
        behav_l = get_behav_ts(behav, params_prev, [1])
        behav_r = get_behav_ts(behav, params_prev, [-1])
        auc_l_pr = get_auc(behav_l, compon, frame_ts, t_pr1,  t_pr2)
        auc_r_pr = get_auc(behav_r, compon, frame_ts, t_pr1,  t_pr2)
        all_aucs_prev.append(auc_l_pr+auc_r_pr)
    labels_resp = [1]*len(auc_l) + [-1]*len(auc_r)
    labels_prev = [1]*len(auc_l_pr) + [-1]*len(auc_r_pr)
    
    # For response
    y = np.array(labels_resp)
    all_aucs_resp = np.array(all_aucs_resp)
    X = all_aucs_resp.T
    # Initialize LogisticRegression
    log_reg = LogisticRegression()
    # Fit the model
    log_reg.fit(X, y)
    # Get the coefficients (weights) assigned to each component
    coefficients = log_reg.coef_[0]
    # Sort components by their absolute coefficient value to find the most important ones
    sorted_indices = np.argsort(np.abs(coefficients))[::-1]
    sorted_components = coefficients[sorted_indices]
    
    print("Most important components in separating the RESPONSE:", sorted_indices+1)
    print("Corresponding coefficients:", sorted_components)

    # for prev 
    y = np.array(labels_prev)
    all_aucs_prev = np.array(all_aucs_prev)
    X = all_aucs_prev.T
    # Initialize LogisticRegression
    log_reg = LogisticRegression()
    # Fit the model
    log_reg.fit(X, y)
    # Get the coefficients (weights) assigned to each component
    coefficients = log_reg.coef_[0]
    # Sort components by their absolute coefficient value to find the most important ones
    sorted_indices = np.argsort(np.abs(coefficients))[::-1]
    sorted_components = coefficients[sorted_indices]
    
    print("Most important components in separating the PREV:", sorted_indices+1)
    print("Corresponding coefficients:", sorted_components)





In [ ]:

trace_type = 'ca'
# ok lets do ica for all the mice
for id in mouse_ids:
    print('=== ' + id + ' ===')
    data_exp = data[id]
    behav = data_exp['behav']
    n_components = 12 #n_comps[id]
    usable = [ data_exp['good_neurons'][n]==1 or data_exp['fair_neurons'][n]==1 for n in np.arange(len(data_exp['good_neurons']))]
    traces = data_exp[trace_type]
    traces = traces[np.array(usable)]
    # Reshape data for PCA (each row is a time point, each column is a neuron)
    traces_t = traces.T
    # Perform ICA on the entire dataset
    for n_components in [16]: # np.arange(8,15):
        print('=== ' + str(n_components) + ' ===')
        ica = FastICA(n_components=n_components, random_state=0)
        ica_result = ica.fit_transform(traces_t)
        # Independent components (mixing matrix)
        mixing_matrix = ica.mixing_
        # Create a DataFrame for easier handling
        ica_cols = ['IC' + str(n+1) for n in np.arange(n_components)]
        comps = pd.DataFrame(ica_result, columns=ica_cols)
        # This is A * S
        mixing_matrix_abs = np.abs(mixing_matrix)
        normalized_mixing = mixing_matrix_abs / mixing_matrix_abs.max(axis=0)
        
        # Create a DataFrame for easier visualization
        neuron_contributions = pd.DataFrame(
            normalized_mixing, 
            columns=['IC' + str(i+1) for i in range(n_components)],
            index=['Neuron' + str(i+1) for i in range(mixing_matrix.shape[0])]
        )
        '''
        # Optionally, sort each column by contribution to see which neurons contribute the most to each IC
        for ic in neuron_contributions.columns:
            print(f"\nTop contributors to {ic}:")
            print(neuron_contributions[ic].sort_values(ascending=False).head(10))
            '''

       
        data[id]['ICA'] = comps
        # prep the figure
        n_cols_fig = 4
        rows_fig = math.ceil(n_components/n_cols_fig)
      #  fig, axs = plt.subplots(rows_fig, n_cols_fig, figsize=(15, rows_fig*3))
        ax_n=0
        pl_n = 0
        # go through the components
        '''
        for comp_n in ica_cols:
            ax_r = pl_n // n_cols_fig; ax_c = pl_n % n_cols_fig
            ax = ax_r, ax_c
            pl_n += 1
            comp = comps[comp_n]
            params = ['prev_reward', 'side', 'choice', 'missed', 'laser']
            span = [1000,4000]
            
            segms_l, segm_ts_l = get_segments(behav, comp, frame_ts, params, [1,1,1,0,0], span=span)
            segms_r, segm_ts_r = get_segments(behav, comp, frame_ts, params, [-1,-1,-1,0,0], span=span)
     
            # side  
            plot_aver_sem(segms_l, segm_ts_l, 'b', ghost_traces=False, ax_n=ax)
            plot_aver_sem(segms_r, segm_ts_r, 'r', ghost_traces=False, ax_n=ax)
            
            segms_l, segm_ts_l = get_segments(behav, comp, frame_ts, params, [-1,1,1,0,0], span=span)
            segms_r, segm_ts_r = get_segments(behav, comp, frame_ts, params, [1,-1,-1,0,0], span=span)
               
            # side  
            plot_aver_sem(segms_l, segm_ts_l, 'c', ghost_traces=False, ax_n=ax)
            plot_aver_sem(segms_r, segm_ts_r, 'm', ghost_traces=False, ax_n=ax)


            axs[ax].axvline(x=0)
            axs[ax].axvline(x=1150)
        
            plt.title(comp_n)
            plt.xlabel('time')
            plt.ylabel('IC')
        plt.show()
        '''
        # LASER VS NO LASER
        fig, axs = plt.subplots(rows_fig, n_cols_fig, figsize=(15, rows_fig*3))
        ax_n=0
        pl_n = 0
        # go through the components
        for comp_n in ica_cols:
            ax_r = pl_n // n_cols_fig; ax_c = pl_n % n_cols_fig
            ax = ax_r, ax_c
            pl_n += 1
            comp = comps[comp_n]
            params = [ 'side', 'choice', 'missed', 'laser']
            span = [1000,4000]
            
            segms_l, segm_ts_l = get_segments(behav, comp, frame_ts, params, [1,1,0,0], span=span)
            segms_r, segm_ts_r = get_segments(behav, comp, frame_ts, params, [-1,-1,0,0], span=span)
     
            # side  
            plot_aver_sem(segms_l, segm_ts_l, 'b', ghost_traces=False, ax_n=ax)
            plot_aver_sem(segms_r, segm_ts_r, 'r', ghost_traces=False, ax_n=ax)

            segms_l, segm_ts_l = get_segments(behav, comp, frame_ts, params, [1,1,0,1], span=span)
            segms_r, segm_ts_r = get_segments(behav, comp, frame_ts, params, [-1,-1,0,-1], span=span)
     
            # side  
            plot_aver_sem(segms_l, segm_ts_l, 'b', linestyle=':', ghost_traces=False, ax_n=ax)
            plot_aver_sem(segms_r, segm_ts_r, 'r', linestyle=':', ghost_traces=False, ax_n=ax)

            axs[ax].axvline(x=0)
            axs[ax].axvline(x=1150)
            axs[ax].set_title(comp_n)
        
            plt.title(comp_n)
            plt.xlabel('time')
            plt.ylabel('IC')
        plt.show()
        '''
        # prev trial left over
        fig, axs = plt.subplots(rows_fig, n_cols_fig, figsize=(15, rows_fig*3))
        ax_n=0
        pl_n = 0
        # go through the components
        for comp_n in ica_cols:
            ax_r = pl_n // n_cols_fig; ax_c = pl_n % n_cols_fig
            ax = ax_r, ax_c
            pl_n += 1
            comp = comps[comp_n]
            params = [ 'prev_reward', 'missed', 'free']
            span = [1000,4000]
            
            segms_l, segm_ts_l = get_segments(behav, comp, frame_ts, params, [1,0,0], span=span)
            segms_r, segm_ts_r = get_segments(behav, comp, frame_ts, params, [-1,0,0], span=span)
     
            # side  
            plot_aver_sem(segms_l, segm_ts_l, 'b', ghost_traces=False, ax_n=ax)
            plot_aver_sem(segms_r, segm_ts_r, 'r', ghost_traces=False, ax_n=ax)

            axs[ax].axvline(x=0)
            axs[ax].axvline(x=1150)
        
            axs[ax].title(comp_n)
            plt.xlabel('time')
            plt.ylabel('IC')
        
        plt.show()
        '''

In [ ]:

# For each mouse
for id in mouse_ids:
    dates = data[id]
    # and date
    
    print(f"::: Mouse  {id},  :::")
    data_exp = data[id][date]
    ica_df_time_points = data[id]['ICA']
    ica_cols = ica_df_time_points.keys()

    # go through the 1st n components
    for ica_comp in ica_cols:
        n=0
        pc = ica_df_time_points[ica_comp]
        
        traces_l = get_segments_old(data_exp, ['side', 'laser'], [1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
        traces_r = get_segments_old(data_exp, ['side', 'laser'],[-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
        
        fig, axs = plt.subplots(1, 2, figsize=(15, 3))
        # side  
        plot_aver_se_old(traces_l, 'b',ghost_traces=False, axn=0)
        plot_aver_se_old(traces_r, 'r',ghost_traces=False, axn=0)
    
        # choice
        traces_l = get_segments_old(data_exp, ['choice', 'laser'], [1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
        traces_r = get_segments_old(data_exp, ['choice', 'laser'], [-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
      #  traces_0 = get_segments_old(data_exp, ['choice', 'laser'], [0,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
        plot_aver_se_old(traces_l, 'b',ghost_traces=False, axn=1)
        plot_aver_se_old(traces_r, 'r',ghost_traces=False, axn=1)
     #   plot_aver_se_old(traces_0, 'k',ghost_traces=False, axn=1)
    
        for ax_n in [0,1]:
            axs[ax_n].axvline(x=0)
            axs[ax_n].axvline(x=1150)
    
        plt.title(ica_comp)
        plt.xlabel('time')
        plt.ylabel('au')
        plt.show()

In [ ]:
ica_cols = ica_columns
# For each mouse
for id in mouse_ids:
    dates = data[id].keys()
    # and date
    for date in dates:
        print(f"::: Mouse  {id}, {date} :::")
        data_exp = data[id][date]
        ica_df_time_points = data[id][date]['ICA']

        # go through the 1st n components
        for ica_comp in ica_cols:
            n=0
            pc = ica_df_time_points[ica_comp]
            params = ['side', 'choice', 'prev_reward', 'laser']
            traces_l_same = get_segments_old(data_exp, params, [1,1,1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
            traces_l_diff = get_segments_old(data_exp, params, [1,1,-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
            traces_r_same = get_segments_old(data_exp, params, [-1,-1,-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
            traces_r_diff = get_segments_old(data_exp, params, [-1,-1,1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
            
            fig, axs = plt.subplots(1, 2, figsize=(15, 3))
            # side  
            plot_aver_se_old(traces_l_same, 'b',ghost_traces=False, axn=0)
            plot_aver_se_old(traces_l_diff, 'c',ghost_traces=False, axn=0)
            plot_aver_se_old(traces_r_same, 'r',ghost_traces=False, axn=0)
            plot_aver_se_old(traces_r_diff, 'm',ghost_traces=False, axn=0)
        
            # same but with laser
            params = ['side', 'choice', 'prev_reward', 'laser']
            traces_l_same = get_segments_old(data_exp, params, [1,1,1,1], n, frame_type='frame_tone', is_pca=True, pc=pc)
            traces_l_diff = get_segments_old(data_exp, params, [1,1,-1,1], n, frame_type='frame_tone', is_pca=True, pc=pc)
            traces_r_same = get_segments_old(data_exp, params, [-1,-1,-1,-1], n, frame_type='frame_tone', is_pca=True, pc=pc)
            traces_r_diff = get_segments_old(data_exp, params, [-1,-1,1,-1], n, frame_type='frame_tone', is_pca=True, pc=pc)
            

            # side  
            plot_aver_se_old(traces_l_same, 'b',ghost_traces=False, axn=1, linestyle=':')
            plot_aver_se_old(traces_l_diff, 'c',ghost_traces=False, axn=1, linestyle=':')
            plot_aver_se_old(traces_r_same, 'r',ghost_traces=False, axn=1, linestyle=':')
            plot_aver_se_old(traces_r_diff, 'm',ghost_traces=False, axn=1, linestyle=':')
         #   plot_aver_se_old(traces_0, 'k',ghost_traces=False, axn=1)
        
            for ax_n in [0,1]:
                axs[ax_n].axvline(x=-200)
                axs[ax_n].axvline(x=950)
        
            plt.title(ica_comp)
            plt.xlabel('time')
            plt.ylabel('au')
            plt.show()

## test the optimal number of ICs

In [ ]:
behav

In [ ]:
for id in mouse_ids:
    dates = data[id].keys()
    #if a:
    for date in dates:
        print('mouse_id = [\'' + id + '\']; dates = [\'' + date + '\']')
        

In [ ]:
#mouse_id = ['os85']; dates = ['2024-06-05']
#mouse_id = ['os85']; dates = ['2024-06-12']
#mouse_id = ['os86']; dates = ['2024-05-26']
mouse_id = ['os86']; dates = ['2024-06-03']
#mouse_id = ['os87']; dates = ['2024-06-12']
#mouse_id = ['os88']; dates = ['2024-06-04']
for id in mouse_id:
   # dates = data[id].keys()
    #if a:
    for date in dates:
        print('=== ' + id + ' ' + date + ' ===')
        


        data_exp = data[id][date]
        traces = data_exp['ca'] 
        usable_neurons = data_exp['usable_neurons']
        traces_clean = [traces[n] for n in np.arange(traces.shape[0]) if usable_neurons[n] == 1]
        traces_clean = np.array(traces_clean)
        # Reshape data for PCA (each row is a time point, each column is a neuron)
        traces_t = traces.T
        traces_clean_t = traces_clean.T
        # Perform ICA on the entire dataset
        n_components = 12
        ica = FastICA(n_components=n_components, random_state=0)
        ica_result = ica.fit_transform(traces_t)
        ica_result_clean = ica.fit_transform(traces_clean_t)
        
        # Independent components (mixing matrix)
        components = ica.mixing_
        
        
        # Create a DataFrame for easier handling
        ica_columns = ['IC' + str(n+1) for n in np.arange(n_components)]
        ica_df_time_points = pd.DataFrame(ica_result, columns=ica_columns)
        ica_df_time_points_clean = pd.DataFrame(ica_result_clean, columns=ica_columns)

      #  data[id][date]['ICA'] = ica_df_time_points

        ica_cols = ica_columns

        print(f"::: Mouse  {id}, {date} :::")


        # go through the 1st n components
        for ica_comp in ica_cols:
            n=0
            pc = ica_df_time_points[ica_comp]
            params = ['side', 'choice', 'prev_reward', 'laser']
            traces_l_same = get_segments_old(data_exp, params, [1,1,1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
            traces_l_diff = get_segments_old(data_exp, params, [1,1,-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
            traces_r_same = get_segments_old(data_exp, params, [-1,-1,-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
            traces_r_diff = get_segments_old(data_exp, params, [-1,-1,1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
            
            fig, axs = plt.subplots(1, 2, figsize=(15, 3))
            # side  
            plot_aver_se_old(traces_l_same, 'b',ghost_traces=False, axn=0)
            plot_aver_se_old(traces_l_diff, 'c',ghost_traces=False, axn=0)
            plot_aver_se_old(traces_r_same, 'r',ghost_traces=False, axn=0)
            plot_aver_se_old(traces_r_diff, 'm',ghost_traces=False, axn=0)
        
            # same but with clean components
            pc = ica_df_time_points_clean[ica_comp]
            params = ['side', 'choice', 'prev_reward', 'laser']
            traces_l_same = get_segments_old(data_exp, params, [1,1,1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
            traces_l_diff = get_segments_old(data_exp, params, [1,1,-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
            traces_r_same = get_segments_old(data_exp, params, [-1,-1,-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
            traces_r_diff = get_segments_old(data_exp, params, [-1,-1,1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
            

            # side  
            plot_aver_se_old(traces_l_same, 'b',ghost_traces=False, axn=1)
            plot_aver_se_old(traces_l_diff, 'c',ghost_traces=False, axn=1)
            plot_aver_se_old(traces_r_same, 'r',ghost_traces=False, axn=1)
            plot_aver_se_old(traces_r_diff, 'm',ghost_traces=False, axn=1)
         #   plot_aver_se_old(traces_0, 'k',ghost_traces=False, axn=1)
        
            for ax_n in [0,1]:
                axs[ax_n].axvline(x=-200)
                axs[ax_n].axvline(x=950)
        
            plt.title(ica_comp)
            plt.xlabel('time')
            plt.ylabel('au')
            plt.show()

## Test the idea with the inter trial intervals
For now, only Mouse  os86, 2024-05-26

In [ ]:

ica_cols = ['IC2', 'IC3']
# For each mouse
for id in ['os88']:
    dates = ['2024-06-04']
    # and date
    for date in dates:
        print(f"::: Mouse  {id}, {date} :::")
        data_exp = data[id][date]
        behav = data[id][date]['behav']
        ica_df_time_points = data[id][date]['ICA']

        # go through the 1st n components
        for ica_comp in ica_cols:
            n=0
            pc = ica_df_time_points[ica_comp]
            params = ['side', 'choice', 'prev_reward', 'laser']
            traces_l_same = get_segments_old(data_exp, params, [1,1,1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
            traces_l_diff = get_segments_old(data_exp, params, [1,1,-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
            traces_r_same = get_segments_old(data_exp, params, [-1,-1,-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
            traces_r_diff = get_segments_old(data_exp, params, [-1,-1,1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
            
            fig, axs = plt.subplots(1, 2, figsize=(15, 3))
            # side  
            plot_aver_se_old(traces_l_same, 'b',ghost_traces=False, axn=0)
            plot_aver_se_old(traces_l_diff, 'c',ghost_traces=False, axn=0)
            plot_aver_se_old(traces_r_same, 'r',ghost_traces=False, axn=0)
            plot_aver_se_old(traces_r_diff, 'm',ghost_traces=False, axn=0)
        
            for ax_n in [0]:
                axs[ax_n].axvline(x=0)
                axs[ax_n].axvline(x=1150)

            # plot the mean value of IC within 800 to 2500 ms window
            
            add_las=0
            for tr_n in np.arange(behav.shape[0]):
                if behav['laser'][tr_n] == 0:
                    if (behav['side'][tr_n] == 1) & (behav['choice'][tr_n] == 1):
                        if (behav['prev_reward'][tr_n] == 1):
                            t = behav['frame_tone'][tr_n]
                            ic_segm_av = np.mean(pc[t+15:t+25])
                            iti_tr = behav['ITI'][tr_n]
                       #     axs[1].plot(iti_tr, ic_segm_av, 'b*')
                        elif (behav['prev_reward'][tr_n] == -1):
                            t = behav['frame_tone'][tr_n]
                            ic_segm_av = np.mean(pc[t+15:t+25])
                            iti_tr = behav['ITI'][tr_n]
                            axs[1].plot(iti_tr, ic_segm_av, 'c*')
                        elif (behav['prev_reward'][tr_n] == 0):
                            t = behav['frame_tone'][tr_n]
                            ic_segm_av = np.mean(pc[t+15:t+25])
                            iti_tr = behav['ITI'][tr_n]
                        #    axs[1].plot(iti_tr, ic_segm_av, 'k*')
                     # now, left side
                    if (behav['side'][tr_n] == -1) & (behav['choice'][tr_n] == -1):
                   
                        if (behav['prev_reward'][tr_n] == -1):
                            t = behav['frame_tone'][tr_n]
                            ic_segm_av = np.mean(pc[t+15:t+25])
                            iti_tr = behav['ITI'][tr_n]
                      #      axs[1].plot(iti_tr, ic_segm_av, 'r*')
                        elif (behav['prev_reward'][tr_n] == 1):
                            t = behav['frame_tone'][tr_n]
                            ic_segm_av = np.mean(pc[t+15:t+25])
                            iti_tr = behav['ITI'][tr_n]
                            axs[1].plot(iti_tr, ic_segm_av, 'm*')
                        elif (behav['prev_reward'][tr_n] == 0):
                            t = behav['frame_tone'][tr_n]
                            ic_segm_av = np.mean(pc[t+15:t+25])
                            iti_tr = behav['ITI'][tr_n]
                           # axs[1].plot(iti_tr, ic_segm_av, 'y*')
                    
                
        
        #    plt.title(ica_comp)
         #   plt.xlabel('time')
         #   plt.ylabel('au')
            plt.show()
print(add_las)

In [ ]:


# 1st, do the analysis for only 1 mouse on 1 date
id = 'os86'; date = '2024-05-26' # seems the best in terms of good neurons
data_exp = data[id][date]
traces = data_exp['ca'] 
behav = data_exp['behav']
# Reshape data for PCA (each row is a time point, each column is a neuron)
traces = traces.T
# Perform ICA on the entire dataset
n_components =  10
ica = FastICA(n_components=n_components, random_state=0)
ica_result = ica.fit_transform(traces)

# Independent components (mixing matrix)
components = ica.mixing_


# Create a DataFrame for easier handling
columns = ['IC' + str(n+1) for n in np.arange(n_components)]
ica_df_time_points = pd.DataFrame(ica_result, columns=columns)



In [ ]:
# go through the 1st n components
for ica_comp in columns:
    n=0
    ic = ica_df_time_points[ica_comp]
    traces_l = get_segments_old(data_exp, ['side', 'laser'], [1,0], n, frame_type='frame_tone', is_pca=True, pc=ic)
    traces_r = get_segments_old(data_exp, ['side', 'laser'],[-1,0], n, frame_type='frame_tone', is_pca=True, pc=ic)
    
    fig, axs = plt.subplots(1, 2, figsize=(15, 3))
    # side  
    plot_aver_se_old(traces_l, 'b',ghost_traces=False, axn=0)
    plot_aver_se_old(traces_r, 'r',ghost_traces=False, axn=0)

    # choice
    traces_l = get_segments_old(data_exp, ['choice', 'laser'], [1,0], n, frame_type='frame_tone', is_pca=True, pc=ic)
    traces_r = get_segments_old(data_exp, ['choice', 'laser'], [-1,0], n, frame_type='frame_tone', is_pca=True, pc=ic)
    traces_0 = get_segments_old(data_exp, ['choice', 'laser'], [0,0], n, frame_type='frame_tone', is_pca=True, pc=ic)
    plot_aver_se_old(traces_l, 'b',ghost_traces=False, axn=1)
    plot_aver_se_old(traces_r, 'r',ghost_traces=False, axn=1)
    plot_aver_se_old(traces_0, 'k',ghost_traces=False, axn=1)

    for ax_n in [0,1]:
        axs[ax_n].axvline(x=-200)
        axs[ax_n].axvline(x=950)

    plt.title(ica_comp)
    plt.xlabel('time')
    plt.ylabel('au')
    plt.show()

In [ ]:
# now for prev chocie
# go through the 1st n components
for pca_comp in columns:
    n=0
    pc = ica_df_time_points[pca_comp]
    fig, axs = plt.subplots(1, 3, figsize=(15, 3))

    # all sides, no laser
    params = ['side', 'prev_side', 'prev_choice', 'laser']
    traces_l_same = get_segments_old(data_exp, params, [1,1,1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
    traces_l_diff = get_segments_old(data_exp, params, [1,-1,-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
    traces_r_same = get_segments_old(data_exp, params,[-1,-1,-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
    traces_r_diff = get_segments_old(data_exp, params,[-1,1,1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
    
    plot_aver_se_old(traces_l_same, 'b',ghost_traces=False, axn=0)
    plot_aver_se_old(traces_l_diff, 'c',ghost_traces=False, axn=0)
    plot_aver_se_old(traces_r_same, 'r',ghost_traces=False, axn=0)
    plot_aver_se_old(traces_r_diff, 'm',ghost_traces=False, axn=0)

    # only left side, laser effects
    params = ['side', 'prev_side', 'prev_choice', 'laser']
    traces_l_same_nolas = get_segments_old(data_exp, params, [1,1,1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
    traces_l_diff_nolas = get_segments_old(data_exp, params, [1,-1,-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
    traces_l_same_las = get_segments_old(data_exp, params, [1,1,1,1], n, frame_type='frame_tone', is_pca=True, pc=pc)
    traces_l_diff_las = get_segments_old(data_exp, params, [1,-1,-1,1], n, frame_type='frame_tone', is_pca=True, pc=pc)
    
    plot_aver_se_old(traces_l_same_nolas, 'b',ghost_traces=False, axn=1)
    plot_aver_se_old(traces_l_diff_nolas, 'c',ghost_traces=False, axn=1)
    plot_aver_se_old(traces_l_same_las, 'b',linestyle=':',ghost_traces=False, axn=1)
    plot_aver_se_old(traces_l_diff_las, 'c',linestyle=':',ghost_traces=False, axn=1)

    # only left side, laser effects
    params = ['side', 'prev_choice', 'prev_side', 'laser']
    traces_r_same_nolas = get_segments_old(data_exp, params, [-1,-1,-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
    traces_r_diff_nolas = get_segments_old(data_exp, params, [-1,1,1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
    traces_r_same_las = get_segments_old(data_exp, params, [-1,-1,-1,-1], n, frame_type='frame_tone', is_pca=True, pc=pc)
    traces_r_diff_las = get_segments_old(data_exp, params, [-1,1,1,-1], n, frame_type='frame_tone', is_pca=True, pc=pc)
    
    plot_aver_se_old(traces_r_same_nolas, 'r',ghost_traces=False, axn=2)
    plot_aver_se_old(traces_r_diff_nolas, 'm',ghost_traces=False, axn=2)
    plot_aver_se_old(traces_r_same_las, 'r',linestyle=':',ghost_traces=False, axn=2)
    plot_aver_se_old(traces_r_diff_las, 'm',linestyle=':',ghost_traces=False, axn=2)

    for ax_n in [0,1,2]:
        axs[ax_n].axvline(x=-200)
        axs[ax_n].axvline(x=950)

    plt.title(pca_comp)
    plt.xlabel('time')
    plt.ylabel('au')
    plt.show()

In [ ]:
# go through the 1st n components
for pca_comp in columns:
    n=0
    pc = ica_df_time_points[pca_comp]
    fig, axs = plt.subplots(1, 3, figsize=(15, 3))

    # all sides, no laser
    params = ['side',  'laser']
    traces_l_same = get_segments_old(data_exp, params, [1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
    traces_l_diff = get_segments_old(data_exp, params, [1,1], n, frame_type='frame_tone', is_pca=True, pc=pc)
    traces_r_same = get_segments_old(data_exp, params,[-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
    traces_r_diff = get_segments_old(data_exp, params,[-1,-1], n, frame_type='frame_tone', is_pca=True, pc=pc)
    
    plot_aver_se_old(traces_l_same, 'b',ghost_traces=False, axn=0)
    plot_aver_se_old(traces_l_diff, 'b',linestyle=':',ghost_traces=False, axn=0)
    plot_aver_se_old(traces_r_same, 'r',ghost_traces=False, axn=0)
    plot_aver_se_old(traces_r_diff, 'r',linestyle=':',ghost_traces=False, axn=0)
    '''

    # only left side, laser effects
    params = ['side', 'laser']
    traces_l_same_nolas = get_segments_old(data_exp, params, [1,1,1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)

    traces_l_same_las = get_segments_old(data_exp, params, [1,1,1,1], n, frame_type='frame_tone', is_pca=True, pc=pc)

    
    plot_aver_se_old(traces_l_same_nolas, 'b',ghost_traces=False, axn=1)
    plot_aver_se_old(traces_l_diff_nolas, 'c',ghost_traces=False, axn=1)
    plot_aver_se_old(traces_l_same_las, 'b',linestyle=':',ghost_traces=False, axn=1)
    plot_aver_se_old(traces_l_diff_las, 'c',linestyle=':',ghost_traces=False, axn=1)

    # only left side, laser effects
    params = ['side', 'prev_choice', 'prev_side', 'laser']
    traces_r_same_nolas = get_segments_old(data_exp, params, [-1,-1,-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
    traces_r_diff_nolas = get_segments_old(data_exp, params, [-1,1,1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
    traces_r_same_las = get_segments_old(data_exp, params, [-1,-1,-1,-1], n, frame_type='frame_tone', is_pca=True, pc=pc)
    traces_r_diff_las = get_segments_old(data_exp, params, [-1,1,1,-1], n, frame_type='frame_tone', is_pca=True, pc=pc)
    
    plot_aver_se_old(traces_r_same_nolas, 'r',ghost_traces=False, axn=2)
    plot_aver_se_old(traces_r_diff_nolas, 'm',ghost_traces=False, axn=2)
    plot_aver_se_old(traces_r_same_las, 'r',linestyle=':',ghost_traces=False, axn=2)
    plot_aver_se_old(traces_r_diff_las, 'm',linestyle=':',ghost_traces=False, axn=2)
'''
    for ax_n in [0,1,2]:
        axs[ax_n].axvline(x=-200)
        axs[ax_n].axvline(x=950)

    plt.title(pca_comp)
    plt.xlabel('time')
    plt.ylabel('au')
    plt.show()

In [ ]:



    
pc = ica_df_time_points['IC3']
n=99
traces_l1 = get_segments_old(data_exp, ['side', 'laser'], [1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
traces_r1 = get_segments_old(data_exp, ['side', 'laser'],[-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
pc = ica_df_time_points['IC2']
traces_l2 = get_segments_old(data_exp, ['side', 'laser'], [1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
traces_r2 = get_segments_old(data_exp, ['side', 'laser'],[-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)

# t_windows = [ [0,5], [6,10], [11,15], [16,20]]
t_windows = [ ]
for t in np.arange(0,30,4):
    t_windows.append([t,t+3])
# setup the plot
fig, axs = plt.subplots(1, len(t_windows), figsize=(15, 2))
ax_n = 0

for w in t_windows:
    segms_l1 = []
    segms_l2 = []
    # plot all the left 
    for segment_n in np.arange(len(traces_l1)):
        segment_pc1 = traces_l1[segment_n]
        segment_pc2 = traces_l2[segment_n]
        av_pc1 = np.mean(segment_pc1[w[0]:w[1]])
        av_pc2 = np.mean(segment_pc2[w[0]:w[1]])
        segms_l1.append(av_pc1)
        segms_l2.append(av_pc2)
        axs[ax_n].plot(av_pc1,av_pc2, 'b*', markersize=3, alpha=0.2)
        
    # plot all the right
    segms_r1 = []
    segms_r2 = []
    for segment_n in np.arange(len(traces_r1)):
        segment_pc1 = traces_r1[segment_n]
        segment_pc2 = traces_r2[segment_n]
        av_pc1 = np.mean(segment_pc1[w[0]:w[1]])
        av_pc2 = np.mean(segment_pc2[w[0]:w[1]])
        segms_r1.append(av_pc1)
        segms_r2.append(av_pc2)
        axs[ax_n].plot(av_pc1,av_pc2, 'r*', markersize=3, alpha=0.2)
 #   axs[ax_n].set_title(f't window {str(w[0]*100)} to {str(w[1]*100)} ms')
   #axs[ax_n].xlabel('PC1')
  #  axs[ax_n].ylabel('PC2')
    av_segms_l1 = np.mean(segms_l1)
    av_segms_l2 = np.mean(segms_l2)
    av_segms_r1 = np.mean(segms_r1)
    av_segms_r2 = np.mean(segms_r2)
    axs[ax_n].plot(av_segms_l1,av_segms_l2, color='b', marker='o', markersize=12,markeredgewidth=2,fillstyle='none')
    axs[ax_n].plot(av_segms_r1,av_segms_r2, color='r', marker='o', markersize=12,markeredgewidth=2,fillstyle='none')
    ax_n = ax_n+1
    
plt.show()


In [ ]:
# Same but with prev choice
    
pc = ica_df_time_points['IC3']
n = 99
params = ['side', 'prev_reward', 'laser']
traces_pc1 = {
    'l_same': get_segments_old(data_exp, params, [1, 1, 0], n, frame_type='frame_tone', is_pca=True, pc=pc),
    'r_same': get_segments_old(data_exp, params, [-1, -1, 0], n, frame_type='frame_tone', is_pca=True, pc=pc),
    'l_diff': get_segments_old(data_exp, params, [1, -1, 0], n, frame_type='frame_tone', is_pca=True, pc=pc),
    'r_diff': get_segments_old(data_exp, params, [-1, 1, 0], n, frame_type='frame_tone', is_pca=True, pc=pc),
}

pc = ica_df_time_points['IC5']
traces_pc2 = {
    'l_same': get_segments_old(data_exp, params, [1, 1, 0], n, frame_type='frame_tone', is_pca=True, pc=pc),
    'r_same': get_segments_old(data_exp, params, [-1, -1, 0], n, frame_type='frame_tone', is_pca=True, pc=pc),
    'l_diff': get_segments_old(data_exp, params, [1, -1, 0], n, frame_type='frame_tone', is_pca=True, pc=pc),
    'r_diff': get_segments_old(data_exp, params, [-1, 1, 0], n, frame_type='frame_tone', is_pca=True, pc=pc),
}

marker_specs = {'l_same':'b','l_diff':'c','r_same':'r','r_diff':'m'}

# t_windows = [ [0,5], [6,10], [11,15], [16,20]]
t_windows = [[t, t + 1] for t in np.arange(0, 30, 5)]

# setup the plot
fig, axs = plt.subplots(1, len(t_windows), figsize=(15, 2))
ax_n = 0

for w in t_windows:
    for key in traces_pc1.keys():
        segms_pc1 = traces_pc1[key]
        segms_pc2 = traces_pc2[key]
        avs_pc1 = []
        avs_pc2 = []
    
        for segment_n in np.arange(len(segms_pc1)):
            segm_pc1 = traces_pc1[key][segment_n]
            segm_pc2 = traces_pc2[key][segment_n]
            av_pc1 = np.mean(segm_pc1[w[0]:w[1]])
            av_pc2 = np.mean(segm_pc2[w[0]:w[1]])
            avs_pc1.append(av_pc1)
            avs_pc2.append(av_pc2)
            color = marker_specs[key]
            style = '*'
            axs[ax_n].plot(av_pc1, av_pc2, color + style, markersize=3, alpha=0.2)
        av_segms_1 = np.mean(avs_pc1)
        av_segms_2 = np.mean(avs_pc2)
        axs[ax_n].plot(av_segms_1,av_segms_2, color=color, marker='o', markersize=12,markeredgewidth=2,fillstyle='none')

    ax_n += 1

plt.show()

In [ ]:

# Trajectories for 2 selected components
# For each mouse
condition_pairs = [ ['left_side_same_las', 'left_side_diff_las'],
                   ['right_side_same_las', 'right_side_diff_las'] ]


for id in mouse_ids:
    dates = data[id].keys()
    # and date
    for date in dates:
        print(f"::: Mouse  {id}, {date} :::")
        data_exp = data[id][date]
        traces = data_exp['ca']
        behav = data_exp['behav']
        pca_df_time_points = data[id][date]['ICA']
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))
        ax_n = 0
        for conditions in condition_pairs:
            print(conditions)
            params = param_val_keys[conditions[0]]['params']
            vals1 = param_val_keys[conditions[0]]['vals']
            vals2 = param_val_keys[conditions[1]]['vals']
    
            pc = pca_df_time_points['IC1']
            n=99
            span=[-5,25]
            traces_l_diff_x = get_segments_old(data_exp, params, vals1, n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
            traces_r_diff_x = get_segments_old(data_exp, params, vals2, n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
            pc = pca_df_time_points['IC3']
            traces_l_diff_y = get_segments_old(data_exp, params, vals1, n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
            traces_r_diff_y = get_segments_old(data_exp, params, vals2, n, frame_type='frame_tone', is_pca=True, pc=pc, span=span)
            

            colors = [param_val_keys[conditions[0]]['color'], param_val_keys[conditions[1]]['color']]
            for tr_n in np.arange(len(traces_l_diff_x)):
                axs[ax_n].plot(traces_l_diff_x[tr_n], -traces_l_diff_y[tr_n], color=colors[0], alpha=0.2)
                axs[ax_n].plot(traces_l_diff_x[tr_n][0], -traces_l_diff_y[tr_n][0], color=colors[0], marker='o', alpha=0.5)
                axs[ax_n].plot(traces_l_diff_x[tr_n][-1], -traces_l_diff_y[tr_n][-1], color=colors[0], marker='D', alpha=0.5)
          #  ax_n += 1
            for trace_n in np.arange(len(traces_r_diff_x)):    
                axs[ax_n].plot(traces_r_diff_x[trace_n], -traces_r_diff_y[trace_n], color=colors[1], alpha=0.2)
                axs[ax_n].plot(traces_r_diff_x[trace_n][0], -traces_r_diff_y[trace_n][0], color=colors[1], marker='o', alpha=0.5)
                axs[ax_n].plot(traces_r_diff_x[trace_n][-1], -traces_r_diff_y[trace_n][-1], color=colors[1], marker='D', alpha=0.5)

      
            
        ax_n += 1
        
        plt.show()
        

In [ ]:
# Same but with laser
    
pc = ica_df_time_points['IC4']
n = 99
params = ['side', 'prev_reward', 'laser']
traces_pc1 = {
    'l_same_nolas': get_segments_old(data_exp, params, [1, 1, 0], n, frame_type='frame_tone', is_pca=True, pc=pc),
    'l_diff_nolas': get_segments_old(data_exp, params, [1, -1, 0], n, frame_type='frame_tone', is_pca=True, pc=pc),
    'l_same_las': get_segments_old(data_exp, params, [1, 1, 1], n, frame_type='frame_tone', is_pca=True, pc=pc),
    'l_diff_las': get_segments_old(data_exp, params, [1, -1, 1], n, frame_type='frame_tone', is_pca=True, pc=pc),
}

pc = ica_df_time_points['IC7']
traces_pc2 = {
    'l_same_nolas': get_segments_old(data_exp, params, [1, 1, 0], n, frame_type='frame_tone', is_pca=True, pc=pc),
    'l_diff_nolas': get_segments_old(data_exp, params, [1, -1, 0], n, frame_type='frame_tone', is_pca=True, pc=pc),
    'l_same_las': get_segments_old(data_exp, params, [1, 1, 1], n, frame_type='frame_tone', is_pca=True, pc=pc),
    'l_diff_las': get_segments_old(data_exp, params, [1, -1, 1], n, frame_type='frame_tone', is_pca=True, pc=pc),
}

marker_specs = {'l_same_nolas': {'color':'b', 'marker':'o'},
                'l_diff_nolas':{'color':'c', 'marker':'o'},
                'l_same_las': {'color':'b', 'marker':'D'},
                'l_diff_las': {'color':'c', 'marker':'D'}}

# t_windows = [ [0,5], [6,10], [11,15], [16,20]]
t_windows = [[t, t + 1] for t in np.arange(0, 30, 5)]

# setup the plot
fig, axs = plt.subplots(1, len(t_windows), figsize=(15, 2))
ax_n = 0

for w in t_windows:
    for key in traces_pc1.keys():
        segms_pc1 = traces_pc1[key]
        segms_pc2 = traces_pc2[key]
        avs_pc1 = []
        avs_pc2 = []
        color = marker_specs[key]['color']
        marker = marker_specs[key]['marker']
    
        for segment_n in np.arange(len(segms_pc1)):
            segm_pc1 = traces_pc1[key][segment_n]
            segm_pc2 = traces_pc2[key][segment_n]
            av_pc1 = np.mean(segm_pc1[w[0]:w[1]])
            av_pc2 = np.mean(segm_pc2[w[0]:w[1]])
            avs_pc1.append(av_pc1)
            avs_pc2.append(av_pc2)
            
            axs[ax_n].plot(av_pc1, av_pc2, color=color, marker=marker, markersize=3, alpha=0.2)
        av_segms_1 = np.mean(avs_pc1)
        av_segms_2 = np.mean(avs_pc2)
        axs[ax_n].plot(av_segms_1,av_segms_2, color=color, marker=marker, markersize=12,markeredgewidth=2,fillstyle='none')

    ax_n += 1

plt.show()


In [ ]:

# Same but with prev choice

    
pc = ica_df_time_points['IC3']
n=99
params = ['side', 'prev_reward', 'laser']
traces_l_same_1 = get_segments_old(data_exp, params, [1,1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
traces_r_same_1 = get_segments_old(data_exp, params,[-1,-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
traces_l_diff_1 = get_segments_old(data_exp, params, [1,-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
traces_r_diff_1 = get_segments_old(data_exp, params,[-1,1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
pc = ica_df_time_points['IC2']
traces_l_same_2 = get_segments_old(data_exp, params, [1,1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
traces_r_same_2 = get_segments_old(data_exp, params,[-1,-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
traces_l_diff_2 = get_segments_old(data_exp, params, [1,-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
traces_r_diff_2 = get_segments_old(data_exp, params,[-1,1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)

# t_windows = [ [0,5], [6,10], [11,15], [16,20]]
titles = [0, 1150, 2000]
t_windows = [ ]
window_width = 10
for t in np.arange(0,30,window_width):
    t_windows.append([t,t+window_width-1])
t_windows = [ [5,10], [15,20], [25,30] ]
# setup the plot
fig, axs = plt.subplots(1, len(t_windows), figsize=(15, 4))
ax_n = 0

for w in t_windows:
    # plot all the left same
    segms_l_same_1 = []
    segms_l_same_2 = []
    for segment_n in np.arange(len(traces_l_same_1)):
        segment_pc1 = traces_l_same_1[segment_n]
        segment_pc2 = traces_l_same_2[segment_n]
        av_pc1 = np.mean(segment_pc1[w[0]:w[1]])
        av_pc2 = np.mean(segment_pc2[w[0]:w[1]])
        segms_l_same_1.append(av_pc1)
        segms_l_same_2.append(av_pc2)
        axs[ax_n].plot(av_pc1, av_pc2, 'b*', markersize=3, alpha=0.3)

    # plot all the left diff
    segms_l_diff_1 = []
    segms_l_diff_2 = []
    for segment_n in np.arange(len(traces_l_diff_1)):
        segment_pc1 = traces_l_diff_1[segment_n]
        segment_pc2 = traces_l_diff_2[segment_n]
        av_pc1 = np.mean(segment_pc1[w[0]:w[1]])
        av_pc2 = np.mean(segment_pc2[w[0]:w[1]])
        segms_l_diff_1.append(av_pc1)
        segms_l_diff_2.append(av_pc2)
        axs[ax_n].plot(av_pc1, av_pc2, 'c*', markersize=3, alpha=0.3)

    # plot all the right same
    segms_r_same_1 = []
    segms_r_same_2 = []
    for segment_n in np.arange(len(traces_r_same_1)):
        segment_pc1 = traces_r_same_1[segment_n]
        segment_pc2 = traces_r_same_2[segment_n]
        av_pc1 = np.mean(segment_pc1[w[0]:w[1]])
        av_pc2 = np.mean(segment_pc2[w[0]:w[1]])
        segms_r_same_1.append(av_pc1)
        segms_r_same_2.append(av_pc2)
        axs[ax_n].plot(av_pc1, av_pc2, 'r*', markersize=3, alpha=0.3)

    # plot all the right diff
    segms_r_diff_1 = []
    segms_r_diff_2 = []
    for segment_n in np.arange(len(traces_r_diff_1)):
        segment_pc1 = traces_r_diff_1[segment_n]
        segment_pc2 = traces_r_diff_2[segment_n]
        av_pc1 = np.mean(segment_pc1[w[0]:w[1]])
        av_pc2 = np.mean(segment_pc2[w[0]:w[1]])
        segms_r_diff_1.append(av_pc1)
        segms_r_diff_2.append(av_pc2)
        axs[ax_n].plot(av_pc1, av_pc2, 'm*', markersize=3, alpha=0.3)

    av_segms_l_same_1 = np.mean(segms_l_same_1)
    av_segms_l_same_2 = np.mean(segms_l_same_2)
    av_segms_l_diff_1 = np.mean(segms_l_diff_1)
    av_segms_l_diff_2 = np.mean(segms_l_diff_2)
    av_segms_r_same_1 = np.mean(segms_r_same_1)
    av_segms_r_same_2 = np.mean(segms_r_same_2)
    av_segms_r_diff_1 = np.mean(segms_r_diff_1)
    av_segms_r_diff_2 = np.mean(segms_r_diff_2)

    axs[ax_n].plot(av_segms_l_same_1, av_segms_l_same_2, color='b', marker='o', markersize=12, markeredgewidth=2, fillstyle='none')
    axs[ax_n].plot(av_segms_l_diff_1, av_segms_l_diff_2, color='c', marker='o', markersize=12, markeredgewidth=2, fillstyle='none')
    axs[ax_n].plot(av_segms_r_same_1, av_segms_r_same_2, color='r', marker='o', markersize=12, markeredgewidth=2, fillstyle='none')
    axs[ax_n].plot(av_segms_r_diff_1, av_segms_r_diff_2, color='m', marker='o', markersize=12, markeredgewidth=2, fillstyle='none')
    axs[ax_n].set_title(f'{titles[ax_n]} ms')
    axs[ax_n].set_ylabel(f'IC2')
    axs[ax_n].set_xlabel(f'IC3')
    ax_n += 1

plt.show()


In [ ]:



    
pc = ica_df_time_points['IC1']
n=99
traces_l1 = get_segments_old(data_exp, ['side', 'laser'], [-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
traces_r1 = get_segments_old(data_exp, ['side', 'laser'],[-1,-1], n, frame_type='frame_tone', is_pca=True, pc=pc)
pc = ica_df_time_points['IC2']
traces_l2 = get_segments_old(data_exp, ['side', 'laser'], [-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
traces_r2 = get_segments_old(data_exp, ['side', 'laser'],[-1,-1], n, frame_type='frame_tone', is_pca=True, pc=pc)

# t_windows = [ [0,5], [6,10], [11,15], [16,20]]
t_windows = [ ]
for t in np.arange(6,27,4):
    t_windows.append([t,t+1])
# setup the plot
fig, axs = plt.subplots(1, len(t_windows), figsize=(15, 2))
ax_n = 0


for w in t_windows:
    # plot all the left 
    segms_l1 = []
    segms_l2 = []
    for segment_n in np.arange(len(traces_l1)):
        segment_pc1 = traces_l1[segment_n]
        segment_pc2 = traces_l2[segment_n]
        av_pc1 = np.mean(segment_pc1[w[0]:w[1]])
        av_pc2 = np.mean(segment_pc2[w[0]:w[1]])
        segms_l1.append(av_pc1)
        segms_l2.append(av_pc2)
        axs[ax_n].plot(av_pc1,av_pc2, 'b*', markersize=3, alpha=0.2)
        
    # plot all the right
    segms_r1 = []
    segms_r2 = []
    for segment_n in np.arange(len(traces_r1)):
        segment_pc1 = traces_r1[segment_n]
        segment_pc2 = traces_r2[segment_n]
        av_pc1 = np.mean(segment_pc1[w[0]:w[1]])
        av_pc2 = np.mean(segment_pc2[w[0]:w[1]])
        segms_r1.append(av_pc1)
        segms_r2.append(av_pc2)
        axs[ax_n].plot(av_pc1,av_pc2, 'c*', markersize=3, alpha=0.2)
 #   axs[ax_n].set_title(f't window {str(w[0]*100)} to {str(w[1]*100)} ms')
   #axs[ax_n].xlabel('PC1')
  #  axs[ax_n].ylabel('PC2')
    av_segms_l1 = np.mean(segms_l1)
    av_segms_l2 = np.mean(segms_l2)
    av_segms_r1 = np.mean(segms_r1)
    av_segms_r2 = np.mean(segms_r2)
    axs[ax_n].plot(av_segms_l1,av_segms_l2, color='blue', marker='o', markersize=12,markeredgewidth=2,fillstyle='none')
    axs[ax_n].plot(av_segms_r1,av_segms_r2, color='c', marker='o', markersize=12,fillstyle='none')
    ax_n = ax_n+1
    
plt.show()


In [ ]:



    
pc = ica_df_time_points['IC3']
n=99
traces_l1 = get_segments_old(data_exp, ['side', 'laser'], [1,0], n, frame_type='frame_tone', is_pca=True, pc=pc, span=[3,50])
traces_r1 = get_segments_old(data_exp, ['side', 'laser'],[-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc, span=[3,50])
pc = ica_df_time_points['IC2']
traces_l2 = get_segments_old(data_exp, ['side', 'laser'], [1,0], n, frame_type='frame_tone', is_pca=True, pc=pc, span=[3,50])
traces_r2 = get_segments_old(data_exp, ['side', 'laser'],[-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc, span=[3,50])

# t_windows = [ [0,5], [6,10], [11,15], [16,20]]
t_windows = [ ]
for t in np.arange(0,50,6):
    t_windows.append([t,t+1])
# setup the plot
fig, axs = plt.subplots(1, len(t_windows), figsize=(15, 2))
ax_n = 0


for w in t_windows:
    # plot all the left 
    segms_l1 = []
    segms_l2 = []
    for segment_n in np.arange(len(traces_l1)):
        segment_pc1 = traces_l1[segment_n]
        segment_pc2 = traces_l2[segment_n]
        av_pc1 = np.mean(segment_pc1[w[0]:w[1]])
        av_pc2 = np.mean(segment_pc2[w[0]:w[1]])
        segms_l1.append(av_pc1)
        segms_l2.append(av_pc2)
        axs[ax_n].plot(av_pc1,av_pc2, 'b*', markersize=3, alpha=0.2)
        
    # plot all the right
    segms_r1 = []
    segms_r2 = []
    for segment_n in np.arange(len(traces_r1)):
        segment_pc1 = traces_r1[segment_n]
        segment_pc2 = traces_r2[segment_n]
        av_pc1 = np.mean(segment_pc1[w[0]:w[1]])
        av_pc2 = np.mean(segment_pc2[w[0]:w[1]])
        segms_r1.append(av_pc1)
        segms_r2.append(av_pc2)
        axs[ax_n].plot(av_pc1,av_pc2, 'r*', markersize=3, alpha=0.2)
 #   axs[ax_n].set_title(f't window {str(w[0]*100)} to {str(w[1]*100)} ms')
   #axs[ax_n].xlabel('PC1')
  #  axs[ax_n].ylabel('PC2')
    av_segms_l1 = np.mean(segms_l1)
    av_segms_l2 = np.mean(segms_l2)
    av_segms_r1 = np.mean(segms_r1)
    av_segms_r2 = np.mean(segms_r2)
    axs[ax_n].plot(av_segms_l1,av_segms_l2, color='blue', marker='o', markersize=12,fillstyle='none')
    axs[ax_n].plot(av_segms_r1,av_segms_r2, color='r', marker='o', markersize=12,fillstyle='none')
    axs[ax_n].set_title(f'{w[0]*100} - {w[1]*100} ms', fontsize=8)
    ax_n = ax_n+1
    
plt.show()


In [ ]:



    
pc = ica_df_time_points['IC1']
n=99
traces_l1 = get_segments_old(data_exp, ['side', 'laser'], [-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
traces_r1 = get_segments_old(data_exp, ['side', 'laser'],[-1,-1], n, frame_type='frame_tone', is_pca=True, pc=pc)
pc = ica_df_time_points['IC2']
traces_l2 = get_segments_old(data_exp, ['side', 'laser'], [-1,0], n, frame_type='frame_tone', is_pca=True, pc=pc)
traces_r2 = get_segments_old(data_exp, ['side', 'laser'],[-1,-1], n, frame_type='frame_tone', is_pca=True, pc=pc)

# t_windows = [ [0,5], [6,10], [11,15], [16,20]]
t_windows = [ ]
for t in np.arange(4,35,4):
    t_windows.append([t,t+1])
# setup the plot
fig, axs = plt.subplots(1, len(t_windows), figsize=(15, 2))
ax_n = 0


for w in t_windows:
    # plot all the left 
    for segment_n in np.arange(len(traces_l1)):
        segment_pc1 = traces_l1[segment_n]
        segment_pc2 = traces_l2[segment_n]
        av_pc1 = np.mean(segment_pc1[w[0]:w[1]])
        av_pc2 = np.mean(segment_pc2[w[0]:w[1]])
        axs[ax_n].plot(av_pc1,av_pc2, 'b*')
        
    # plot all the right
    for segment_n in np.arange(len(traces_r1)):
        segment_pc1 = traces_r1[segment_n]
        segment_pc2 = traces_r2[segment_n]
        av_pc1 = np.mean(segment_pc1[w[0]:w[1]])
        av_pc2 = np.mean(segment_pc2[w[0]:w[1]])
        axs[ax_n].plot(av_pc1,av_pc2, 'c*')
 #   axs[ax_n].set_title(f't window {str(w[0]*100)} to {str(w[1]*100)} ms')
   #axs[ax_n].xlabel('PC1')
  #  axs[ax_n].ylabel('PC2')
    
    ax_n = ax_n+1
    
plt.show()


In [ ]:

# left same vs different

    
pc = ica_df_time_points['IC3']
n=99
params = ['side', 'prev_reward']
traces_l1 = get_segments_old(data_exp, params, [1,1], n, frame_type='frame_tone', is_pca=True, pc=pc)
traces_r1 = get_segments_old(data_exp, params, [1,-1], n, frame_type='frame_tone', is_pca=True, pc=pc)
pc = ica_df_time_points['IC6']
traces_l2 = get_segments_old(data_exp, params, [1,1], n, frame_type='frame_tone', is_pca=True, pc=pc)
traces_r2 = get_segments_old(data_exp, params, [1,-1], n, frame_type='frame_tone', is_pca=True, pc=pc)

t_windows = [ [0,5], [6,9], [10,13], [14,17], [18,21], [22,25], [26,29]]
# setup the plot
fig, axs = plt.subplots(1, 7, figsize=(15, 3))
ax_n = 0


for w in t_windows:
    # plot all the left 
    for segment_n in np.arange(len(traces_l1)):
        segment_pc1 = traces_l1[segment_n]
        segment_pc2 = traces_l2[segment_n]
        av_pc1 = np.mean(segment_pc1[w[0]:w[1]])
        av_pc2 = np.mean(segment_pc2[w[0]:w[1]])
        axs[ax_n].plot(av_pc1,av_pc2, 'b*')
        
    # plot all the right
    for segment_n in np.arange(len(traces_r1)):
        segment_pc1 = traces_r1[segment_n]
        segment_pc2 = traces_r2[segment_n]
        av_pc1 = np.mean(segment_pc1[w[0]:w[1]])
        av_pc2 = np.mean(segment_pc2[w[0]:w[1]])
        axs[ax_n].plot(av_pc1,av_pc2, 'c*')
 #   axs[ax_n].set_title(f't window {str(w[0]*100)} to {str(w[1]*100)} ms')
   #axs[ax_n].xlabel('PC1')
  #  axs[ax_n].ylabel('PC2')
    
    ax_n = ax_n+1
    
plt.show()


## Trajectories ICA!!!!!!!!!!!!

In [ ]:
cross_pcs ={'os85a': [13, 15],
 'os85b': [5, 16],
 'os86a': [16, 11],
 'os86b': [5, 7],
 'os87': [8, 14],
 'os88': [15, 1],
 'os89': [5, 1]}
# Trajectories for 2 selected components
# For each mouse
condition_pairs = [ ['left_side_same', 'left_side_diff'],
                   ['right_side_same', 'right_side_diff'] ]
sides_code = {'l': 1, 'r': -1, 1: 'l', -1: 'r', 'same':1, 'diff':-1}
#mice_ids = ['os85a',  'os86a',  'os87', 'os88', 'os89']

#for try1 in np.arange(1,17):
#    print(try1)
  #  for try2 in np.arange(1,17):
comps_ns = [12,5,2,7]
for id in mouse_ids[:1]:
   # print(f"::: Mouse  {id} :::")
    data_exp = data[id]
    behav = data_exp['behav']
    pcas = data[id]['ICA']
    frame_ts = data_exp['frame_ts']
    ax_n = 0
    comp_ns = [try1, try2] # cross_pcs[id]
  #  pc1 = pcas['PC' + str(comp_ns[0])]; pc2 = pcas['PC' + str(comp_ns[1])]
    comps_all = [pcas['IC' + str(n)] for n in comps_ns]
    params = ['prev_side', 'side', 'missed', 'laser']
    span = [0, 4000]

    distances_traveled = {'same': {'l': [], 'r': []}, 'diff': {'l': [], 'r': []}}
    for sameness in ['same', 'diff']:
        for side in ['l', 'r']:
            side_val = sides_code[side]
            modif = sides_sode[sameness]
            peak_ts = []; max_vals = []
            # same left 
            segms_all = []
            for each_comp in comps_all:
                segms1, segm_ts1 = get_segments(behav, each_comp, frame_ts, params, [modif*side_val,side_val,0,0], span=span)
                segms_all.append(segms1)
            #segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [modif*side_val,side_val,side_val,0,0], span=span)
            # each segment has to start from 0 and end at the peak
            for s_n in np.arange(len(segms1)):
                # Calculate differences between consecutive points
                dx = np.diff(segms_all[0][s_n])
                dy = np.diff(segms_all[1][s_n])
                dz = np.diff(segms_all[2][s_n])
                dw = np.diff(segms_all[3][s_n])


                # Calculate the Euclidean distance in 4D between consecutive points
                distances = np.sqrt(dx**2 + dy**2 + dz**2 + dw**2)

                # Sum the distances to get the total length of the trajectory in 4D
                total_length = np.sum(distances)

                distances_traveled[sameness][side].append(total_length)

    data[id]['travels'] = distances_traveled
    print(np.mean(distances_traveled['same']['l']))
    print(np.mean(distances_traveled['diff']['l']))
    stat, p_value_l= ttest_ind(distances_traveled['same']['l'], distances_traveled['diff']['l'])
    
    
    stat, p_value = ttest_ind(distances_traveled['same']['r'], distances_traveled['diff']['r'])

    
    print(f"Statistic L: {stat}, P-value: {p_value_l}")
    print(f"Statistic R: {stat}, P-value: {p_value}")


In [ ]:
plt.boxplot([distances_traveled['same']['l'], distances_traveled['diff']['l'], distances_traveled['same']['r'], distances_traveled['diff']['r']])
plt.show()

In [ ]:
plt.boxplot([distances_traveled['same']['l'], distances_traveled['diff']['l'], distances_traveled['same']['r'], distances_traveled['diff']['r']])
plt.show()

In [ ]:
data[id]['ICA'].keys()

In [ ]:
cross_pcs = {'os85a': [4,6], 'os85b': [3,5], 'os86a': [7,12], 'os86b': [5,3], 
                     'os87': [8,14], 'os88': [2,1], 'os89': [12,5]}


cross_pcs = {'os85a': [13, 6],
 'os85b': [3, 15],
 'os86a': [16, 11],
 'os86b': [12, 7],
 'os87': [3, 5],
 'os88': [7, 8],
 'os89': [5, 12]}

'''
cross_pcs ={'os85a': [13, 15],
 'os85b': [5, 16],
 'os86a': [16, 11],
 'os86b': [5, 7],
 'os87': [8, 14],
 'os88': [15, 1],
 'os89': [5, 1]}
 '''
# demure
cross_pcs = {'os85a': [4,15],
 'os85b': [6, 11],
 'os86a': [4, 16],
 'os86b': [13, 5],
 'os87': [8, 1],
 'os88': [1, 13],
 'os89': [13, 14]}
# Trajectories for 2 selected components
# For each mouse
condition_pairs = [ ['left_side_same', 'left_side_diff'],
                   ['right_side_same', 'right_side_diff'] ]
all_peak_ts_same = []
all_peak_ts_diff = []
max_comp_mice = {}

import itertools

# Generate all combinations of 2 different numbers between 1 and 16 (inclusive)
combinations = list(itertools.combinations(range(1, 17), 2))

t_to_peak_mice = {} 
max_distances_mice = {}
for id in mouse_ids:
    print(f"::: Mouse  {id} :::")
    mouse_peak_ts_same = []
    mouse_peak_ts_diff = []
    
    data_exp = data[id]
    behav = data_exp['behav']
    pcas = data[id]['ICA']
    frame_ts = data_exp['frame_ts']
 #   fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    ax_n = 0
    comp_ns = cross_pcs[id]
    pc1 = pcas['IC' + str(comp_ns[0])]; pc2 = pcas['IC' + str(comp_ns[1])]
    #pc1 = pcas['PC1']; pc2 = pcas['PC2']
    params = ['prev_side', 'side',  'missed',  'laser']
    span = [0, 4000]
    max_distances = {'same': {'l': [], 'r': []}, 'diff': {'l': [], 'r': []}}
    peak_ts = {'same': {'l': [], 'r': []}, 'diff': {'l': [], 'r': []}}
    for sameness in ['same', 'diff']:
        for side in ['l', 'r']:
            side_val = sides_code[side]
            modif = sides_sode[sameness]
            max_ts = []; max_vals = []
            # same left 
            segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [modif*side_val,side_val,0,0], span=span)
            segms2, segm_ts1 = get_segments(behav, pc2, frame_ts, params, [modif*side_val,side_val,0,0], span=span)
            #segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [modif*side_val,side_val,side_val,0,0], span=span)
            # each segment has to start from 0 and end at the peak
            for s_n in np.arange(len(segms1)):
                segm1 = np.array(segms1[s_n]); segm2 = np.array(segms2[s_n])
                ref_x, ref_y = segm1[0], segm2[0]
                distance = np.sqrt((segm1 - ref_x)**2 + (segm2 - ref_y)**2).tolist()
                max_t = np.argmax(distance)
                max_distance = distance[max_t]
                max_ts.append(max_t)
                max_vals.append(max_distance)
            max_distances[sameness][side] = max_vals
            peak_ts[sameness][side] = max_ts
    
    
    stat, p_val = stats.ttest_ind(max_distances['same']['l']+max_distances['same']['r'],
                                  max_distances['diff']['l']+max_distances['diff']['r']) 
    stat, p_val_t = stats.ttest_ind(peak_ts['same']['l']+peak_ts['same']['r'],
                                  peak_ts['diff']['l']+peak_ts['diff']['r']) 
    '''
       # stat, p_val = stats.ttest_ind(peak_ts['same'][side], peak_ts['diff'][side]) 
    p_thr = 0.05
    if (p_val < p_thr) & (p_val_t < p_thr):
        print(combo)
        print('dist: ', 
              round(np.mean(max_distances['same']['l']+max_distances['same']['r']),2), 
              round(np.mean(max_distances['diff']['l']+max_distances['diff']['r']),2), 
              p_val)
        print('time: ', 
              round(np.mean(peak_ts['same']['l']+peak_ts['same']['r']),2), 
              round(np.mean(peak_ts['diff']['l']+peak_ts['diff']['r']),2), 
              p_val_t)
              '''

    data[id]['max_distances'] = max_distances
    data[id]['peak_ts'] = peak_ts
    t_to_peak_mice[id] = peak_ts
    max_distances_mice[id] = max_distances

            
           # plt.show()
print('done')

In [ ]:
max_comp_mice = {}
         
for id in mouse_ids:
    print(f"::: Mouse  {id} :::")
    mouse_peak_ts_same = []
    mouse_peak_ts_diff = []
    
    data_exp = data[id]
    behav = data_exp['behav']
    pcas = data[id]['ICA']
    frame_ts = data_exp['frame_ts']
 #   fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    ax_n = 0
    comp_ns = cross_pcs[id]
    pc1 = pcas['IC' + str(comp_ns[0])]; pc2 = pcas['IC' + str(comp_ns[1])]
    #pc1 = pcas['PC1']; pc2 = pcas['PC2']
    params = ['prev_side', 'side',   'missed',  'laser']
    span = [0, 4000]
    max_distances = {'same': {'l': [], 'r': []}, 'diff': {'l': [], 'r': []}}
    peak_ts = {'same': {'l': [], 'r': []}, 'diff': {'l': [], 'r': []}}
    for sameness in ['same', 'diff']:
        for side in ['l', 'r']:
            side_val = sides_code[side]
            modif = sides_sode[sameness]
            max_ts = []; max_vals = []
            # same left 
            segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [modif*side_val,side_val,0,side_val], span=span)
            segms2, segm_ts1 = get_segments(behav, pc2, frame_ts, params, [modif*side_val,side_val,0,side_val], span=span)
            #segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [modif*side_val,side_val,side_val,0,0], span=span)
            # each segment has to start from 0 and end at the peak
            for s_n in np.arange(len(segms1)):
                segm1 = np.array(segms1[s_n]); segm2 = np.array(segms2[s_n])
                ref_x, ref_y = segm1[0], segm2[0]
                distance = np.sqrt((segm1 - ref_x)**2 + (segm2 - ref_y)**2).tolist()
                max_t = np.argmax(distance)
                max_distance = distance[max_t]
                max_ts.append(max_t)
                max_vals.append(max_distance)
            max_distances[sameness][side] = max_vals
            peak_ts[sameness][side] = max_ts
    
    
    stat, p_val = stats.ttest_ind(max_distances['same']['l']+max_distances['same']['r'],
                                  max_distances['diff']['l']+max_distances['diff']['r']) 
    stat, p_val_t = stats.ttest_ind(peak_ts['same']['l']+peak_ts['same']['r'],
                                  peak_ts['diff']['l']+peak_ts['diff']['r']) 
    '''
       # stat, p_val = stats.ttest_ind(peak_ts['same'][side], peak_ts['diff'][side]) 
    p_thr = 0.05
    if (p_val < p_thr) & (p_val_t < p_thr):
        print(combo)
        print('dist: ', 
              round(np.mean(max_distances['same']['l']+max_distances['same']['r']),2), 
              round(np.mean(max_distances['diff']['l']+max_distances['diff']['r']),2), 
              p_val)
        print('time: ', 
              round(np.mean(peak_ts['same']['l']+peak_ts['same']['r']),2), 
              round(np.mean(peak_ts['diff']['l']+peak_ts['diff']['r']),2), 
              p_val_t)
              '''

    data[id]['max_distances']['same_l'] = max_distances['same']
    data[id]['max_distances']['diff_l'] = max_distances['diff']
    data[id]['peak_ts']['same_l'] = peak_ts['same']
    data[id]['peak_ts']['diff_l'] = peak_ts['diff']

    t_to_peak_mice[id]['same_l'] = peak_ts['same']
    t_to_peak_mice[id]['diff_l'] = peak_ts['diff']
    max_distances_mice[id]['same_l'] = max_distances['same']
    max_distances_mice[id]['diff_l'] = max_distances['diff']

            
           # plt.show()
print('done')

### Fold

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import mannwhitneyu, combine_pvalues

# Aggregate data for 'same' and 'diff'
stats_to_check = t_to_peak_mice
same_all = []
diff_all = []
median_same = []
median_diff = []
for id in mouse_ids:
    mouse_values = stats_to_check[id]
    same_all.extend(mouse_values['same']['r'])
    diff_all.extend(mouse_values['diff']['r'])

    # Compute median for each mouse
    median_same.append(np.median(mouse_values['same']['r']))
    median_diff.append(np.median(mouse_values['diff']['r']))


# Step 1: Perform Mann-Whitney U test for each mouse
p_values = []
for id in mouse_ids:
    mouse_values = stats_to_check[id]
    same = mouse_values['same']['r']
    diff = mouse_values['diff']['r']
    stat, p_value = mannwhitneyu(same, diff, alternative='two-sided')
    print(f'mannwhitneyu: {p_value}')
    p_values.append(p_value)



# Step 2: Combine p-values using Fisher's method
stat_combined, p_value_combined = combine_pvalues(p_values, method='fisher')
print(f'Combined p-value across all mice (Fisher\'s method): {p_value_combined}')

# Step 3: Plot the boxplot for 'same' and 'diff' conditions
fig, ax = plt.subplots(figsize=(8, 6))

# Create boxplots
box_labels = ['Same', 'Diff']
box = ax.boxplot([same_all, diff_all], labels=box_labels, patch_artist=True, notch=True)

# Customize colors
colors = ['green', 'purple']
for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)

# Add one line for each experiment (median values for same and diff)
for i in range(len(median_same)):
    ax.plot([1, 2], [median_same[i], median_diff[i]], color='black', linestyle='-', alpha=0.5)

# Add labels and title
ax.set_ylabel('Time to Peak')
ax.set_title('Comparison of Same vs Diff Conditions')

# Remove top and right spines for cleaner look
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Display plot
plt.tight_layout()
plt.show()


### Fold

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import mannwhitneyu, combine_pvalues

# Aggregate data for 'same' and 'diff'
stats_to_check = t_to_peak_mice
same_all = []
diff_all = []
median_same = []
median_diff = []
for id in mouse_ids:
    mouse_values = stats_to_check[id]
    same_all.extend(mouse_values['same']['l'])
    diff_all.extend(mouse_values['diff_l']['l'])

    # Compute median for each mouse
    median_same.append(np.median(mouse_values['same']['l']))
    median_diff.append(np.median(mouse_values['same_l']['l']))


# Step 1: Perform Mann-Whitney U test for each mouse
p_values = []
for id in mouse_ids:
    mouse_values = stats_to_check[id]
    same = mouse_values['same']['l']
    diff = mouse_values['same_l']['l']
    stat, p_value = mannwhitneyu(same, diff, alternative='two-sided')
    print(f'mannwhitneyu: {p_value}')
    p_values.append(p_value)



# Step 2: Combine p-values using Fisher's method
stat_combined, p_value_combined = combine_pvalues(p_values, method='fisher')
print(f'Combined p-value across all mice (Fisher\'s method): {p_value_combined}')

# Step 3: Plot the boxplot for 'same' and 'diff' conditions
fig, ax = plt.subplots(figsize=(8, 6))

# Create boxplots
box_labels = ['Same', 'Diff']
box = ax.boxplot([same_all, diff_all], labels=box_labels, patch_artist=True, notch=True)

# Customize colors
colors = ['green', 'purple']
for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)

# Add one line for each experiment (median values for same and diff)
for i in range(len(median_same)):
    ax.plot([1, 2], [median_same[i], median_diff[i]], color='black', linestyle='-', alpha=0.5)

# Add labels and title
ax.set_ylabel('Time to Peak')
ax.set_title('Comparison of Same vs Diff Conditions')

# Remove top and right spines for cleaner look
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Display plot
plt.tight_layout()
plt.show()


### My Playground

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import mannwhitneyu, combine_pvalues

# Aggregate data for 'same' and 'diff'
stats_to_check = max_distances_mice # max_distances_mice # t_to_peak_mice
cond1s = []
cond2s = []
cond3s = []
cond4s = []
median_cond1s = []
median_cond2s = []
p_values = []
side = 'l'
for id in mouse_ids[:-1]:
    mouse_values = stats_to_check[id]
    cond1 = mouse_values['same'][side] 
    cond2 = mouse_values['same_l'][side]
    cond1s.extend(cond1)
    cond2s.extend(cond2)
    cond3 = mouse_values['diff'][side] 
    cond4 = mouse_values['diff_l'][side]
    cond3s.extend(cond3)
    cond4s.extend(cond4)

    # Compute median for each mouse
    median_same.append(np.median(cond1))
    median_diff.append(np.median(cond2))
    # Perform one-tailed t-test (assuming condition1 > condition2)
    t_stat, p_value = stats.ttest_ind(cond3, cond4, alternative='greater')

    #stat, p_value = mannwhitneyu(cond1, cond2, alternative='two-sided')
  #  stat, p_value = stats.ttest_ind(cond1, cond2)
    print(f'mannwhitneyu: {p_value}')
    p_values.append(p_value)


# Step 1: Perform Mann-Whitney U test for each mouse
# Step 2: Combine p-values using Fisher's method
stat_combined, p_value_combined = combine_pvalues(p_values, method='fisher')
print(f'Combined p-value across all mice (Fisher\'s method): {p_value_combined}')
stat, p_value = mannwhitneyu(cond1s, cond2s, alternative='two-sided')
print(f'AAAmannwhitneyu: {p_value}')
# Step 3: Plot the boxplot for 'same' and 'diff' conditions
fig, axs = plt.subplots(1,2, figsize=(9, 6))

# Create boxplots
box_labels = ['Same', 'Same+stim', 'Diff', 'Diff+stim']
box = axs[0].boxplot([cond1s, cond2s, cond3s, cond4s], labels=box_labels, patch_artist=True, notch=True)

# Customize colors
colors = ['green', 'lime', 'purple', 'pink']
for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)

# Add one line for each experiment (median values for same and diff)
for i in range(len(median_cond1s)):
    ax.plot([1, 2], [median_cond1s[i], median_cond2s[i]], color='black', linestyle='-', alpha=0.5)


cond1s = []
cond2s = []
cond3s = []
cond4s = []
median_cond1s = []
median_cond2s = []
p_values = []
side = 'r'
for id in mouse_ids[:-1]:
    mouse_values = stats_to_check[id]
    cond1 = mouse_values['same'][side] 
    cond2 = mouse_values['same_l'][side]
    cond1s.extend(cond1)
    cond2s.extend(cond2)
    cond3 = mouse_values['diff'][side] 
    cond4 = mouse_values['diff_l'][side]
    cond3s.extend(cond3)
    cond4s.extend(cond4)

    # Compute median for each mouse
    median_same.append(np.median(cond1))
    median_diff.append(np.median(cond2))
    # Perform one-tailed t-test (assuming condition1 > condition2)
    t_stat, p_value = stats.ttest_ind(cond2, cond1, alternative='greater')

    #stat, p_value = mannwhitneyu(cond1, cond2, alternative='two-sided')
  #  stat, p_value = stats.ttest_ind(cond1, cond2)
    print(f'mannwhitneyu: {p_value}')
    p_values.append(p_value)


# Step 1: Perform Mann-Whitney U test for each mouse
# Step 2: Combine p-values using Fisher's method
stat_combined, p_value_combined = combine_pvalues(p_values, method='fisher')
print(f'Combined p-value across all mice (Fisher\'s method): {p_value_combined}')
stat, p_value = mannwhitneyu(cond1s, cond2s, alternative='two-sided')
print(f'AAAmannwhitneyu: {p_value}')

# Create boxplots
box_labels = ['Same', 'Same+stim', 'Diff', 'Diff+stim']
box = axs[1].boxplot([cond1s, cond2s, cond3s, cond4s], labels=box_labels, patch_artist=True, notch=True)

# Customize colors
colors = ['green', 'lime', 'purple', 'pink']
for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)

axs[0].set_title('Left laser', fontsize=16)
axs[1].set_title('Right laser', fontsize=16)

# Add labels and title
for ax_n in [0,1]:
    axs[ax_n].set_ylabel('Maximum value in IC space', fontsize=14)

    # Remove top and right spines for cleaner look
    axs[ax_n].spines['right'].set_visible(False)
    axs[ax_n].spines['top'].set_visible(False)

# Display plot
plt.tight_layout()
plt.savefig('D:\\papers_thesis\\figs\\' + 'laser_comp_summaryBB.png', dpi=300)
plt.show()


In [ ]:
### Laser

In [ ]:
#cross_pcs = {'os85a': [4,6], 'os85b': [3,5], 'os86a': [7,12], 'os86b': [5,3], 
  #                   'os87': [8,14], 'os88': [2,1], 'os89': [12,5]}

'''
cross_pcs = {'os85a': [13, 6],
 'os85b': [3, 15],
 'os86a': [16, 11],
 'os86b': [12, 7],
 'os87': [3, 5],
 'os88': [7, 8],
 'os89': [5, 12]}
 '''
# Trajectories for 2 selected components
# For each mouse
condition_pairs = [ ['left_side_same', 'left_side_diff'],
                   ['right_side_same', 'right_side_diff'] ]
all_peak_ts_same = []
all_peak_ts_diff = []
max_comp_mice_las = {}

            
for id in mouse_ids:
    mouse_peak_ts_same = []
    mouse_peak_ts_diff = []
#    print(f"::: Mouse  {id} :::")
    data_exp = data[id]
    behav = data_exp['behav']
    pcas = data[id]['PCA']
    frame_ts = data_exp['frame_ts']
 #   fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    ax_n = 0
    comp_ns = cross_pcs[id]
   # pc1 = pcas['IC' + str(comp_ns[0])]; pc2 = pcas['IC' + str(comp_ns[1])]
    pc1 = pcas['PC1']; pc2 = pcas['PC2']
    params = ['prev_side', 'side', 'missed', 'free', 'choice', 'laser']
    span = [0, 4000]
    # ==== LEFT ====
    peak_ts_same = []; dist_same = []
    dist_max_same = []; dist_max_diff = []
    max_comp_vals = {'same': [], 'diff': []}
    # same left 
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [1,1,0,0,1,1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [1,1,0,0,1,1], span=span)
    # each segment has to start from 0 and end at the peak
    
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
       # segm1 = [s-segm1[0] for s in segm1]
      #  segm2 = [s-segm2[0] for s in segm2]

        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
     #   time_furthest_away = time[max_distance_idx]     
        peak_t = max_distance_idx; # abs_segm.index(max(abs_segm))
        peak_ts_same.append(peak_t)
        dist_max_same.append(max(distances))
        #segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
     #   axs[ax_n].plot(segm1, segm2, 'b', alpha=0.5)
       # axs[ax_n].plot(segm1[0], segm2[0], 'b', marker='o', alpha=0.5)
    #    axs[ax_n].plot(segm1[-1], segm2[-1], 'b', marker='D', alpha=0.5)
        # Assuming segm1 and segm2 are lists or arrays of coordinates [x1, y1] and [x2, y2]
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_same.append(distance)
        
    max_comp_vals['same'].append(dist_same)
    # same left 
    peak_ts_diff = []; dist_diff = []
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [-1,1,0,0,1,1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [-1,1,0,0,1,1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
      #  segm1 = [s-segm1[0] for s in segm1]
      #  segm2 = [s-segm2[0] for s in segm2]
        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
      #  time_furthest_away = time[max_distance_idx]

        
        peak_t = max_distance_idx #abs_segm.index(max(abs_segm))
        peak_ts_diff.append(peak_t)
        dist_max_diff.append(max(distances))
        segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
      #  axs[ax_n].plot(segm1, segm2, 'c', alpha=0.5)
        #axs[ax_n].plot(segm1[0], segm2[0], 'c', marker='o', alpha=0.5)
     #   axs[ax_n].plot(segm1[-1], segm2[-1], 'c', marker='D', alpha=0.5)
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_diff.append(distance)
    
#    print(np.mean(peak_ts_same), np.mean(peak_ts_diff)) 
    left_info = [np.mean(peak_ts_same), np.mean(peak_ts_diff)]
    all_peak_ts_same += peak_ts_same
    all_peak_ts_diff += peak_ts_diff
    mouse_peak_ts_same += peak_ts_same
    mouse_peak_ts_diff += peak_ts_diff
    stat, p_val_l_t = stats.ttest_ind(peak_ts_same, peak_ts_diff)  

    
    data[id]['t_to_peak_L'] = {'same': peak_ts_same, 'diff': peak_ts_diff}
    max_vals_l = {'same': peak_ts_same, 'diff': peak_ts_diff}
    max_comp_vals['same'].append(dist_same)
    '''
    print(np.mean(dist_same), np.mean(dist_diff)) 
    stat, p_val = stats.ttest_ind(dist_same, dist_diff)    
    print('p for distances', p_val)
    '''
    # ==== RIGHT =====
    ax_n += 1
    peak_ts_same = []; dist_same = []
    # same left 
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [-1,-1,0,0,-1,-1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [-1,-1,0,0,-1,-1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
      #  segm1 = [s-segm1[-1] for s in segm1]
      #  segm2 = [s-segm2[-1] for s in segm2]

        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
     #   time_furthest_away = time[max_distance_idx]


        dist_max_same.append(max(distances))
        peak_t = max_distance_idx; # abs_segm.index(max(abs_segm))
        peak_ts_same.append(peak_t)
        segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
      #  axs[ax_n].plot(segm1, segm2, 'r', alpha=0.5)
      #  axs[ax_n].plot(segm1[0], segm2[0], 'r', marker='o', alpha=0.5)
       # axs[ax_n].plot(segm1[-1], segm2[-1], 'r', marker='D', alpha=0.5)
        # Assuming segm1 and segm2 are lists or arrays of coordinates [x1, y1] and [x2, y2]
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_same.append(distance)

    # same left 
    #ax_n += 1
    peak_ts_diff = []; dist_diff = []
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [1,-1,0,0,-1,-1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [1,-1,0,0,-1,-1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
     #   segm1 = [s-segm1[-1] for s in segm1]
      #  segm2 = [s-segm2[-1] for s in segm2]
        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
      #  time_furthest_away = time[max_distance_idx]

        
        peak_t = max_distance_idx #abs_segm.index(max(abs_segm))
        peak_ts_diff.append(peak_t)
        dist_max_diff.append(max(distances))
        segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
      #  axs[ax_n].plot(segm1, segm2, 'm', alpha=0.5)
      #  axs[ax_n].plot(segm1[0], segm2[0], 'm', marker='o', alpha=0.5)
       # axs[ax_n].plot(segm1[-1], segm2[-1], 'm', marker='D', alpha=0.5)
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_diff.append(distance)
    
    
    stat, p_val_r_t = stats.ttest_ind(peak_ts_same, peak_ts_diff)    
    
   # print(np.mean(peak_ts_same), np.mean(peak_ts_diff)) 
   # print('p for time to peak', p_val)
    
  #  print(np.mean(dist_max_same), np.mean(dist_max_diff)) 
    stat, p_val_m = stats.ttest_ind(dist_max_diff, dist_max_same)  
    p_thr = 0.01
    if (p_val_m < p_thr): 
        print(p_val_m)
   #     print(np.mean(peak_ts_same), np.mean(peak_ts_diff)) 
        print(np.mean(dist_max_same), np.mean(dist_max_diff)) 
  #  print('max value p', p_val)
    all_peak_ts_same += peak_ts_same
    all_peak_ts_diff += peak_ts_diff
    mouse_peak_ts_same += peak_ts_same
    mouse_peak_ts_diff += peak_ts_diff
    data[id]['t_to_peak_R'] = {'same': peak_ts_same, 'diff': peak_ts_diff}
    data[id]['t_to_peak'] = {'same': mouse_peak_ts_same, 'diff': mouse_peak_ts_diff}



    '''
    print(np.mean(dist_same), np.mean(dist_diff)) 
    stat, p_val = stats.ttest_ind(dist_same, dist_diff)    
    print('p for distances', p_val)
    '''
    max_comp_vals = {'same': dist_max_same, 'diff': dist_max_diff}
    max_comp_mice_las[id] = max_comp_vals

            
           # plt.show()
print('done')

In [ ]:
import numpy as np
from scipy.stats import wilcoxon, combine_pvalues
from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind
# Example data for multiple mice
t_to_peak_mice = max_comp_mice_las
# Step 1: Perform Mann-Whitney U test for each mouse
p_values = []
all_same = []; all_diff = []
for mouse, mouse_values in t_to_peak_mice.items():
    same = mouse_values['same']
    diff = mouse_values['diff']
    all_same += same; all_diff += diff
    # Perform Mann-Whitney U test
    print(f' mean same {round(np.mean(same),2)} std same {round(np.std(same),2)}')
    print(f' mean diff {round(np.mean(diff),2)} std diff {round(np.std(diff),2)}')
    stat, p_value = mannwhitneyu(same, diff, alternative='two-sided')
    stat, p_value = ttest_ind(same, diff) #, alternative='two-sided')
    p_values.append(p_value)
    print(f'{mouse}: Mann-Whitney U p-value = {p_value}')

# Step 2: Combine p-values using Fisher's method
stat_combined, p_value_combined = combine_pvalues(p_values, method='fisher')

# Step 3: Output the combined p-value
print(f'Combined p-value across all mice (Fisher\'s method): {p_value_combined}')
print(f' mean same {round(np.mean(all_same),2)} std same {round(np.std(all_same),2)}')
print(f' mean diff {round(np.mean(all_diff),2)} std diff {round(np.std(all_diff),2)}')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import mannwhitneyu, combine_pvalues

# Aggregate data for 'same' and 'diff'
same_all = []
diff_all = []
median_same = []
median_diff = []
for mouse_values in t_to_peak_mice.values():
    same_all.extend(mouse_values['same'])
    diff_all.extend(mouse_values['diff'])
    
    # Compute median for each mouse
    median_same.append(np.median(mouse_values['same']))
    median_diff.append(np.median(mouse_values['diff']))

# Aggregate data for 'same' and 'diff'
same_alll = []
diff_alll = []
median_samel = []
median_diffl = []
for mouse_values in max_comp_mice_las.values():
    same_alll.extend(mouse_values['same'])
    diff_alll.extend(mouse_values['diff'])
    
    # Compute median for each mouse
    median_samel.append(np.median(mouse_values['same']))
    median_diffl.append(np.median(mouse_values['diff']))

# Step 1: Perform Mann-Whitney U test for each mouse
p_values = []
for id in mouse_ids:
#for mouse, mouse_values in max_comp_mice.items():
    same = max_comp_mice[id]['diff']
    same_l = max_comp_mice_las[id]['diff']
    stat, p_value = mannwhitneyu(same, same_l, alternative='two-sided')
    print(f'mannwhitneyu: {p_value}')
    p_values.append(p_value)



# Step 2: Combine p-values using Fisher's method
stat_combined, p_value_combined = combine_pvalues(p_values, method='fisher')
print(f'Combined p-value across all mice (Fisher\'s method): {p_value_combined}')

# Step 3: Plot the boxplot for 'same' and 'diff' conditions
fig, ax = plt.subplots(figsize=(8, 6))

# Create boxplots
box_labels = ['Same', 'Diff']
box = ax.boxplot([same_all, same_alll], labels=box_labels, patch_artist=True, notch=True)

# Customize colors
colors = ['green', 'purple']
for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)

# Add one line for each experiment (median values for same and diff)
for i in range(len(median_same)):
    ax.plot([1, 2], [median_same[i], median_diff[i]], color='black', linestyle='-', alpha=0.5)

# Add labels and title
ax.set_ylabel('Time to Peak')
ax.set_title('Comparison of Same vs Diff Conditions')

# Remove top and right spines for cleaner look
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Display plot
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import mannwhitneyu, combine_pvalues

# Aggregate data for 'same' and 'diff'
same_all = []
diff_all = []
median_same = []
median_diff = []
same_alll = []
diff_alll = []
median_samel = []
median_diffl = []
for mouse_values in t_to_peak_mice.values():
    same_alll.extend(mouse_values['same'])
    diff_alll.extend(mouse_values['diff'])
    
    # Compute median for each mouse
    median_samel.append(np.median(mouse_values['same']))
    median_diffl.append(np.median(mouse_values['diff']))
for mouse_values in t_to_peak_mice_las.values():
    same_all.extend(mouse_values['same'])
    diff_all.extend(mouse_values['diff'])
    
    # Compute median for each mouse
    median_same.append(np.median(mouse_values['same']))
    median_diff.append(np.median(mouse_values['diff']))


# Step 3: Plot the boxplot for 'same' and 'diff' conditions
fig, ax = plt.subplots(figsize=(8, 6))

# Create boxplots
box_labels = ['Same', 'Same l',  'Diff','Diff l']
box = ax.boxplot([same_all, same_alll, diff_all, diff_alll], labels=box_labels, patch_artist=True, notch=True)

# Customize colors
colors = ['green', 'lime', 'purple', 'pink']
for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)

# Add one line for each experiment (median values for same and diff)
#for i in range(len(median_same)):
 #   ax.plot([1, 2, 3, 4], [median_same[i], median_diff[i]], color='black', linestyle='-', alpha=0.5)

# Add labels and title
ax.set_ylabel('Time to Peak')
ax.set_title('Comparison of Same vs Diff Conditions')

# Remove top and right spines for cleaner look
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Display plot
plt.tight_layout()
plt.show()


In [ ]:
## Unburden

In [ ]:
cross_pcs = {'os85a': [1,2], 'os85b': [1,3], 'os86a': [2,5], 'os86b': [1,2], 
                     'os87': [1,2], 'os88': [1,4], 'os89': [1,2]}
cross_pcs = {'os85a': [1,2], 'os85b': [1,3], 'os86a': [2,5], 'os86b': [1,2], 
                     'os87': [1,2], 'os88': [1,3], 'os89': [1,2]}
'''

# side
# 12 comps
cross_pcs = {'os85a': [3,2], 'os85b': [8,5], 'os86a': [1,4], 'os86b': [12,9], 
                     'os87': [7,6], 'os88': [1,8], 'os89': [7,8]}
                     '''
# 16 comps
cross_pcs = {'os85a': [13,4], 'os85b': [2,5], 'os86a': [2,3], 'os86b': [5,3], 
                     'os87': [8,14], 'os88': [2,1], 'os89': [12,5]}
                     
# laser
#cross_pcs = {'os85a': [12,2], 'os85b': [8,9], 'os86a': [1,10], 'os86b': [2,5], 
         #            'os87': [7,6], 'os88': [6,9], 'os89': [12,3]}
# ================ LASER NOW ===================
for id in mice_ids:
    print(f"::: Mouse  {id} :::")
    data_exp = data[id]
    behav = data_exp['behav']
    pcas = data[id]['PCA']
    pc1 = pcas['PC1']; pc2 = pcas['PC2']
    pc1 = pcas['PC' + str(comp_ns[0])]; pc2 = pcas['PC' + str(comp_ns[1])]
  #  pcas = data[id]['ICA']
  #  pc1 = pcas['IC' + str(comp_ns[0])]; pc2 = pcas['IC' + str(comp_ns[1])]
    frame_ts = data_exp['frame_ts']
    fig, axs = plt.subplots(1, 4, figsize=(15, 3))
    ax_n = 0
    comp_ns = cross_pcs[id]
    
    params = ['prev_side', 'side', 'missed', 'laser', 'choice']
    span = [0, 4000]
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # ==== LEFT ====
    peak_vals_same = []; dist_same = []
    # same left 
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [1,1,0,0,1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [1,1,0,0,1], span=span)
    table_means = {'side': [], 'laser': [], 'is_same': [], 'mean': []}
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
       # segm1 = [s-segm1[0] for s in segm1]
      #  segm2 = [s-segm2[0] for s in segm2]

        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
     #   time_furthest_away = time[max_distance_idx]     
        peak_t = max_distance_idx; # abs_segm.index(max(abs_segm))
        peak_vals_same.append(max(distances))
        #segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'b', alpha=0.5)
       # axs[ax_n].plot(segm1[0], segm2[0], 'b', marker='o', alpha=0.5)
        axs[ax_n].plot(segm1[-1], segm2[-1], 'b', marker='D', alpha=0.5)
        # Assuming segm1 and segm2 are lists or arrays of coordinates [x1, y1] and [x2, y2]
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_same.append(distance)

    # same left 
    peak_vals_diff = []; dist_diff = []
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [-1,1,0,0,1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [-1,1,0,0,1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
      #  segm1 = [s-segm1[0] for s in segm1]
      #  segm2 = [s-segm2[0] for s in segm2]
        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
      #  time_furthest_away = time[max_distance_idx]

        
        peak_t = max_distance_idx #abs_segm.index(max(abs_segm))
        peak_vals_diff.append(max(distances))
        segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'c', alpha=0.5)
        #axs[ax_n].plot(segm1[0], segm2[0], 'c', marker='o', alpha=0.5)
        axs[ax_n].plot(segm1[-1], segm2[-1], 'c', marker='D', alpha=0.5)
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_diff.append(distance)
    
   # print(np.mean(peak_vals_same), np.mean(peak_vals_diff)) 
    stat, p_val = stats.ttest_ind(peak_vals_same, peak_vals_diff)    
    print('p for max val L', p_val)
    table_means['side'].append(1)
    table_means['laser'].append(False)
    table_means['is_same'].append(True)
    table_means['mean'].append(np.mean(peak_vals_same))
    table_means['side'].append(1)
    table_means['laser'].append(False)
    table_means['is_same'].append(False)
    table_means['mean'].append(np.mean(peak_vals_diff))
    
    '''
    print(np.mean(dist_same), np.mean(dist_diff)) 
    stat, p_val = stats.ttest_ind(dist_same, dist_diff)    
    print('p for distances', p_val)
    '''
    # ==== RIGHT =====
    ax_n += 1
    peak_vals_same = []; dist_same = []
    # same left 
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [-1,-1,0,0,-1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [-1,-1,0,0,-1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
      #  segm1 = [s-segm1[-1] for s in segm1]
      #  segm2 = [s-segm2[-1] for s in segm2]

        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
     #   time_furthest_away = time[max_distance_idx]


        
        peak_t = max_distance_idx; # abs_segm.index(max(abs_segm))
        peak_vals_same.append(max(distances))
        segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'r', alpha=0.5)
        axs[ax_n].plot(segm1[0], segm2[0], 'r', marker='o', alpha=0.5)
       # axs[ax_n].plot(segm1[-1], segm2[-1], 'r', marker='D', alpha=0.5)
        # Assuming segm1 and segm2 are lists or arrays of coordinates [x1, y1] and [x2, y2]
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_same.append(distance)

    # same left 
    #ax_n += 1
    peak_vals_diff = []; dist_diff = []
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [1,-1,0,0,-1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [1,-1,0,0,-1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
     #   segm1 = [s-segm1[-1] for s in segm1]
      #  segm2 = [s-segm2[-1] for s in segm2]
        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
      #  time_furthest_away = time[max_distance_idx]

        
        peak_t = max_distance_idx #abs_segm.index(max(abs_segm))
        peak_vals_diff.append(max(distances))
        segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'm', alpha=0.5)
        axs[ax_n].plot(segm1[0], segm2[0], 'm', marker='o', alpha=0.5)
       # axs[ax_n].plot(segm1[-1], segm2[-1], 'm', marker='D', alpha=0.5)
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_diff.append(distance)
    
  #  print(np.mean(peak_vals_same), np.mean(peak_vals_diff)) 
    stat, p_val = stats.ttest_ind(peak_vals_same, peak_vals_diff)    
 #   print('p for max val R', p_val)
    table_means['side'].append(-1)
    table_means['laser'].append(False)
    table_means['is_same'].append(True)
    table_means['mean'].append(np.mean(peak_vals_same))
    table_means['side'].append(-1)
    table_means['laser'].append(False)
    table_means['is_same'].append(False)
    table_means['mean'].append(np.mean(peak_vals_diff))
    '''
    print(np.mean(dist_same), np.mean(dist_diff)) 
    stat, p_val = stats.ttest_ind(dist_same, dist_diff)    
    print('p for distances', p_val)
    '''
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # ==== LEFT ====
    ax_n += 1
    peak_vals_same = []; dist_same = []
    # same left 
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [1,1,0,1,1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [1,1,0,1,1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
       # segm1 = [s-segm1[0] for s in segm1]
      #  segm2 = [s-segm2[0] for s in segm2]

        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
     #   time_furthest_away = time[max_distance_idx]     
        peak_t = max_distance_idx; # abs_segm.index(max(abs_segm))
        peak_vals_same.append(max(distances))
        #segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'b', alpha=0.5)
       # axs[ax_n].plot(segm1[0], segm2[0], 'b', marker='o', alpha=0.5)
        axs[ax_n].plot(segm1[-1], segm2[-1], 'b', marker='D', alpha=0.5)
        # Assuming segm1 and segm2 are lists or arrays of coordinates [x1, y1] and [x2, y2]
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_same.append(distance)

    # same left 
    peak_vals_diff = []; dist_diff = []
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [-1,1,0,1,1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [-1,1,0,1,1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
      #  segm1 = [s-segm1[0] for s in segm1]
      #  segm2 = [s-segm2[0] for s in segm2]
        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
      #  time_furthest_away = time[max_distance_idx]

        
        peak_t = max_distance_idx #abs_segm.index(max(abs_segm))
        peak_vals_diff.append(max(distances))
        segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'c', alpha=0.5)
        #axs[ax_n].plot(segm1[0], segm2[0], 'c', marker='o', alpha=0.5)
        axs[ax_n].plot(segm1[-1], segm2[-1], 'c', marker='D', alpha=0.5)
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_diff.append(distance)
    
  #  print(np.mean(peak_vals_same), np.mean(peak_vals_diff)) 
    stat, p_val = stats.ttest_ind(peak_vals_same, peak_vals_diff)    
  #  print('p for max val L laser', p_val)
    table_means['side'].append(1)
    table_means['laser'].append(True)
    table_means['is_same'].append(True)
    table_means['mean'].append(np.mean(peak_vals_same))
    table_means['side'].append(1)
    table_means['laser'].append(True)
    table_means['is_same'].append(False)
    table_means['mean'].append(np.mean(peak_vals_diff))
    '''
    print(np.mean(dist_same), np.mean(dist_diff)) 
    stat, p_val = stats.ttest_ind(dist_same, dist_diff)    
    print('p for distances', p_val)
    '''
    # ==== RIGHT =====
    ax_n += 1
    peak_vals_same = []; dist_same = []
    # same left 
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [-1,-1,0,-1,-1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [-1,-1,0,-1,-1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
      #  segm1 = [s-segm1[-1] for s in segm1]
      #  segm2 = [s-segm2[-1] for s in segm2]

        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
     #   time_furthest_away = time[max_distance_idx]


        
        peak_t = max_distance_idx; # abs_segm.index(max(abs_segm))
        peak_vals_same.append(max(distances))
        segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'r', alpha=0.5)
        axs[ax_n].plot(segm1[0], segm2[0], 'r', marker='o', alpha=0.5)
       # axs[ax_n].plot(segm1[-1], segm2[-1], 'r', marker='D', alpha=0.5)
        # Assuming segm1 and segm2 are lists or arrays of coordinates [x1, y1] and [x2, y2]
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_same.append(distance)

    # same left 
    #ax_n += 1
    peak_vals_diff = []; dist_diff = []
    segms1, segm_ts1 = get_segments(behav, pc1, frame_ts, params, [1,-1,0,-1,-1], span=span)
    segms2, segm_ts2 = get_segments(behav, pc2, frame_ts, params, [1,-1,0,-1,-1], span=span)
    # each segment has to start from 0 and end at the peak
    for s_n in np.arange(len(segms1)):
        segm1 = segms1[s_n]; segm2 = segms2[s_n]
        segm1 = segm1.tolist(); segm2 = segm2.tolist()
     #   segm1 = [s-segm1[-1] for s in segm1]
      #  segm2 = [s-segm2[-1] for s in segm2]
        # Assuming segm1 is an array of x coordinates and segm2 is an array of y coordinates
        # And time is an array of corresponding time steps

        # Calculate the distance from the start (first point)
        distances = np.sqrt((np.array(segm1) - segm1[0])**2 + (np.array(segm2) - segm2[0])**2)

        # Find the index of the maximum distance
        max_distance_idx = np.argmax(distances)
        
        # The time when the line is furthest away
      #  time_furthest_away = time[max_distance_idx]

        
        peak_t = max_distance_idx #abs_segm.index(max(abs_segm))
        peak_vals_diff.append(max(distances))
        segm1 = segm1[:peak_t]; segm2 = segm2[:peak_t]
        axs[ax_n].plot(segm1, segm2, 'm', alpha=0.5)
        axs[ax_n].plot(segm1[0], segm2[0], 'm', marker='o', alpha=0.5)
       # axs[ax_n].plot(segm1[-1], segm2[-1], 'm', marker='D', alpha=0.5)
        segm1 = np.array(segm1); segm2 = np.array(segm2)
        # Calculate the Euclidean distance
        distance = np.linalg.norm(segm2 - segm1); dist_diff.append(distance)
    
   # print(np.mean(peak_vals_same), np.mean(peak_vals_diff)) 
    stat, p_val = stats.ttest_ind(peak_vals_same, peak_vals_diff)    
   # print('p for max val R laser', p_val)
    table_means['side'].append(-1)
    table_means['laser'].append(True)
    table_means['is_same'].append(True)
    table_means['mean'].append(np.mean(peak_vals_same))
    table_means['side'].append(-1)
    table_means['laser'].append(True)
    table_means['is_same'].append(False)
    table_means['mean'].append(np.mean(peak_vals_diff))
    '''
    print(np.mean(dist_same), np.mean(dist_diff)) 
    stat, p_val = stats.ttest_ind(dist_same, dist_diff)    
    print('p for distances', p_val)
    '''
    table_means = pd.DataFrame(table_means)
  #  print(table_means)
    data[id]['table_means'] = table_means

    
    plt.show()



In [ ]:
all_sl = []
all_dl = []
all_sr = []
all_dr = []
same_nolas = []; same_las = []
diff_nolas = []; diff_las = []
for id in mice_ids:
    table_means = data[id]['table_means']
    same_l = table_means[(table_means['is_same']==True) & (table_means['side']==1)]['mean'].tolist()
    same_r = table_means[(table_means['is_same']==True) & (table_means['side']==-1)]['mean'].tolist()
    diff_l = table_means[(table_means['is_same']==False) & (table_means['side']==1)]['mean'].tolist()
    diff_r = table_means[(table_means['is_same']==False) & (table_means['side']==-1)]['mean'].tolist()
   # print(same_las, 'vs', same_nolas)
    plt.plot([0,1], same_l, 'b')
    plt.plot([0,1], same_r, 'r')
    plt.plot([2,3], diff_l, 'c')
    plt.plot([2,3], diff_r, 'm')
    all_sl.append(same_l); all_sr.append(same_r); 
    all_dl.append(diff_l); all_dr.append(diff_r); 
    same_nolas.append(same_l[0]); same_nolas.append(same_r[0]); 
    same_las.append(same_l[1]); same_las.append(same_r[1]); 
    diff_nolas.append(diff_l[0]); diff_nolas.append(diff_r[0]); 
    diff_las.append(diff_l[1]); diff_las.append(diff_r[1]); 

#fig = plt.figure()
plt.plot([0,1], np.mean(all_sl, axis=0), 'b', linewidth=6)
plt.plot([0,1], np.mean(all_sr, axis=0), 'r', linewidth=6)

plt.plot([2,3], np.mean(all_dl, axis=0), 'c', linewidth=6)
plt.plot([2,3], np.mean(all_dr, axis=0), 'm', linewidth=6)
plt.show()
# do the ttest
stat, p_val = stats.ttest_ind(same_nolas, same_las)    
print('p for ', p_val)
stat, p_val = stats.ttest_ind(diff_nolas, diff_las)    
print('p for ', p_val)


# Perform Wilcoxon signed-rank test
stat, p_val = wilcoxon(same_nolas, same_las)
print(f"Wilcoxon signed-rank test results: statistic = {stat}, p-value = {round(p_val,7)}")
# Perform Wilcoxon signed-rank test
stat, p_val = wilcoxon(diff_nolas, diff_las)
print(f"Wilcoxon signed-rank test results: statistic = {stat}, p-value = {round(p_val,7)}")

In [ ]:
same_nolas

In [ ]:
all_sl = []
all_dl = []
all_sr = []
all_dr = []
same_nolas = []; same_las = []
diff_nolas = []; diff_las = []
for id in mouse_ids[:-1]:
    table_means = data[id]['table_means']
    same_l = table_means[(table_means['is_same']==True) & (table_means['side']==1)]['mean'].tolist()
    same_r = table_means[(table_means['is_same']==True) & (table_means['side']==-1)]['mean'].tolist()
    diff_l = table_means[(table_means['is_same']==False) & (table_means['side']==1)]['mean'].tolist()
    diff_r = table_means[(table_means['is_same']==False) & (table_means['side']==-1)]['mean'].tolist()
   # print(same_las, 'vs', same_nolas)
    plt.plot([0,1], same_l, 'b')
    plt.plot([0,1], same_r, 'r')
    plt.plot([2,3], diff_l, 'c')
    plt.plot([2,3], diff_r, 'm')
    all_sl.append(same_l); all_sr.append(same_r); 
    all_dl.append(diff_l); all_dr.append(diff_r); 
    same_nolas.append(same_l[0]); same_nolas.append(same_r[0]); 
    same_las.append(same_l[1]); same_las.append(same_r[1]); 
    diff_nolas.append(diff_l[0]); diff_nolas.append(diff_r[0]); 
    diff_las.append(diff_l[1]); diff_las.append(diff_r[1]); 

#fig = plt.figure()
plt.plot([0,1], np.mean(all_sl, axis=0), 'b', linewidth=6)
plt.plot([0,1], np.mean(all_sr, axis=0), 'r', linewidth=6)

plt.plot([2,3], np.mean(all_dl, axis=0), 'c', linewidth=6)
plt.plot([2,3], np.mean(all_dr, axis=0), 'm', linewidth=6)
plt.show()
# do the ttest
stat, p_val = stats.ttest_ind(same_nolas, same_las)    
print('p for ', p_val)
stat, p_val = stats.ttest_ind(diff_nolas, diff_las)    
print('p for ', p_val)


# Perform Wilcoxon signed-rank test
stat, p_val = wilcoxon(same_nolas, same_las)
print(f"Wilcoxon signed-rank test results: statistic = {stat}, p-value = {round(p_val,7)}")
# Perform Wilcoxon signed-rank test
stat, p_val = wilcoxon(diff_nolas, diff_las)
print(f"Wilcoxon signed-rank test results: statistic = {stat}, p-value = {round(p_val,7)}")

In [ ]:
all_sl = []
all_dl = []
all_sr = []
all_dr = []
for id in mouse_ids[1:]:
    table_means = data[id]['table_means']
    same_l = table_means[(table_means['is_same']==True) & (table_means['side']==1)]['mean'].tolist()
    same_r = table_means[(table_means['is_same']==True) & (table_means['side']==-1)]['mean'].tolist()
    diff_l = table_means[(table_means['is_same']==False) & (table_means['side']==1)]['mean'].tolist()
    diff_r = table_means[(table_means['is_same']==False) & (table_means['side']==-1)]['mean'].tolist()
   # print(same_las, 'vs', same_nolas)
    plt.plot([0,1], same_l, 'b')
    plt.plot([0,1], same_r, 'r')
    plt.plot([2,3], diff_l, 'c')
    plt.plot([2,3], diff_r, 'm')
    all_sl.append(same_l); all_sr.append(same_r); 
    all_dl.append(diff_l); all_dr.append(diff_r); 

#fig = plt.figure()
plt.plot([0,1], np.mean(all_sl, axis=0), 'b', linewidth=6)
plt.plot([0,1], np.mean(all_sr, axis=0), 'r', linewidth=6)

plt.plot([2,3], np.mean(all_dl, axis=0), 'c', linewidth=6)
plt.plot([2,3], np.mean(all_dr, axis=0), 'm', linewidth=6)
plt.show()

In [ ]:
all_sl = []
all_dl = []
all_sr = []
all_dr = []
for id in ['os86a']:
    table_means = data[id]['table_means']
    same_l = table_means[(table_means['is_same']==True) & (table_means['side']==1)]['mean'].tolist()
    same_r = table_means[(table_means['is_same']==True) & (table_means['side']==-1)]['mean'].tolist()
    diff_l = table_means[(table_means['is_same']==False) & (table_means['side']==1)]['mean'].tolist()
    diff_r = table_means[(table_means['is_same']==False) & (table_means['side']==-1)]['mean'].tolist()
   # print(same_las, 'vs', same_nolas)
    plt.plot([0,1], same_l, 'b')
    plt.plot([0,1], same_r, 'r')
    plt.plot([2,3], diff_l, 'c')
    plt.plot([2,3], diff_r, 'm')
    all_sl.append(same_l); all_sr.append(same_r); 
    all_dl.append(diff_l); all_dr.append(diff_r); 

#fig = plt.figure()
plt.plot([0,1], np.mean(all_sl, axis=0), 'b', linewidth=6)
plt.plot([0,1], np.mean(all_sr, axis=0), 'r', linewidth=6)

plt.plot([2,3], np.mean(all_dl, axis=0), 'c', linewidth=6)
plt.plot([2,3], np.mean(all_dr, axis=0), 'm', linewidth=6)
plt.show()

## Time to calculate distances

In [ ]:
# Start with calculating the distanecs btw left and right and verifying it with permutation
coords_av = {'left_side_nolas': [], 'left_side_las': []}
coords_av_permut = {'left_side': [], 'right_side': []}
param_val_keys = {'left_side': {'params': ['side', 'laser', 'missed'], 'vals': [1,0,0]}, 
              'right_side': {'params': ['side', 'laser', 'missed'], 'vals': [-1,0,0]},
              'left_side_nolas': {'params': ['side', 'laser', 'missed'], 'vals': [1,0,0]}, 
              'left_side_las': {'params': ['side', 'laser', 'missed'], 'vals': [1,1,0]} }
w = [9,17]
span = [0,30]
icas = ica_df_time_points.keys()
for key in coords_av.keys():
    params = param_val_keys[key]['params']
    vals = param_val_keys[key]['vals']
    coords_cond = [] # all the coordinates of the given condition
    # select segments based on the parameters and values specified
    for ica_n in icas:
        comp = ica_df_time_points[ica_n]
        traces = get_segments_old(data_exp, params, vals, n, frame_type='frame_tone', span=span, is_pca=True, pc=comp)
        coords_n = []
        for tr_n in np.arange(len(traces)):
            coord = np.mean(traces[tr_n][span[0]+w[0]:span[0]+w[1]])
            coords_n.append(coord)
        coord_n_av = np.mean(coords_n)
        coords_cond.append(coord_n_av)
   # coords_cond = np.array(coords_cond).T
    coords_av[key] = coords_cond
        
        
        
# Compute the Euclidean distance btw the averaged coordinates
distance = np.sqrt(np.sum((np.array(coords_av['left_side_nolas']) - np.array(coords_av['left_side_las']))**2))

print(f'The distance between {str(coords_av.keys())[11:-2]} is {distance}')

In [ ]:
behav

In [ ]:


# same thing as above but as a function

def distance_btw_comp(condtions, data_exp, ca_df_time_points, w=[9, 17], span=[0, 30]):
    coords_av = {condtions[0]:[], condtions[1]:[]}
    

    cas = ca_df_time_points.keys()
    
    for key in coords_av.keys():
        params = param_val_keys[key]['params']
        vals = param_val_keys[key]['vals']
        coords_cond = []
        
        for ca_n in cas:
            comp = ca_df_time_points[ca_n]
            traces = get_segments_old(data_exp, params, vals, n, frame_type='frame_tone', span=span, is_pca=True, pc=comp)
            coords_n = [np.mean(trace[span[0]+w[0]:span[0]+w[1]]) for trace in traces]
            coord_n_av = np.mean(coords_n)
            coords_cond.append(coord_n_av)
        
        coords_av[key] = coords_cond
    
    distance = np.sqrt(np.sum((np.array(coords_av[condtions[0]]) - np.array(coords_av[condtions[1]]))**2))
    return distance




In [ ]:
# mouse 86 on 2024-05-26
# For prev. conrtast laser vs no laser
condtions_pairs = [
                    ['left_side', 'right_side'],
                    ['left_side_same_nolas', 'left_side_diff_nolas'],
                    ['right_side_same_nolas', 'right_side_diff_nolas'],
                    ['left_side_same_las', 'left_side_diff_las'],
                    ['right_side_same_las', 'right_side_diff_las'],
                  ]

# ok lets do ica for all the mice
for id in mouse_ids:
    dates = data[id].keys()
    #if a:
    for date in dates:
        print('=== ' + id + ' ' + date + ' ===')
        data_exp = data[id][date]
        ica_df_time_points = data_exp['ICA']
        for conditions in condtions_pairs:
            distance = distance_btw_comp(conditions, data_exp, ica_df_time_points, w=[8,14])
            print(f'The distance between {conditions[0]} and {conditions[1]} is {distance}')

In [ ]:

for conditions in condtions_pairs:
    distance = distance_btw_comp(conditions, data_exp, ica_df_time_points, w=[8,14])
    print(f'The distance between {conditions[0]} and {conditions[1]} is {distance}')

In [ ]:
# Example usage:
condtions_example = ['left_side_nolas', 'left_side_las']
data_exp_example = data_exp  # Replace with your data
ica_df_time_points_example = ica_df_time_points  # Replace with your data

distance = distance_btw_comp(condtions_example, data_exp_example, ica_df_time_points_example)
print(f'The distance between {condtions_example[0]} and {condtions_example[1]} is {distance}')

In [ ]:
# Same but with laser
    
pc = ica_df_time_points['IC4']
n = 99
params = ['side', 'prev_reward', 'laser']
traces_pc1 = {
    'l_same_nolas': get_segments_old(data_exp, params, [1, 1, 0], n, frame_type='frame_tone', is_pca=True, pc=pc),
    'l_diff_nolas': get_segments_old(data_exp, params, [1, -1, 0], n, frame_type='frame_tone', is_pca=True, pc=pc),
    'l_same_las': get_segments_old(data_exp, params, [1, 1, 1], n, frame_type='frame_tone', is_pca=True, pc=pc),
    'l_diff_las': get_segments_old(data_exp, params, [1, -1, 1], n, frame_type='frame_tone', is_pca=True, pc=pc),
}

pc = ica_df_time_points['IC7']
traces_pc2 = {
    'l_same_nolas': get_segments_old(data_exp, params, [1, 1, 0], n, frame_type='frame_tone', is_pca=True, pc=pc),
    'l_diff_nolas': get_segments_old(data_exp, params, [1, -1, 0], n, frame_type='frame_tone', is_pca=True, pc=pc),
    'l_same_las': get_segments_old(data_exp, params, [1, 1, 1], n, frame_type='frame_tone', is_pca=True, pc=pc),
    'l_diff_las': get_segments_old(data_exp, params, [1, -1, 1], n, frame_type='frame_tone', is_pca=True, pc=pc),
}

marker_specs = {'l_same_nolas': {'color':'b', 'marker':'o'},
                'l_diff_nolas':{'color':'c', 'marker':'o'},
                'l_same_las': {'color':'b', 'marker':'D'},
                'l_diff_las': {'color':'c', 'marker':'D'}}

# t_windows = [ [0,5], [6,10], [11,15], [16,20]]
t_windows = [[t, t + 1] for t in np.arange(0, 30, 5)]

# setup the plot
fig, axs = plt.subplots(1, len(t_windows), figsize=(15, 2))
ax_n = 0

for w in t_windows:
    for key in traces_pc1.keys():
        segms_pc1 = traces_pc1[key]
        segms_pc2 = traces_pc2[key]
        avs_pc1 = []
        avs_pc2 = []
        color = marker_specs[key]['color']
        marker = marker_specs[key]['marker']
    
        for segment_n in np.arange(len(segms_pc1)):
            segm_pc1 = traces_pc1[key][segment_n]
            segm_pc2 = traces_pc2[key][segment_n]
            av_pc1 = np.mean(segm_pc1[w[0]:w[1]])
            av_pc2 = np.mean(segm_pc2[w[0]:w[1]])
            avs_pc1.append(av_pc1)
            avs_pc2.append(av_pc2)
            
            axs[ax_n].plot(av_pc1, av_pc2, color=color, marker=marker, markersize=3, alpha=0.2)
        av_segms_1 = np.mean(avs_pc1)
        av_segms_2 = np.mean(avs_pc2)
        axs[ax_n].plot(av_segms_1,av_segms_2, color=color, marker=marker, markersize=12,markeredgewidth=2,fillstyle='none')

    ax_n += 1

plt.show()


In [ ]:


for id in mouse_ids:
    dates = data[id].keys()
    #if a:
    for date in dates:
        print(f"::: Mouse  {id}, {date} :::")

        data_exp = data[id][date]
        traces = data_exp['deconv'] 
        behav = data_exp['behav']
        params = ['side', 'prev_choice', 'laser']
        
        all_aucs = []
        for neu in np.arange(traces.shape[0]):
            # if the neurons is not good or fair, skip it
            if data_exp['good_neurons'][neu] + data_exp['fair_neurons'][neu] == 0:
                continue
            
            trace = traces[neu,:]
            aucs = []
            for trial_n in np.arange(behav.shape[0]):
                f = behav['frame_tone'][trial_n]
                if f > traces.shape[1]:
                    continue
                auc = sum(trace[f+10:f+20])
                aucs.append(auc)
            if len(all_aucs) == 0:
                all_aucs = aucs
            else:
                all_aucs = np.column_stack((all_aucs,aucs))
        # Perform PCA to reduce from 60 to 2 dimensions
        pca = PCA(n_components=10)
        pca_result = pca.fit_transform(all_aucs)
        
        # Create a DataFrame for easier handling
        columns=['PC1', 'PC2','PC3', 'PC4']
        laser_conds = [str(behav['prev_choice'][n]) + str(behav['choice'][n]) for n in np.arange(all_aucs.shape[0])] 
        pcaa_df = pd.DataFrame(pca_result[:,0:4], columns=columns[0:4] ) #, 'PC3', 'PC4', 'PC5', 'PC6','PC7', 'PC8', 'PC9'])
        pcaa_df['TrialType'] =  behav['choice'][0:all_aucs.shape[0]] #laser_conds
        pcaa_df['laser'] = behav['laser'][0:all_aucs.shape[0]]
        pcaa_df['side'] = behav['side'][0:all_aucs.shape[0]]
        pcaa_df['choice'] = behav['choice'][0:all_aucs.shape[0]]
        pcaa_df['prev_choice'] = behav['prev_choice'][0:all_aucs.shape[0]]
        
        # Plot the PCA results
        fig = plt.figure(figsize=(10, 6))
        ax = fig.add_subplot(111, projection='3d')

        pca_l_same = pcaa_df[(pcaa_df['side']==1) & (pcaa_df['prev_choice']==1) & (pcaa_df['laser']==0)]
        pca_l_diff = pcaa_df[(pcaa_df['side']==1) & (pcaa_df['prev_choice']==1) & (pcaa_df['laser']==1)]
        pca_r_same = pcaa_df[(pcaa_df['side']==1) & (pcaa_df['prev_choice']==-1) & (pcaa_df['laser']==0)]
        pca_r_diff = pcaa_df[(pcaa_df['side']==1) & (pcaa_df['prev_choice']==-1) & (pcaa_df['laser']==1)]
        ax.scatter(pca_l_same['PC1'], pca_l_same['PC2'], pca_l_same['PC3'], label='left same no laser')
        ax.scatter(pca_l_diff['PC1'], pca_l_diff['PC2'], pca_l_diff['PC2'], label='left same laser')
        ax.scatter(pca_r_same['PC1'], pca_r_same['PC2'], pca_r_same['PC3'], label='left diff no laser')
        ax.scatter(pca_r_diff['PC1'], pca_r_diff['PC2'], pca_r_diff['PC3'], label='left diff laser')
        
        #plt.scatter(pca_df['PC1'], pca_df['PC2'], label=trial_type)
        plt.title('PCA of Neuronal Activity (AUC Features)')
        plt.xlabel('Principal Component 1')
        plt.ylabel('Principal Component 2')
        plt.legend()
        plt.show()

In [ ]:
for id in mouse_ids:
    dates = data[id].keys()
    #if a:
    for date in dates:
        print(f"::: Mouse  {id}, {date} :::")

        data_exp = data[id][date]
        traces = data_exp['ca'] 
        behav = data_exp['behav']
        params = ['side', 'prev_choice', 'laser']
        
        all_aucs = []
        for neu in np.arange(traces.shape[0]):
            # if the neurons is not good or fair, skip it
            if data_exp['good_neurons'][neu] == 0: # + data_exp['fair_neurons'][neu] == 0:

                continue
            
            trace = traces[neu,:]

            aucs = []
            for trial_n in np.arange(behav.shape[0]):
                f = behav['frame_tone'][trial_n]
                if f > traces.shape[1]:
                    continue
                auc = sum(trace[f+5:f+25])
                aucs.append(auc)
            if len(all_aucs) == 0:
                all_aucs = aucs
            else:
                all_aucs = np.column_stack((all_aucs,aucs))
        # Perform PCA to reduce from 60 to 2 dimensions
        pca = PCA(n_components=10)
        pca_result = pca.fit_transform(all_aucs)
        
        # Create a DataFrame for easier handling
        columns=['PC1', 'PC2','PC3', 'PC4']
        laser_conds = [behav['laser'][n] + behav['choice'][n] for n in np.arange(all_aucs.shape[0])] 
        pcaa_df = pd.DataFrame(pca_result[:,0:4], columns=columns[0:4] ) #, 'PC3', 'PC4', 'PC5', 'PC6','PC7', 'PC8', 'PC9'])
        pcaa_df['TrialType'] = behav['side'][0:all_aucs.shape[0]] 
        pcaa_df['laser'] = behav['laser'][0:all_aucs.shape[0]]
        pcaa_df['side'] = behav['side'][0:all_aucs.shape[0]]
        pcaa_df['choice'] = behav['choice'][0:all_aucs.shape[0]]
        pcaa_df['prev_choice'] = behav['prev_choice'][0:all_aucs.shape[0]]
        
        # Plot the PCA results
        plt.figure(figsize=(8, 4))
        for trial_type in pcaa_df['TrialType'].unique():
            subset = pcaa_df[pcaa_df['TrialType'] == trial_type]
            plt.scatter(subset['PC1'], subset['PC2'], label=trial_type)
        #plt.scatter(pca_df['PC1'], pca_df['PC2'], label=trial_type)
        plt.title('PCA of Neuronal Activity (AUC Features)')
        plt.xlabel('Principal Component 1')
        plt.ylabel('Principal Component 2')
        plt.legend()
        plt.show()

In [ ]:
for id in mouse_ids:
    dates = data[id].keys()
    #if a:
    for date in dates:
        print(f"::: Mouse  {id}, {date} :::")

        data_exp = data[id][date]
        traces = data_exp['ca'] 
        behav = data_exp['behav']
        params = ['side', 'prev_choice', 'laser']
        
        all_aucs = []
        for neu in np.arange(traces.shape[0]):
            # if the neurons is not good or fair, skip it
            if data_exp['good_neurons'][neu] == 0: # + data_exp['fair_neurons'][neu] == 0:

                continue
            
            trace = traces[neu,:]

            aucs = []
            for trial_n in np.arange(behav.shape[0]):
                f = behav['frame_tone'][trial_n]
                if f > traces.shape[1]:
                    continue
                auc = sum(trace[f+5:f+25])
                aucs.append(auc)
            if len(all_aucs) == 0:
                all_aucs = aucs
            else:
                all_aucs = np.column_stack((all_aucs,aucs))
        # Perform PCA to reduce from 60 to 2 dimensions
        pca = PCA(n_components=10)
        pca_result = pca.fit_transform(all_aucs)
        
        # Create a DataFrame for easier handling
        columns=['PC1', 'PC2','PC3', 'PC4']
        laser_conds = [behav['laser'][n] + behav['choice'][n] for n in np.arange(all_aucs.shape[0])] 
        pcaa_df = pd.DataFrame(pca_result[:,0:4], columns=columns[0:4] ) #, 'PC3', 'PC4', 'PC5', 'PC6','PC7', 'PC8', 'PC9'])
        pcaa_df['TrialType'] = behav['side'][0:all_aucs.shape[0]] 
        pcaa_df['laser'] = behav['laser'][0:all_aucs.shape[0]]
        pcaa_df['side'] = behav['side'][0:all_aucs.shape[0]]
        pcaa_df['choice'] = behav['choice'][0:all_aucs.shape[0]]
        pcaa_df['prev_choice'] = behav['prev_choice'][0:all_aucs.shape[0]]
        
        # Plot the PCA results
        plt.figure(figsize=(8, 4))
        for trial_type in pcaa_df['TrialType'].unique():
            subset = pcaa_df[pcaa_df['TrialType'] == trial_type]
            plt.scatter(subset['PC3'], subset['PC2'], label=trial_type)
        #plt.scatter(pca_df['PC1'], pca_df['PC2'], label=trial_type)
        plt.title('PCA of Neuronal Activity (AUC Features)')
        plt.xlabel('Principal Component 1')
        plt.ylabel('Principal Component 2')
        plt.legend()
        plt.show()

In [ ]:
# Perform PCA to reduce from 60 to 2 dimensions
pca = PCA(n_components=10)
pca_result = pca.fit_transform(all_aucs)

# Create a DataFrame for easier handling
columns=['PC1', 'PC2','PC3', 'PC4']

pcaa_df = pd.DataFrame(pca_result[:,0:4], columns=columns[0:4] ) #, 'PC3', 'PC4', 'PC5', 'PC6','PC7', 'PC8', 'PC9'])
pcaa_df['TrialType'] = behav['prev_choice'][0:all_aucs.shape[0]]
pcaa_df['laser'] = behav['laser'][0:all_aucs.shape[0]]
pcaa_df['side'] = behav['side'][0:all_aucs.shape[0]]
pcaa_df['choice'] = behav['choice'][0:all_aucs.shape[0]]
pcaa_df['prev_choice'] = behav['prev_choice'][0:all_aucs.shape[0]]

# Plot the PCA results
plt.figure(figsize=(10, 6))
for trial_type in pcaa_df['TrialType'].unique():
    subset = pcaa_df[pcaa_df['TrialType'] == trial_type]
    plt.scatter(subset['PC1'], subset['PC2'], label=trial_type)
#plt.scatter(pca_df['PC1'], pca_df['PC2'], label=trial_type)
plt.title('PCA of Neuronal Activity (AUC Features)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend()
plt.show()

In [ ]:
tracw

In [ ]:
behav

In [ ]:

# Example data: rows are time points, columns are neurons
id = 'os86'
date = '2024-06-03'
data_exp = data[id][date]
traces = data_exp['deconv'] 
behav = data_exp['behav']
params = ['side', 'prev_choice', 'laser']

for neu in np.arange(traces.shape[0]):
    # if the neurons is not good or fair, skip it
    if data_exp['good_neurons'][neu] == 0:
        continue
    trace = traces[neu,:]
    
    aucs = []
    for trial_n in behav.shape[0]:
        f = behav['frame_tone']
        if f > traces.shape[1]-11:
            continue
        auc = sum(trace[f:f+10])
        aucs.append(auc)

    


traces_left_same = get_segments_old(data_exp, params, [-1,-1,0], n, frame_type='frame_tone', 
                        trace_type='deconv') 
traces_left_diff = get_segments_old(data_exp, params, [-1,1,0], n, frame_type='frame_tone', 
                        trace_type='deconv') 
pre_tone = [0, 10]
post_tone = [10, 20]






# Extract AUC features for each trial
left_same_aucs = []
left_diff_aucs = []

for trial_n in np.arange(traces_left_same.shape[0]):  # Assuming each column is a trial
    signal = traces_left_same[trial_n,:]
    auc_pre = sum(signal[0:10])
    auc_post = sum(signal[10:20])
    left_same_aucs.append([auc_pre, auc_post])

for trial_n in np.arange(traces_left_diff.shape[0]):  # Assuming each column is a trial
    signal = traces_left_diff[trial_n,:]
    auc_pre = sum(signal[0:10])
    auc_post = sum(signal[10:20])
    left_diff_aucs.append([auc_pre, auc_post])
    

# Perform PCA to reduce from 60 to 2 dimensions
pca = PCA(n_components=2)
pca_result = pca.fit_transform(all_aucs)

# Create a DataFrame for easier handling
pca_df = pd.DataFrame(pca_result, columns=['PC1', 'PC2'])
pca_df['TrialType'] = trial_labels

# Plot the PCA results
plt.figure(figsize=(10, 6))
for trial_type in pca_df['TrialType'].unique():
    subset = pca_df[pca_df['TrialType'] == trial_type]
    plt.scatter(subset['PC1'], subset['PC2'], label=trial_type)

plt.title('PCA of Neuronal Activity (AUC Features)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend()
plt.show()

import ace_tools as tools; tools.display_dataframe_to_user(name="PCA of Neuronal Activity", dataframe=pca_df)




